# Import libraries

In [1]:
import pypots
import os
import sys
from pypots.nn.functional import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012 as preprocess_physionet2012
from missingData.toolkits import toolkits
import math
import scipy.stats as st

2025-05-01 07:56:30.259123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746096990.282509 1240605 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746096990.288803 1240605 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-01 07:56:30.309283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidge


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Loading database

### Standard Scaler 

In [2]:
set_random_seed()
physionet2012_dataset_standard = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)
print(physionet2012_dataset_standard.keys())

2025-05-01 07:56:33 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-05-01 07:56:33 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-05-01 07:56:33 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-05-01 07:56:33 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-05-01 07:56:33 [INFO]: Loaded successfully!
2025-05-01 07:56:51 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-05-01 07:56:51 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-05-01 07:56:51 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
scaler_standard = physionet2012_dataset_standard["scaler"]

In [4]:
dataset_for_training_standard, dataset_for_validating_standard, dataset_for_testing_ori_standard, dataset_for_testing_standard = toolkits.separating_dataset(physionet2012_dataset_standard)

# Creating indicating mask and get test_X_ori

### Standard Scaler

In [5]:
dataset_for_testing_ori_standard_norm = toolkits.dict_to_list(dataset_for_testing_ori_standard)
dataset_for_testing_standard_norm = toolkits.dict_to_list(dataset_for_testing_standard)
indicating_mask_variable_standard_norm, test_X_ori_variable_standard_norm = toolkits.components_mae(dataset_for_testing_ori_standard_norm, dataset_for_testing_standard_norm)
indicating_mask_variable_standard_norm = toolkits.pre_reshape(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.pre_reshape(test_X_ori_variable_standard_norm)
indicating_mask_variable_standard_norm = toolkits.reshape_variable(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.reshape_variable(test_X_ori_variable_standard_norm)

# Inicialize the models

### SAITS

<h4>Inicialize new model</h4>

In [6]:
saits = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 21:54:46 [INFO]: No given device, using default device: cuda
2025-04-22 21:54:46 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250422_T215446
2025-04-22 21:54:46 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250422_T215446/tensorboard
2025-04-22 21:54:46 [INFO]: Using customized MAE as the training loss function.
2025-04-22 21:54:46 [INFO]: Using customized MSE as the validation metric function.
2025-04-22 21:54:46 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize model</h4>

In [6]:
saits = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-01 07:57:04 [INFO]: No given device, using default device: cpu
2025-05-01 07:57:04 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-01 07:57:04 [INFO]: Using customized MAE as the training loss function.
2025-05-01 07:57:04 [INFO]: Using customized MSE as the validation metric function.
2025-05-01 07:57:04 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


### BRITS

<h4>Inicialize new model</h4>

In [7]:
brits = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: Using customized MAE as the training loss function.
2025-04-22 18:16:43 [INFO]: Using customized MSE as the validation metric function.
2025-04-22 18:16:43 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize model</h4>

In [7]:
brits = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-01 07:57:04 [INFO]: No given device, using default device: cpu
2025-05-01 07:57:04 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-01 07:57:04 [INFO]: Using customized MAE as the training loss function.
2025-05-01 07:57:04 [INFO]: Using customized MSE as the validation metric function.
2025-05-01 07:57:04 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


### USGAN

<h4>Inicialize new model</h4>

In [8]:
us_gan = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize model</h4>

In [8]:
us_gan = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-01 07:57:04 [INFO]: No given device, using default device: cpu
2025-05-01 07:57:04 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-01 07:57:04 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


### GPVAE

<h4>Inicialize new model</h4>

In [9]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize model</h4>

In [9]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-01 07:57:04 [INFO]: No given device, using default device: cpu
2025-05-01 07:57:04 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-01 07:57:04 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


### MRNN

<h4>Inicialize new model</h4>

In [10]:
mrnn = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: Using customized RMSE as the training loss function.
2025-04-22 18:16:43 [INFO]: Using customized MSE as the validation metric function.
2025-04-22 18:16:43 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize model</h4>

In [10]:
mrnn = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-01 07:57:04 [INFO]: No given device, using default device: cpu
2025-05-01 07:57:04 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-01 07:57:04 [INFO]: Using customized RMSE as the training loss function.
2025-05-01 07:57:04 [INFO]: Using customized MSE as the validation metric function.
2025-05-01 07:57:04 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


# Train/Load the models

### SAITS

<h4>Train</h4>

In [7]:
saits.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 21:54:58 [INFO]: Epoch 001 - training loss (MAE): 0.7233, validation MSE: 6.7652
2025-04-22 21:55:03 [INFO]: Epoch 002 - training loss (MAE): 0.5383, validation MSE: 6.7408
2025-04-22 21:55:08 [INFO]: Epoch 003 - training loss (MAE): 0.4956, validation MSE: 6.7218
2025-04-22 21:55:12 [INFO]: Epoch 004 - training loss (MAE): 0.4643, validation MSE: 6.6987
2025-04-22 21:55:16 [INFO]: Epoch 005 - training loss (MAE): 0.4392, validation MSE: 6.6806
2025-04-22 21:55:21 [INFO]: Epoch 006 - training loss (MAE): 0.4207, validation MSE: 6.6686
2025-04-22 21:55:24 [INFO]: Epoch 007 - training loss (MAE): 0.4065, validation MSE: 6.6527
2025-04-22 21:55:28 [INFO]: Epoch 008 - training loss (MAE): 0.3953, validation MSE: 6.6548
2025-04-22 21:55:32 [INFO]: Epoch 009 - training loss (MAE): 0.3866, validation MSE: 6.6511
2025-04-22 21:55:36 [INFO]: Epoch 010 - training loss (MAE): 0.3798, validation MSE: 6.6584
2025-04-22 21:55:36 [INFO]: Finished training. The best model is from epoch#9.
2

<h4> Load</h4>

In [11]:
saits.load("tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots")

2025-05-01 07:57:06 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots


### BRITS

<h4>Train</h4>

In [12]:
brits.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 18:20:14 [INFO]: Epoch 001 - training loss (MAE): 0.9371, validation MSE: 6.7840
2025-04-22 18:22:18 [INFO]: Epoch 002 - training loss (MAE): 0.7328, validation MSE: 6.7456
2025-04-22 18:24:23 [INFO]: Epoch 003 - training loss (MAE): 0.6822, validation MSE: 6.7326
2025-04-22 18:26:17 [INFO]: Epoch 004 - training loss (MAE): 0.6576, validation MSE: 6.7294
2025-04-22 18:28:36 [INFO]: Epoch 005 - training loss (MAE): 0.6427, validation MSE: 6.7280
2025-04-22 18:30:47 [INFO]: Epoch 006 - training loss (MAE): 0.6318, validation MSE: 6.7267
2025-04-22 18:32:52 [INFO]: Epoch 007 - training loss (MAE): 0.6229, validation MSE: 6.7294
2025-04-22 18:35:07 [INFO]: Epoch 008 - training loss (MAE): 0.6149, validation MSE: 6.7316
2025-04-22 18:37:26 [INFO]: Epoch 009 - training loss (MAE): 0.6087, validation MSE: 6.7332
2025-04-22 18:37:26 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-04-22 18:37:26 [INFO]: Finished training. The best model is from epoc

<h4>Load</h4>

In [12]:
brits.load("tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots")

2025-05-01 07:57:08 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots


### USGAN

<h4>Train</h4>

In [13]:
us_gan.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 18:41:35 [INFO]: Epoch 001 - generator training loss: 0.4346, discriminator training loss: 0.1907, validation loss: 6.7750
2025-04-22 18:45:20 [INFO]: Epoch 002 - generator training loss: 0.3701, discriminator training loss: 0.0566, validation loss: 6.7282
2025-04-22 18:49:07 [INFO]: Epoch 003 - generator training loss: 0.3347, discriminator training loss: 0.0374, validation loss: 6.7004
2025-04-22 18:52:56 [INFO]: Epoch 004 - generator training loss: 0.3165, discriminator training loss: 0.0311, validation loss: 6.6837
2025-04-22 18:56:43 [INFO]: Epoch 005 - generator training loss: 0.3020, discriminator training loss: 0.0281, validation loss: 6.6737
2025-04-22 19:00:32 [INFO]: Epoch 006 - generator training loss: 0.2923, discriminator training loss: 0.0265, validation loss: 6.6685
2025-04-22 19:04:19 [INFO]: Epoch 007 - generator training loss: 0.2877, discriminator training loss: 0.0252, validation loss: 6.6691
2025-04-22 19:08:07 [INFO]: Epoch 008 - generator training los

<h4>Load</h4>

In [13]:
us_gan.load("tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots")

2025-05-01 07:57:11 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots


### GPVAE

<h4>Train</h4>

In [14]:
gp_vae.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 19:15:55 [INFO]: Epoch 001 - training loss (default): 26027.5423, validation loss: 22882.8125
2025-04-22 19:16:01 [INFO]: Epoch 002 - training loss (default): 22877.1633, validation loss: 22807.7179
2025-04-22 19:16:07 [INFO]: Epoch 003 - training loss (default): 22842.6190, validation loss: 22788.8947
2025-04-22 19:16:14 [INFO]: Epoch 004 - training loss (default): 22832.1135, validation loss: 22783.6572
2025-04-22 19:16:20 [INFO]: Epoch 005 - training loss (default): 22827.5731, validation loss: 22780.0771
2025-04-22 19:16:27 [INFO]: Epoch 006 - training loss (default): 22821.9797, validation loss: 22773.8286
2025-04-22 19:16:34 [INFO]: Epoch 007 - training loss (default): 22814.3062, validation loss: 22767.3447
2025-04-22 19:16:40 [INFO]: Epoch 008 - training loss (default): 22810.7426, validation loss: 22771.6522
2025-04-22 19:16:46 [INFO]: Epoch 009 - training loss (default): 22808.6126, validation loss: 22762.6405
2025-04-22 19:16:52 [INFO]: Epoch 010 - training loss (

<h4>Load</h4>

In [14]:
gp_vae.load("tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots")

2025-05-01 07:57:15 [INFO]: Model loaded successfully from tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots


### MRNN

<h4>Train</h4>

In [15]:
mrnn.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 19:17:30 [INFO]: Epoch 001 - training loss (RMSE): 0.7504, validation MSE: 7.3129
2025-04-22 19:17:47 [INFO]: Epoch 002 - training loss (RMSE): 0.5341, validation MSE: 7.2684
2025-04-22 19:18:05 [INFO]: Epoch 003 - training loss (RMSE): 0.4959, validation MSE: 7.2556
2025-04-22 19:18:22 [INFO]: Epoch 004 - training loss (RMSE): 0.4612, validation MSE: 7.2489
2025-04-22 19:18:40 [INFO]: Epoch 005 - training loss (RMSE): 0.4541, validation MSE: 7.2464
2025-04-22 19:18:59 [INFO]: Epoch 006 - training loss (RMSE): 0.4473, validation MSE: 7.2449
2025-04-22 19:19:16 [INFO]: Epoch 007 - training loss (RMSE): 0.4298, validation MSE: 7.2458
2025-04-22 19:19:34 [INFO]: Epoch 008 - training loss (RMSE): 0.4340, validation MSE: 7.2485
2025-04-22 19:19:51 [INFO]: Epoch 009 - training loss (RMSE): 0.4229, validation MSE: 7.2492
2025-04-22 19:19:51 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-04-22 19:19:51 [INFO]: Finished training. The best model is 

<h4>Load</h4>

In [15]:
mrnn.load("tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots")

2025-05-01 07:57:18 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots


# Testing stage

### SAITS

#### Standard Scaler

In [16]:
saits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, saits)
saits_imputation_standard = toolkits.pre_reshape(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.desnormalization(saits_imputation_standard, scaler_standard)
saits_imputation_standard = toolkits.reshape_variable(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.reshape_variable(saits_imputation_standard_ori)

### BRITS

#### Standard Scaler

In [17]:
brits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, brits)
brits_imputation_standard = toolkits.pre_reshape(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.desnormalization(brits_imputation_standard, scaler_standard)
brits_imputation_standard = toolkits.reshape_variable(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.reshape_variable(brits_imputation_standard_ori)

### USGAN

In [18]:
usgan_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, us_gan)
usgan_imputation_standard = toolkits.pre_reshape(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.desnormalization(usgan_imputation_standard, scaler_standard)
usgan_imputation_standard = toolkits.reshape_variable(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.reshape_variable(usgan_imputation_standard_ori)

### GPVAE

In [19]:
gpvae_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, gp_vae)
gpvae_imputation_standard = toolkits.pre_reshape(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.desnormalization(gpvae_imputation_standard, scaler_standard)
gpvae_imputation_standard = toolkits.reshape_variable(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.reshape_variable(gpvae_imputation_standard_ori)

### MRNN

In [20]:
mrnn_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, mrnn)
mrnn_imputation_standard = toolkits.pre_reshape(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.desnormalization(mrnn_imputation_standard, scaler_standard)
mrnn_imputation_standard = toolkits.reshape_variable(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.reshape_variable(mrnn_imputation_standard_ori)

# Calculate mean absolute error

In [21]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

## SAITS

### Calculate MAE and AE

In [22]:
testing_mae_saits_variables_standard, testing_ae_saits_variables_standard = toolkits.calculate_mae(saits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for SAITS model

#### Difference between gender group

In [23]:
diff_saits_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 1, 2, variables)
diff_saits_gender_subgroup

[('Cholesterol', 0.5369465600078649),
 ('TroponinI', 0.21670827441781393),
 ('SaO2', 0.2006324035650055),
 ('Lactate', 0.10301878818965454),
 ('Urine', 0.06260751107865509)]

#### Difference between age group

In [24]:
diff_saits_age_subgroup = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 4, 5, variables)
diff_saits_age_subgroup

[('Cholesterol', 0.7145901970213923),
 ('TroponinI', 0.5566432434151988),
 ('ALP', 0.3592549477972564),
 ('SaO2', 0.1956973703724636),
 ('Bilirubin', 0.19557828811797667)]

#### Difference between ICUType group

In [25]:
diff_saits_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 6, 7, variables)
diff_saits_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 6, 8, variables)
diff_saits_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 6, 9, variables)
diff_saits_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 7, 8, variables)
diff_saits_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 7, 9, variables)
diff_saits_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 8, 9, variables)

In [26]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_saits_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_saits_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_saits_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_saits_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_saits_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_saits_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('TroponinI', 0.9325070778805344), ('Cholesterol', 0.9139836206325719), ('TroponinT', 0.610061672941423), ('Albumin', 0.3227603119355845), ('AST', 0.2527107766945585)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('TroponinI', 1.0714226157212936), ('SaO2', 1.0588494232986556), ('ALP', 0.7533389720997652), ('TroponinT', 0.5510963137613376), ('Albumin', 0.20918888526467827)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.6292154153589171), ('TroponinT', 0.5631873029767689), ('Albumin', 0.2424261680183718), ('Bilirubin', 0.2032288517692434), ('Glucose', 0.18363430222468535)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.2740858586846673), ('Cholesterol', 0.8991669157791677), ('ALP', 0.8536263954002238), ('Temp', 0.2721045789688794), ('Glucose', 0.22842305459952772)]
----------------------------
Diferença(MAE)

In [27]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[31])
print(variables[6])
print(variables[0])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
SaO2
TroponinI
Cholesterol
ALP
TroponinT


#### Difference between BMI group

In [28]:
diff_saits_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 11, variables)
diff_saits_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 12, variables)
diff_saits_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 13, variables)
diff_saits_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 14, variables)
diff_saits_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 15, variables)
diff_saits_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 16, variables)

diff_saits_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 12, variables)
diff_saits_classification_low_over = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 13, variables)
diff_saits_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 14, variables)
diff_saits_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 15, variables)
diff_saits_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 16, variables)

diff_saits_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 13, variables)
diff_saits_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 14, variables)
diff_saits_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 15, variables)
diff_saits_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 16, variables)

diff_saits_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 13, 14, variables)
diff_saits_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 13, 15, variables)
diff_saits_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 13, 16, variables)


diff_saits_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 14, 15, variables)
diff_saits_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 14, 16, variables)

diff_saits_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 15, 16, variables)

In [29]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_saits_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_saits_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_saits_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_saits_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_saits_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_saits_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.0695110496684588), ('Bilirubin', 0.9915184933318177), ('TroponinI', 0.6146236310366883), ('TroponinT', 0.44333031899542663), ('Glucose', 0.416639242489911)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('Cholesterol', 0.2895575098137716), ('SaO2', 0.2683085879682801), ('TroponinI', 0.1622792886545023), ('Bilirubin', 0.1397830646769168), ('PaCO2', 0.1369607780183822)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.4194546739320397), ('TroponinI', 0.2691988036465801), ('SaO2', 0.24053120129581373), ('TroponinT', 0.22793862706114962), ('Temp', 0.15260043693171418)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8884223546988217), ('SaO2', 0.25746554212335293), ('Albumin', 0.20186192626912858), ('Glucose', 0.1748223376573

In [30]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_saits_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_saits_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_saits_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_saits_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_saits_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 0.9982780836518492), ('Bilirubin', 0.8517354286549008), ('Cholesterol', 0.6024235381405012), ('Glucose', 0.45734829458249626), ('TroponinI', 0.45234434238218596)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 1.2036099992152576), ('Bilirubin', 1.1366820218459297), ('TroponinI', 0.8838224346832684), ('Cholesterol', 0.7323207022587693), ('Albumin', 0.45063840953236967)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.2290137796990863), ('Bilirubin', 1.0050968827848537), ('Ph', 0.8831229129347288), ('Albumin', 0.5967439438309785), ('Glucose', 0.5914615801472558)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 1.2728403997417839), ('Bilirubin', 1.1380893342352523), ('TroponinT', 0.7408628093945846), ('Albumin', 0.6101654410062806), ('Glucose', 0.395

In [31]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_saits_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_saits_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_saits_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_saits_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('TroponinI', 0.4314780923010824), ('Bilirubin', 0.2849465931910288), ('ALP', 0.2053319155634084), ('TroponinT', 0.19265691687489067), ('Cholesterol', 0.12989716411826813)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8949578059134441), ('ALP', 0.23073569604723718), ('Albumin', 0.18762873907869004), ('Mg', 0.17643656092671073), ('Cholesterol', 0.16955198480248967)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 1.1489114182037523), ('Cholesterol', 0.8242084172433273), ('TroponinI', 0.45234434238218596), ('Bilirubin', 0.28635390558035134), ('ALP', 0.27456231608993475)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('TroponinT', 0.9380846965551555), ('Cholesterol', 0.8267873356681623), ('ALP', 0.23797857377517134), ('ALT', 0.22522062790349448), 

In [32]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_saits_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_saits_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_saits_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Ph', 0.8907623608787905), ('TroponinI', 0.3704914364449188), ('Cholesterol', 0.2994491489207578), ('Glucose', 0.15595710055083023), ('Albumin', 0.14610553429860884)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('TroponinT', 0.9562545013288617), ('Cholesterol', 0.9541055813615954), ('TroponinI', 0.8838224346832684), ('Albumin', 0.1595270314739109), ('K', 0.11861301604813246)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 0.9566844997864304), ('TroponinT', 0.7454277796802649), ('TroponinI', 0.5011837233592029), ('Glucose', 0.17570701271287492), ('FiO2', 0.16851930435322476)]
----------------------------


In [33]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_saits_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_saits_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.1014465142680878), ('Ph', 0.8916481947347921), ('Cholesterol', 0.6546564324408376), ('TroponinI', 0.5133309982383496), ('Glucose', 0.19586706902845652)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Ph', 0.8969041099117874), ('TroponinT', 0.890619792619491), ('Cholesterol', 0.6572353508656726), ('Glucose', 0.33166411326370515), ('ALT', 0.18581162118574304)]
----------------------------


In [34]:

print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_saits_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('TroponinI', 0.38263871132406546), ('TroponinT', 0.21082672164859684), ('K', 0.16598073639207245), ('Glucose', 0.13579704423524863), ('HCO3', 0.1312658019106418)]
----------------------------


In [35]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[32])
print(variables[0])
print(variables[6])
print(variables[31])
print(variables[5])

Top 5 variáveis BMI
-----------------------
TroponinT
ALP
Cholesterol
TroponinI
Bilirubin


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [36]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_saits_female = []

for i in indices:
       results_bootstrap_saits_female.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [37]:
means_bootstraps_saits_female,standards_deviations_saits_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_female)

print(means_bootstraps_saits_female)
print(standards_deviations_saits_female)


[np.float64(1.3709565954271359), np.float64(0.37340746641577754), np.float64(0.5514148109724911), np.float64(0.33219783151411125), np.float64(0.3127126006634792), np.float64(0.1344058768679122), np.float64(0.15004390659670197), np.float64(0.6290111725113285), np.float64(0.25990652535575487), np.float64(0.134371002704543), np.float64(0.08328852545934053), np.float64(0.5210149778023866)]
[np.float64(0.28141828979738265), np.float64(0.07163803672687734), np.float64(0.13740379564534702), np.float64(0.02541877537794542), np.float64(0.008793741630019346), np.float64(0.04882230345968876), np.float64(0.010893276810424193), np.float64(0.033966982819504714), np.float64(0.056886855698661046), np.float64(0.048427337503807684), np.float64(0.004729043421165223), np.float64(0.02245024712647019)]


#### Calculating lower bound and upper bound

In [38]:
lower_bounds_saits_female, upper_bounds_saits_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_female, means_bootstraps_saits_female,standards_deviations_saits_female)

print(lower_bounds_saits_female)
print(upper_bounds_saits_female)

[np.float64(0.8193868828334101), np.float64(0.2329994945079403), np.float64(0.28210832016850945), np.float64(0.28237794724222476), np.float64(0.2954771837792908), np.float64(0.03871592044463695), np.float64(0.1286934763746452), np.float64(0.5624371095216085), np.float64(0.1484103369926521), np.float64(0.039455165329914096), np.float64(0.0740197706725306), np.float64(0.4770133019904812)]
[np.float64(1.9225263080208617), np.float64(0.5138154383236148), np.float64(0.8207213017764727), np.float64(0.38201771578599775), np.float64(0.32994801754766767), np.float64(0.23009583329118743), np.float64(0.17139433681875874), np.float64(0.6955852355010486), np.float64(0.37140271371885764), np.float64(0.22928684007917188), np.float64(0.09255728024615045), np.float64(0.565016653614292)]


In [39]:
lower_bounds_saits_female, upper_bounds_saits_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_female)

print(lower_bounds_saits_female)
print(upper_bounds_saits_female)

[np.float64(0.7956492099657536), np.float64(0.23760147981080015), np.float64(0.31828644884334323), np.float64(0.28378240468009003), np.float64(0.29572219940107847), np.float64(0.06067526607776592), np.float64(0.1300153151859639), np.float64(0.564655620050617), np.float64(0.16172599007870753), np.float64(0.05922063882042538), np.float64(0.07466409156145183), np.float64(0.47796719947937777)]
[np.float64(1.9226152106244694), np.float64(0.5194834637979123), np.float64(0.8484750064650793), np.float64(0.3837919745568725), np.float64(0.3300071456578719), np.float64(0.24500733036190417), np.float64(0.17308137833531811), np.float64(0.6969997681732706), np.float64(0.3813886456340512), np.float64(0.24560340589051607), np.float64(0.09309534886068198), np.float64(0.5657639504442826)]


#### Mean values of lower bound and upper bound


In [40]:
mean_values_ci_saits_female = toolkits.calc_mean_values_ci(lower_bounds_saits_female, upper_bounds_saits_female)

print(mean_values_ci_saits_female)

[np.float64(1.3591322102951116), np.float64(0.37854247180435624), np.float64(0.5833807276542113), np.float64(0.33378718961848125), np.float64(0.3128646725294752), np.float64(0.15284129821983505), np.float64(0.151548346760641), np.float64(0.6308276941119437), np.float64(0.2715573178563794), np.float64(0.15241202235547072), np.float64(0.0838797202110669), np.float64(0.5218655749618302)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [41]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_saits_male = []

for i in indices:
       results_bootstrap_saits_male.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [42]:
means_bootstraps_saits_male,standards_deviations_saits_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_male)

print(means_bootstraps_saits_male)
print(standards_deviations_saits_male)


[np.float64(0.8438058109897287), np.float64(0.5869423566271345), np.float64(0.3518805600283484), np.float64(0.43558126847157913), np.float64(0.375552431719391), np.float64(0.09281901372823773), np.float64(0.1690444660224031), np.float64(0.5757740658688472), np.float64(0.27776270400965464), np.float64(0.09338513088853735), np.float64(0.07236860789377567), np.float64(0.4871324217581555)]
[np.float64(0.14484001062154944), np.float64(0.1704863043219284), np.float64(0.03277833988488178), np.float64(0.03375522091077515), np.float64(0.011914226378457826), np.float64(0.019806856883581282), np.float64(0.009005653419694348), np.float64(0.02569829910524861), np.float64(0.04482147852835932), np.float64(0.01992827195446364), np.float64(0.0030149581872297346), np.float64(0.019178377337048934)]


#### Calculating lower bound and upper bound

In [43]:
lower_bounds_saits_male, upper_bounds_saits_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_male, means_bootstraps_saits_male,standards_deviations_saits_male)

print(lower_bounds_saits_male)
print(upper_bounds_saits_male)

[np.float64(0.559924606651093), np.float64(0.2527953402988194), np.float64(0.2876361943809673), np.float64(0.3694222511962665), np.float64(0.3522009771139566), np.float64(0.05399828758947916), np.float64(0.15139370966255222), np.float64(0.525406325158622), np.float64(0.18991422036023503), np.float64(0.05432643558366898), np.float64(0.06645939843191122), np.float64(0.4495434928956204)]
[np.float64(1.1276870153283645), np.float64(0.9210893729554495), np.float64(0.41612492567572945), np.float64(0.5017402857468918), np.float64(0.39890388632482543), np.float64(0.13163973986699629), np.float64(0.186695222382254), np.float64(0.6261418065790724), np.float64(0.36561118765907424), np.float64(0.13244382619340572), np.float64(0.07827781735564011), np.float64(0.5247213506206906)]


In [44]:
lower_bounds_saits_male, upper_bounds_saits_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_male)

print(lower_bounds_saits_male)
print(upper_bounds_saits_male)

[np.float64(0.5543691095812918), np.float64(0.24383298697774514), np.float64(0.2913003458887765), np.float64(0.3728388718325378), np.float64(0.3534882648294865), np.float64(0.059506104688290054), np.float64(0.15194163170799502), np.float64(0.5272282213020713), np.float64(0.1988940094151244), np.float64(0.059478934499948836), np.float64(0.06681734315554012), np.float64(0.4504542166091075)]
[np.float64(1.1198583711659658), np.float64(0.9200887406982938), np.float64(0.4211647434429169), np.float64(0.5048942135005), np.float64(0.40002559431022744), np.float64(0.1365678947406039), np.float64(0.18747326919939217), np.float64(0.6277415611360948), np.float64(0.3747574600297154), np.float64(0.1377540376898181), np.float64(0.07844891936116019), np.float64(0.5255574881262816)]


#### Mean values of lower bound and upper bound


In [45]:
mean_values_ci_saits_male = toolkits.calc_mean_values_ci(lower_bounds_saits_male, upper_bounds_saits_male)

print(mean_values_ci_saits_male)

[np.float64(0.8371137403736288), np.float64(0.5819608638380195), np.float64(0.3562325446658467), np.float64(0.4388665426665189), np.float64(0.37675692956985696), np.float64(0.09803699971444699), np.float64(0.1697074504536936), np.float64(0.5774848912190831), np.float64(0.2868257347224199), np.float64(0.09861648609488347), np.float64(0.07263313125835016), np.float64(0.48800585236769456)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [46]:
indices = [6, 32, 0, 28, 5, 31, 3]


results_bootstrap_saits_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_saits_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [47]:
means_bootstraps_saits_more_than_or_equal_to_65_subgroup,standards_deviations_saits_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_saits_more_than_or_equal_to_65_subgroup)
print(standards_deviations_saits_more_than_or_equal_to_65_subgroup)

[np.float64(0.6840107251428338), np.float64(0.306890478696312), np.float64(0.5410854493791742), np.float64(0.3593861740725029), np.float64(0.2174941018946004), np.float64(0.9223744805456358), np.float64(0.5744216340030307)]
[np.float64(0.17674009842842184), np.float64(0.06812227123612309), np.float64(0.11748070729527743), np.float64(0.033110838324512136), np.float64(0.024756910288949063), np.float64(0.323474177873407), np.float64(0.060833523112653914)]


#### Calculating lower bound and upper bound


In [48]:
lower_bounds_saits_more_than_or_equal_to_65_subgroup, upper_bounds_saits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_more_than_or_equal_to_65_subgroup, means_bootstraps_saits_more_than_or_equal_to_65_subgroup,standards_deviations_saits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_saits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_saits_more_than_or_equal_to_65_subgroup)


[np.float64(0.33760649759906275), np.float64(0.17337328052844184), np.float64(0.3108274942021385), np.float64(0.29449012345853054), np.float64(0.16897144935977113), np.float64(0.2883767419850548), np.float64(0.45519011964954403)]
[np.float64(1.0304149526866049), np.float64(0.44040767686418214), np.float64(0.77134340455621), np.float64(0.42428222468647525), np.float64(0.2660167544294296), np.float64(1.5563722191062168), np.float64(0.6936531483565174)]


In [49]:
lower_bounds_saits_more_than_or_equal_to_65_subgroup, upper_bounds_saits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_saits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_saits_more_than_or_equal_to_65_subgroup)

[np.float64(0.35107048872115837), np.float64(0.19405457858945557), np.float64(0.3309408378621302), np.float64(0.3012529987658585), np.float64(0.1722652547233647), np.float64(0.3744696407841734), np.float64(0.4631869896193011)]
[np.float64(1.0522101190096362), np.float64(0.45790638712920423), np.float64(0.7899029574949497), np.float64(0.43035416737196197), np.float64(0.26915957286882153), np.float64(1.6354705428625909), np.float64(0.6978495924750472)]


#### Mean values of lower bound and upper bound

In [50]:
mean_values_ci_saits_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_more_than_or_equal_to_65_subgroup , upper_bounds_saits_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_saits_more_than_or_equal_to_65_subgroup)

[np.float64(0.7016403038653973), np.float64(0.3259804828593299), np.float64(0.5604218976785399), np.float64(0.36580358306891025), np.float64(0.22071241379609313), np.float64(1.0049700918233822), np.float64(0.5805182910471741)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [51]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_saits_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_saits_less_than_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [52]:
means_bootstraps_saits_less_than_65_subgroup,standards_deviations_saits_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_less_than_65_subgroup)

print(means_bootstraps_saits_less_than_65_subgroup)
print(standards_deviations_saits_less_than_65_subgroup)

[np.float64(1.2071296148172286), np.float64(0.2725344788786177), np.float64(0.8978045284833425), np.float64(0.5526726596231104), np.float64(0.4140160720931158), np.float64(0.3646116414487031), np.float64(0.6601403948860949)]
[np.float64(0.7955134454995609), np.float64(0.08230259752908997), np.float64(0.3725300533810867), np.float64(0.13485242331156377), np.float64(0.07105143502438786), np.float64(0.08783285426784743), np.float64(0.05310376481708998)]


#### Calculating lower bound and upper bound

In [53]:
lower_bounds_saits_less_than_65_subgroup, upper_bounds_saits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_less_than_65_subgroup, means_bootstraps_saits_less_than_65_subgroup, standards_deviations_saits_less_than_65_subgroup)

print(lower_bounds_saits_less_than_65_subgroup)
print(upper_bounds_saits_less_than_65_subgroup)


[np.float64(-0.35204808757927797), np.float64(0.11122435188750612), np.float64(0.16765904069762871), np.float64(0.2883667667044958), np.float64(0.2747578183954278), np.float64(0.192462410424367), np.float64(0.5560589284011133)]
[np.float64(2.766307317213735), np.float64(0.4338446058697293), np.float64(1.627950016269056), np.float64(0.816978552541725), np.float64(0.5532743257908038), np.float64(0.5367608724730393), np.float64(0.7642218613710765)]


In [54]:
lower_bounds_saits_less_than_65_subgroup, upper_bounds_saits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_less_than_65_subgroup)

print(lower_bounds_saits_less_than_65_subgroup)
print(upper_bounds_saits_less_than_65_subgroup)


[np.float64(0.0), np.float64(0.1376943592692674), np.float64(0.3233972572576607), np.float64(0.32445719658454286), np.float64(0.286189075230866), np.float64(0.1889877196050605), np.float64(0.5601331906484173)]
[np.float64(2.297978957925913), np.float64(0.4524516282420147), np.float64(1.730549831346401), np.float64(0.8480803514414874), np.float64(0.5624647859409228), np.float64(0.5277707171008981), np.float64(0.7670367606413029)]


#### Mean values of lower bound and upper bound

In [55]:
mean_values_ci_saits_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_less_than_65_subgroup, upper_bounds_saits_less_than_65_subgroup)

print(mean_values_ci_saits_less_than_65_subgroup)

[np.float64(1.1489894789629564), np.float64(0.29507299375564106), np.float64(1.026973544302031), np.float64(0.5862687740130151), np.float64(0.4243269305858944), np.float64(0.35837921835297926), np.float64(0.6635849756448601)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [56]:
indices = [28, 31, 6, 0, 32, 24]




results_bootstrap_saits_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_1_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [57]:
means_bootstraps_saits_ICUType_1_subgroup, standards_deviations_saits_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_1_subgroup)

print(means_bootstraps_saits_ICUType_1_subgroup)
print(standards_deviations_saits_ICUType_1_subgroup)


[np.float64(0.5219567254652908), np.float64(1.4191329972717064), np.float64(0.9139329648049112), np.float64(0.21569407166429788), np.float64(0.7211401097401062), np.float64(0.4648222222352919)]
[np.float64(0.18500220990963184), np.float64(0.4711921029067144), np.float64(0.19021476797505155), np.float64(0.03900834305972772), np.float64(0.1525881619809211), np.float64(0.04182740358682482)]


#### Calculating lower bound and upper bound

In [58]:
lower_bounds_saits_ICUType_1_subgroup, upper_bounds_saits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_1_subgroup, means_bootstraps_saits_ICUType_1_subgroup, standards_deviations_saits_ICUType_1_subgroup)

print(lower_bounds_saits_ICUType_1_subgroup)
print(upper_bounds_saits_ICUType_1_subgroup)

[np.float64(0.1593590569820933), np.float64(0.4956134457748552), np.float64(0.5411188702461672), np.float64(0.1392391241706486), np.float64(0.42207280779033696), np.float64(0.3828420176382938)]
[np.float64(0.8845543939484883), np.float64(2.3426525487685574), np.float64(1.286747059363655), np.float64(0.29214901915794717), np.float64(1.0202074116898756), np.float64(0.54680242683229)]


In [59]:
lower_bounds_saits_ICUType_1_subgroup, upper_bounds_saits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_1_subgroup)

print(lower_bounds_saits_ICUType_1_subgroup)
print(upper_bounds_saits_ICUType_1_subgroup)

[np.float64(0.2724862865316178), np.float64(0.5714906008460912), np.float64(0.5423973514465497), np.float64(0.14321453098769518), np.float64(0.4430609084144976), np.float64(0.38418463195425406)]
[np.float64(0.9495419093825632), np.float64(2.412294909547965), np.float64(1.2983912355171199), np.float64(0.2973851173252235), np.float64(1.0472720745698492), np.float64(0.5491950726501879)]


#### Mean values of lower bound and upper bound

In [60]:
mean_values_ci_saits_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_1_subgroup, upper_bounds_saits_ICUType_1_subgroup)

print(mean_values_ci_saits_ICUType_1_subgroup)

[np.float64(0.6110140979570905), np.float64(1.491892755197028), np.float64(0.9203942934818348), np.float64(0.22029982415645932), np.float64(0.7451664914921734), np.float64(0.46668985230222093)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [61]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_saits_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_2_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [62]:
means_bootstraps_saits_ICUType_2_subgroup, standards_deviations_saits_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_2_subgroup)

print(means_bootstraps_saits_ICUType_2_subgroup)
print(standards_deviations_saits_ICUType_2_subgroup)


[np.float64(0.30234261908205756), np.float64(0.4822451154959138), np.float64(0.0), np.float64(0.11505758221413943), np.float64(0.11150250301424608), np.float64(0.44450325827242426)]
[np.float64(0.02262483990234257), np.float64(0.14270434095330978), np.float64(0.0), np.float64(0.015155524044065732), np.float64(0.011272872453300223), np.float64(0.018251955560383098)]


#### Calculating lower bound and upper bound

In [63]:
lower_bounds_saits_ICUType_2_subgroup, upper_bounds_saits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_2_subgroup, means_bootstraps_saits_ICUType_2_subgroup, standards_deviations_saits_ICUType_2_subgroup)

print(lower_bounds_saits_ICUType_2_subgroup)
print(upper_bounds_saits_ICUType_2_subgroup)

[np.float64(0.25799874771748144), np.float64(0.20254974678990234), np.float64(0.0), np.float64(0.08535330092093976), np.float64(0.08940807900346395), np.float64(0.4087300827266478)]
[np.float64(0.3466864904466337), np.float64(0.7619404842019253), np.float64(0.0), np.float64(0.1447618635073391), np.float64(0.1335969270250282), np.float64(0.48027643381820073)]


In [64]:
lower_bounds_saits_ICUType_2_subgroup, upper_bounds_saits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_2_subgroup)

print(lower_bounds_saits_ICUType_2_subgroup)
print(upper_bounds_saits_ICUType_2_subgroup)

[np.float64(0.2612377146330772), np.float64(0.18986491055554502), np.float64(0.0), np.float64(0.08598441095084189), np.float64(0.09026797114338445), np.float64(0.40965400315649914)]
[np.float64(0.3491110665417255), np.float64(0.7859344037034782), np.float64(0.0), np.float64(0.14570947953394509), np.float64(0.1345600276637829), np.float64(0.4812821650776725)]


#### Mean values of lower bound and upper bound

In [65]:
mean_values_ci_saits_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_2_subgroup, upper_bounds_saits_ICUType_2_subgroup)

print(mean_values_ci_saits_ICUType_2_subgroup)

[np.float64(0.30517439058740137), np.float64(0.4878996571295116), np.float64(0.0), np.float64(0.11584694524239349), np.float64(0.11241399940358367), np.float64(0.4454680841170858)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [66]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_saits_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_3_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [67]:
means_bootstraps_saits_ICUType_3_subgroup, standards_deviations_saits_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_3_subgroup)

print(means_bootstraps_saits_ICUType_3_subgroup)
print(standards_deviations_saits_ICUType_3_subgroup)


[np.float64(1.5873176168992222), np.float64(0.3451376351349188), np.float64(0.8838193565096376), np.float64(0.9719416794769047), np.float64(0.17008102185635626), np.float64(0.6029704811331233)]
[np.float64(0.4492088863497037), np.float64(0.0655364408499348), np.float64(0.17397706126823817), np.float64(0.3872023228794582), np.float64(0.04196961293018973), np.float64(0.03460610203052623)]


#### Calculating lower bound and upper bound

In [68]:
lower_bounds_saits_ICUType_3_subgroup, upper_bounds_saits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_3_subgroup, means_bootstraps_saits_ICUType_3_subgroup, standards_deviations_saits_ICUType_3_subgroup)

print(lower_bounds_saits_ICUType_3_subgroup)
print(upper_bounds_saits_ICUType_3_subgroup)

[np.float64(0.7068843781184567), np.float64(0.21668857139410702), np.float64(0.5428305822877724), np.float64(0.21303907190291727), np.float64(0.08782209206809784), np.float64(0.5351437675079735)]
[np.float64(2.467750855679988), np.float64(0.47358669887573057), np.float64(1.2248081307315029), np.float64(1.7308442870508922), np.float64(0.2523399516446147), np.float64(0.6707971947582732)]


In [69]:
lower_bounds_saits_ICUType_3_subgroup, upper_bounds_saits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_3_subgroup)

print(lower_bounds_saits_ICUType_3_subgroup)
print(upper_bounds_saits_ICUType_3_subgroup)

[np.float64(0.780404067787305), np.float64(0.2202083405208857), np.float64(0.6727157881964088), np.float64(0.3642860059565915), np.float64(0.09971476394281487), np.float64(0.5375214962234229)]
[np.float64(2.54731502169647), np.float64(0.4801916645249752), np.float64(1.2507688888161221), np.float64(1.8416487085107636), np.float64(0.2615151655889945), np.float64(0.6728665632862576)]


#### Mean values of lower bound and upper bound

In [70]:
mean_values_ci_saits_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_3_subgroup, upper_bounds_saits_ICUType_3_subgroup)

print(mean_values_ci_saits_ICUType_3_subgroup)

[np.float64(1.6638595447418876), np.float64(0.35020000252293043), np.float64(0.9617423385062654), np.float64(1.1029673572336776), np.float64(0.1806149647659047), np.float64(0.6051940297548403)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [71]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_saits_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_4_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [72]:
means_bootstraps_saits_ICUType_4_subgroup, standards_deviations_saits_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_4_subgroup)

print(means_bootstraps_saits_ICUType_4_subgroup)
print(standards_deviations_saits_ICUType_4_subgroup)


[np.float64(0.4374924981118807), np.float64(0.7710783833108681), np.float64(0.7361433475539954), np.float64(0.3833519769442785), np.float64(0.15810549637325846), np.float64(0.4660701136770505)]
[np.float64(0.09361222958290319), np.float64(0.3506626777117994), np.float64(0.17848971361751265), np.float64(0.06867994998652105), np.float64(0.026035599351315757), np.float64(0.021997324882899502)]


#### Calculating lower bound and upper bound

In [73]:
lower_bounds_saits_ICUType_4_subgroup, upper_bounds_saits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_4_subgroup, means_bootstraps_saits_ICUType_4_subgroup, standards_deviations_saits_ICUType_4_subgroup)

print(lower_bounds_saits_ICUType_4_subgroup)
print(upper_bounds_saits_ICUType_4_subgroup)

[np.float64(0.2540158996168954), np.float64(0.083792164273365), np.float64(0.3863099372528021), np.float64(0.2487417485106851), np.float64(0.10707665932876519), np.float64(0.4229561491503407)]
[np.float64(0.620969096606866), np.float64(1.4583646023483712), np.float64(1.0859767578551887), np.float64(0.517962205377872), np.float64(0.20913433341775173), np.float64(0.5091840782037602)]


In [74]:
lower_bounds_saits_ICUType_4_subgroup, upper_bounds_saits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_4_subgroup)

print(lower_bounds_saits_ICUType_4_subgroup)
print(upper_bounds_saits_ICUType_4_subgroup)

[np.float64(0.27930576874390706), np.float64(0.3728340243467206), np.float64(0.36871716024740075), np.float64(0.26153937912889297), np.float64(0.10962410035351977), np.float64(0.4232686475006617)]
[np.float64(0.6447007036079485), np.float64(1.8003804990136956), np.float64(1.0708848629739434), np.float64(0.527720487030236), np.float64(0.21165193489381803), np.float64(0.5101341787631254)]


#### Mean values of lower bound and upper bound

In [75]:
mean_values_ci_saits_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_4_subgroup, upper_bounds_saits_ICUType_4_subgroup)

print(mean_values_ci_saits_ICUType_4_subgroup)

[np.float64(0.46200323617592776), np.float64(1.086607261680208), np.float64(0.7198010116106721), np.float64(0.3946299330795645), np.float64(0.16063801762366892), np.float64(0.4667014131318935)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [76]:
indices = [32, 0, 6, 31, 5, 1, 35]


results_bootstrap_saits_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_undefined_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [77]:
means_bootstraps_saits_undefined_classification_subgroup, standards_deviations_saits_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_undefined_classification_subgroup)

print(means_bootstraps_saits_undefined_classification_subgroup)
print(standards_deviations_saits_undefined_classification_subgroup)


[np.float64(0.2220789243699558), np.float64(0.36322793707051365), np.float64(0.5358393333607179), np.float64(0.6174529701974534), np.float64(0.34997728226605834), np.float64(0.1486205100765959), np.float64(0.09019084920226916)]
[np.float64(0.04570653322260835), np.float64(0.05263916241599928), np.float64(0.1666043195868283), np.float64(0.21180064268718604), np.float64(0.10245179483312185), np.float64(0.04951555561227327), np.float64(0.005528848192846053)]


#### Calculating lower bound and upper bound

In [78]:
lower_bounds_saits_undefined_classification_subgroup, upper_bounds_saits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_undefined_classification_subgroup, means_bootstraps_saits_undefined_classification_subgroup, standards_deviations_saits_undefined_classification_subgroup)

print(lower_bounds_saits_undefined_classification_subgroup)
print(upper_bounds_saits_undefined_classification_subgroup)

[np.float64(0.13249576539545999), np.float64(0.26005707455880067), np.float64(0.2093008673017333), np.float64(0.20233133862813196), np.float64(0.1491754542416527), np.float64(0.051571804402050156), np.float64(0.07935450586830153)]
[np.float64(0.3116620833444516), np.float64(0.46639879958222663), np.float64(0.8623777994197024), np.float64(1.0325746017667747), np.float64(0.550779110290464), np.float64(0.24566921575114165), np.float64(0.10102719253623679)]


In [79]:
lower_bounds_saits_undefined_classification_subgroup, upper_bounds_saits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_undefined_classification_subgroup)

print(lower_bounds_saits_undefined_classification_subgroup)
print(upper_bounds_saits_undefined_classification_subgroup)

[np.float64(0.14861351880442525), np.float64(0.2681309927056473), np.float64(0.21667597714838674), np.float64(0.22678503532965885), np.float64(0.19076055650846094), np.float64(0.07058041202715513), np.float64(0.079863334033074)]
[np.float64(0.32444401367727815), np.float64(0.4722891131963517), np.float64(0.8712429101228379), np.float64(1.0597512498483057), np.float64(0.5819828141296248), np.float64(0.26020473146627615), np.float64(0.10139542073969701)]


#### Mean values of lower bound and upper bound

In [80]:
mean_values_ci_saits_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_undefined_classification_subgroup, upper_bounds_saits_undefined_classification_subgroup)

print(mean_values_ci_saits_undefined_classification_subgroup)

[np.float64(0.2365287662408517), np.float64(0.3702100529509995), np.float64(0.5439594436356123), np.float64(0.6432681425889822), np.float64(0.38637168531904287), np.float64(0.16539257174671565), np.float64(0.0906293773863855)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [81]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_low_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [82]:
means_bootstraps_saits_low_weight_classification_subgroup, standards_deviations_saits_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_low_weight_classification_subgroup)

print(means_bootstraps_saits_low_weight_classification_subgroup)
print(standards_deviations_saits_low_weight_classification_subgroup)


[np.float64(0.4179372777585539), np.float64(1.2434660978146057), np.float64(0.14389198214777257), np.float64(0.0), np.float64(1.3355777932607396), np.float64(0.05999720428400326), np.float64(0.08402665695116406)]
[np.float64(0.3211289804440317), np.float64(1.086221728773829), np.float64(0.10760691902498223), np.float64(0.0), np.float64(1.11933454327662), np.float64(0.020827781538153973), np.float64(0.008895762289132428)]


#### Calculating lower bound and upper bound

In [83]:
lower_bounds_saits_low_weight_classification_subgroup, upper_bounds_saits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_low_weight_classification_subgroup, means_bootstraps_saits_low_weight_classification_subgroup, standards_deviations_saits_low_weight_classification_subgroup)

print(lower_bounds_saits_low_weight_classification_subgroup)
print(upper_bounds_saits_low_weight_classification_subgroup)

[np.float64(-0.21146395830381565), np.float64(-0.8854893698069339), np.float64(-0.06701370362851056), np.float64(0.0), np.float64(-0.858277598213026), np.float64(0.01917550259135322), np.float64(0.06659128324943492)]
[np.float64(1.0473385138209235), np.float64(3.372421565436145), np.float64(0.3547976679240557), np.float64(0.0), np.float64(3.529433184734505), np.float64(0.1008189059766533), np.float64(0.10146203065289319)]


In [84]:
lower_bounds_saits_low_weight_classification_subgroup, upper_bounds_saits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_low_weight_classification_subgroup)

print(lower_bounds_saits_low_weight_classification_subgroup)
print(upper_bounds_saits_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.06532619020593648), np.float64(0.02636130159392695), np.float64(0.06807568231807692)]
[np.float64(0.6646820109257148), np.float64(2.8082561340301457), np.float64(0.22436379752781074), np.float64(0.0), np.float64(3.898138116424551), np.float64(0.10568388666059643), np.float64(0.10248588150567063)]


#### Mean values of lower bound and upper bound

In [85]:
mean_values_ci_saits_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_low_weight_classification_subgroup, upper_bounds_saits_low_weight_classification_subgroup)

print(mean_values_ci_saits_low_weight_classification_subgroup)

[np.float64(0.3323410054628574), np.float64(1.4041280670150729), np.float64(0.11218189876390537), np.float64(0.0), np.float64(1.9817321533152439), np.float64(0.06602259412726169), np.float64(0.08528078191187377)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [86]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_normal_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [87]:
means_bootstraps_saits_normal_weight_classification_subgroup, standards_deviations_saits_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_normal_weight_classification_subgroup)

print(means_bootstraps_saits_normal_weight_classification_subgroup)
print(standards_deviations_saits_normal_weight_classification_subgroup)


[np.float64(0.25744368744577145), np.float64(0.4349773922115265), np.float64(0.717382366934424), np.float64(0.4541860461731961), np.float64(0.49248182712618804), np.float64(0.061875693671140285), np.float64(0.061840634043035705)]
[np.float64(0.06530091398193434), np.float64(0.1130191030039274), np.float64(0.36496621546347674), np.float64(0.17155187474732164), np.float64(0.16924381363494684), np.float64(0.017466578077892432), np.float64(0.0036097147752741235)]


#### Calculating lower bound and upper bound

In [88]:
lower_bounds_saits_normal_weight_classification_subgroup, upper_bounds_saits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_normal_weight_classification_subgroup, means_bootstraps_saits_normal_weight_classification_subgroup, standards_deviations_saits_normal_weight_classification_subgroup)

print(lower_bounds_saits_normal_weight_classification_subgroup)
print(upper_bounds_saits_normal_weight_classification_subgroup)

[np.float64(0.12945624788363208), np.float64(0.21346402075880616), np.float64(0.002061729052124317), np.float64(0.11795055018811929), np.float64(0.16077004779548332), np.float64(0.027641829705314272), np.float64(0.05476572308903633)]
[np.float64(0.3854311270079108), np.float64(0.6564907636642467), np.float64(1.4327030048167237), np.float64(0.7904215421582729), np.float64(0.8241936064568928), np.float64(0.0961095576369663), np.float64(0.06891554499703509)]


In [89]:
lower_bounds_saits_normal_weight_classification_subgroup, upper_bounds_saits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_normal_weight_classification_subgroup)

print(lower_bounds_saits_normal_weight_classification_subgroup)
print(upper_bounds_saits_normal_weight_classification_subgroup)

[np.float64(0.14427118713539758), np.float64(0.24249607025212516), np.float64(0.0), np.float64(0.12762180966123823), np.float64(0.22082274351885345), np.float64(0.03289849328793306), np.float64(0.05517902856700568)]
[np.float64(0.39878917968234284), np.float64(0.6804758208233153), np.float64(1.157952311730471), np.float64(0.810228541120111), np.float64(0.8680464036506664), np.float64(0.10027582513098275), np.float64(0.06920380693573645)]


#### Mean values of lower bound and upper bound

In [90]:
mean_values_ci_saits_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_normal_weight_classification_subgroup, upper_bounds_saits_normal_weight_classification_subgroup)

print(mean_values_ci_saits_normal_weight_classification_subgroup)

[np.float64(0.2715301834088702), np.float64(0.4614859455377202), np.float64(0.5789761558652355), np.float64(0.46892517539067463), np.float64(0.54443457358476), np.float64(0.0665871592094579), np.float64(0.06219141775137106)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [91]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_overweight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [92]:
means_bootstraps_saits_overweight_classification_subgroup, standards_deviations_saits_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_overweight_classification_subgroup)

print(means_bootstraps_saits_overweight_classification_subgroup)
print(standards_deviations_saits_overweight_classification_subgroup)


[np.float64(0.44788379857909943), np.float64(0.22978055068985156), np.float64(0.9617003448767828), np.float64(0.8690059501858922), np.float64(0.20889000167879582), np.float64(0.1554328496222451), np.float64(0.05987518990506586)]
[np.float64(0.1963573758845288), np.float64(0.046913847941276524), np.float64(0.21814397668322832), np.float64(0.5515070410248044), np.float64(0.06296380705055063), np.float64(0.04989517332353856), np.float64(0.0029144577918172316)]


#### Calculating lower bound and upper bound

In [93]:
lower_bounds_saits_overweight_classification_subgroup, upper_bounds_saits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_overweight_classification_subgroup, means_bootstraps_saits_overweight_classification_subgroup, standards_deviations_saits_overweight_classification_subgroup)

print(lower_bounds_saits_overweight_classification_subgroup)
print(upper_bounds_saits_overweight_classification_subgroup)

[np.float64(0.06303041374662921), np.float64(0.137831098348761), np.float64(0.53414600713331), np.float64(-0.21192798744297847), np.float64(0.08548320753018745), np.float64(0.05764010690572585), np.float64(0.054162957598641946)]
[np.float64(0.8327371834115697), np.float64(0.3217300030309421), np.float64(1.3892546826202556), np.float64(1.949939887814763), np.float64(0.3322967958274042), np.float64(0.25322559233876435), np.float64(0.06558742221148976)]


In [94]:
lower_bounds_saits_overweight_classification_subgroup, upper_bounds_saits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_overweight_classification_subgroup)

print(lower_bounds_saits_overweight_classification_subgroup)
print(upper_bounds_saits_overweight_classification_subgroup)

[np.float64(0.12413702513626826), np.float64(0.14433416587788625), np.float64(0.5280093967897358), np.float64(0.10499430382568216), np.float64(0.1190599458217266), np.float64(0.07927384479062116), np.float64(0.05455167757690384)]
[np.float64(0.8832924474115974), np.float64(0.32829721086014874), np.float64(1.3861176353592872), np.float64(2.0863204489780642), np.float64(0.35445178174038233), np.float64(0.2663539917461887), np.float64(0.06601488658930574)]


#### Mean values of lower bound and upper bound

In [95]:
mean_values_ci_saits_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_overweight_classification_subgroup, upper_bounds_saits_overweight_classification_subgroup)

print(mean_values_ci_saits_overweight_classification_subgroup)

[np.float64(0.5037147362739328), np.float64(0.2363156883690175), np.float64(0.9570635160745116), np.float64(1.095657376401873), np.float64(0.23675586378105445), np.float64(0.17281391826840492), np.float64(0.06028328208310479)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [96]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_obs1_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [97]:
means_bootstraps_saits_obs1_classification_subgroup, standards_deviations_saits_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_obs1_classification_subgroup)

print(means_bootstraps_saits_obs1_classification_subgroup)
print(standards_deviations_saits_obs1_classification_subgroup)


[np.float64(0.3029890874167423), np.float64(0.20407255566287066), np.float64(0.6208529973138541), np.float64(0.4395335826294013), np.float64(0.3425874360473905), np.float64(0.10179621402459223), np.float64(0.06688625776136924)]
[np.float64(0.08587512838896379), np.float64(0.060811315475714646), np.float64(0.3138483810115808), np.float64(0.21299261450393206), np.float64(0.1145905761579488), np.float64(0.04148364623779175), np.float64(0.0034668575994258057)]


#### Calculating lower bound and upper bound

In [98]:
lower_bounds_saits_obs1_classification_subgroup, upper_bounds_saits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_obs1_classification_subgroup, means_bootstraps_saits_obs1_classification_subgroup, standards_deviations_saits_obs1_classification_subgroup)

print(lower_bounds_saits_obs1_classification_subgroup)
print(upper_bounds_saits_obs1_classification_subgroup)

[np.float64(0.13467692860662012), np.float64(0.08488456747796673), np.float64(0.005721473924951148), np.float64(0.022075729228670904), np.float64(0.11799403381011664), np.float64(0.020489761451119887), np.float64(0.06009134172696567)]
[np.float64(0.4713012462268645), np.float64(0.3232605438477746), np.float64(1.235984520702757), np.float64(0.8569914360301316), np.float64(0.5671808382846644), np.float64(0.18310266659806457), np.float64(0.0736811737957728)]


In [99]:
lower_bounds_saits_obs1_classification_subgroup, upper_bounds_saits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_obs1_classification_subgroup)

print(lower_bounds_saits_obs1_classification_subgroup)
print(upper_bounds_saits_obs1_classification_subgroup)

[np.float64(0.15418243606681148), np.float64(0.09379579326862329), np.float64(0.0), np.float64(0.0), np.float64(0.15843812547099442), np.float64(0.04089258082946061), np.float64(0.060593833475975654)]
[np.float64(0.4890280355937019), np.float64(0.3325682569395985), np.float64(1.2406693076044368), np.float64(0.6809092237281027), np.float64(0.6005876011055841), np.float64(0.1972119507960664), np.float64(0.07412772674871662)]


#### Mean values of lower bound and upper bound

In [100]:
mean_values_ci_saits_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_obs1_classification_subgroup, upper_bounds_saits_obs1_classification_subgroup)

print(mean_values_ci_saits_obs1_classification_subgroup)

[np.float64(0.3216052358302567), np.float64(0.2131820251041109), np.float64(0.6203346538022184), np.float64(0.34045461186405135), np.float64(0.37951286328828926), np.float64(0.11905226581276351), np.float64(0.06736078011234614)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [101]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_obs2_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [102]:
means_bootstraps_saits_obs2_classification_subgroup, standards_deviations_saits_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_obs2_classification_subgroup)

print(means_bootstraps_saits_obs2_classification_subgroup)
print(standards_deviations_saits_obs2_classification_subgroup)


[np.float64(1.3202318318531734), np.float64(0.16018320474459607), np.float64(0.0016333150023958836), np.float64(0.0), np.float64(0.20565140281883998), np.float64(0.1904748878318178), np.float64(0.0748960300784776)]
[np.float64(1.0284769556193523), np.float64(0.04163536032079147), np.float64(0.0012427663723273534), np.float64(0.0), np.float64(0.14065179783514387), np.float64(0.09377300495839148), np.float64(0.006743679991352325)]


#### Calculating lower bound and upper bound

In [103]:
lower_bounds_saits_obs2_classification_subgroup, upper_bounds_saits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_obs2_classification_subgroup, means_bootstraps_saits_obs2_classification_subgroup, standards_deviations_saits_obs2_classification_subgroup)

print(lower_bounds_saits_obs2_classification_subgroup)
print(upper_bounds_saits_obs2_classification_subgroup)

[np.float64(-0.6955459600901566), np.float64(0.07857939803249675), np.float64(-0.0008024623285632243), np.float64(0.0), np.float64(-0.07002105529885072), np.float64(0.006683175391274582), np.float64(0.06167866017216366)]
[np.float64(3.336009623796503), np.float64(0.2417870114566954), np.float64(0.004069092333354991), np.float64(0.0), np.float64(0.4813238609365307), np.float64(0.374266600272361), np.float64(0.08811339998479154)]


In [104]:
lower_bounds_saits_obs2_classification_subgroup, upper_bounds_saits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_obs2_classification_subgroup)

print(lower_bounds_saits_obs2_classification_subgroup)
print(upper_bounds_saits_obs2_classification_subgroup)

[np.float64(0.0), np.float64(0.08663420909169421), np.float64(0.0), np.float64(0.0), np.float64(0.05801357838495656), np.float64(0.05135770937952164), np.float64(0.062486301173007935)]
[np.float64(3.5585249882396806), np.float64(0.2493556649833264), np.float64(0.0025789184248367356), np.float64(0.0), np.float64(0.5359872611109567), np.float64(0.39997488231726197), np.float64(0.08915991716810763)]


#### Mean values of lower bound and upper bound

In [105]:
mean_values_ci_saits_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_obs2_classification_subgroup, upper_bounds_saits_obs2_classification_subgroup)

print(mean_values_ci_saits_obs2_classification_subgroup)

[np.float64(1.7792624941198403), np.float64(0.1679949370375103), np.float64(0.0012894592124183678), np.float64(0.0), np.float64(0.29700041974795666), np.float64(0.22566629584839182), np.float64(0.07582310917055779)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [106]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [107]:
means_bootstraps_saits_obs3_classification_subgroup, standards_deviations_saits_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_obs3_classification_subgroup)

print(means_bootstraps_saits_obs3_classification_subgroup)
print(standards_deviations_saits_obs3_classification_subgroup)


[np.float64(1.174122801941639), np.float64(0.19605819975450417), np.float64(0.0), np.float64(0.24114741895895714), np.float64(0.2807075619561548), np.float64(0.27174246354233467), np.float64(0.12161519628120666)]
[np.float64(0.8152811692023881), np.float64(0.05438973613686845), np.float64(0.0), np.float64(0.18471670189518583), np.float64(0.14691368485557318), np.float64(0.18967925991644569), np.float64(0.01616573041190024)]


#### Calculating lower bound and upper bound

In [108]:
lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_obs3_classification_subgroup, means_bootstraps_saits_obs3_classification_subgroup, standards_deviations_saits_obs3_classification_subgroup)

print(lower_bounds_saits_obs3_classification_subgroup)
print(upper_bounds_saits_obs3_classification_subgroup)

[np.float64(-0.42379892696874766), np.float64(0.08945627579760532), np.float64(0.0), np.float64(-0.12089066409862864), np.float64(-0.007237969196836225), np.float64(-0.1000220545081108), np.float64(0.08993094689009834)]
[np.float64(2.7720445308520256), np.float64(0.30266012371140305), np.float64(0.0), np.float64(0.6031855020165429), np.float64(0.5686530931091458), np.float64(0.6435069815927801), np.float64(0.153299445672315)]


In [109]:
lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_obs3_classification_subgroup)

print(lower_bounds_saits_obs3_classification_subgroup)
print(upper_bounds_saits_obs3_classification_subgroup)

[np.float64(0.09197122917451109), np.float64(0.10864533313994013), np.float64(0.0), np.float64(0.0), np.float64(0.041572934649777814), np.float64(0.0), np.float64(0.09330420152930746)]
[np.float64(3.00022367963562), np.float64(0.31846198470855486), np.float64(0.0), np.float64(0.38263871132432054), np.float64(0.5959809854230184), np.float64(0.6848965195437268), np.float64(0.15631952632364246)]


#### Mean values of lower bound and upper bound

In [110]:
mean_values_ci_saits_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup)

print(mean_values_ci_saits_obs3_classification_subgroup)

[np.float64(1.5460974544050654), np.float64(0.2135536589242475), np.float64(0.0), np.float64(0.19131935566216027), np.float64(0.3187769600363981), np.float64(0.3424482597718634), np.float64(0.12481186392647496)]


## BRITS

### Calculate MAE and AE

In [111]:
testing_mae_brits_variables_standard, testing_ae_brits_variables_standard = toolkits.calculate_mae(brits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for BRITS model

#### Difference between gender group

In [112]:
diff_brits_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 1, 2, variables)
diff_brits_gender_subgroup

[('Cholesterol', 0.33467020046017315),
 ('AST', 0.17888955772137866),
 ('TroponinI', 0.17526927063842523),
 ('Creatinine', 0.15555507367732185),
 ('Lactate', 0.0961695135309471)]

#### Difference between age group

In [113]:
diff_brits_age_subgroup = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 4, 5, variables)
diff_brits_age_subgroup

[('ALP', 0.4455670614286247),
 ('TroponinI', 0.3868195082859647),
 ('Bilirubin', 0.23810434213767862),
 ('Cholesterol', 0.22772024967991167),
 ('Albumin', 0.13833296504538817)]

#### Difference between ICUType group

In [114]:
diff_brits_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 6, 7, variables)
diff_brits_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 6, 8, variables)
diff_brits_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 6, 9, variables)
diff_brits_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 7, 8, variables)
diff_brits_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 7, 9, variables)
diff_brits_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 8, 9, variables)

In [115]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_brits_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_brits_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_brits_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_brits_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_brits_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_brits_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 0.9649941105882632), ('TroponinI', 0.9581594407990047), ('TroponinT', 0.5486699079166011), ('Platelets', 0.2518916923226591), ('AST', 0.24124108598094132)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('SaO2', 1.0707510714622313), ('ALP', 0.9106143557497219), ('TroponinI', 0.7970523312899177), ('TroponinT', 0.4839761749137556), ('Cholesterol', 0.25934743317704234)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.6343111242413422), ('TroponinT', 0.5409005868767687), ('Bilirubin', 0.2897210435032669), ('ALP', 0.26571689159730005), ('Cholesterol', 0.23623932465153985)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.030567265690355), ('ALP', 0.88221080881332), ('Cholesterol', 0.7056466774112209), ('BUN', 0.33074673635065266), ('Platelets', 0.31153489001028306)]
----------------------------
Difere

In [116]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[0])
print(variables[6])
print(variables[31])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
SaO2
ALP
Cholesterol
TroponinI
TroponinT


#### Difference between BMI group

In [117]:
diff_brits_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 11, variables)
diff_brits_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 12, variables)
diff_brits_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 13, variables)
diff_brits_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 14, variables)
diff_brits_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 15, variables)
diff_brits_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 16, variables)

diff_brits_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 12, variables)
diff_brits_classification_low_over = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 13, variables)
diff_brits_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 14, variables)
diff_brits_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 15, variables)
diff_brits_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 16, variables)

diff_brits_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 13, variables)
diff_brits_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 14, variables)
diff_brits_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 15, variables)
diff_brits_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 16, variables)

diff_brits_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 13, 14, variables)
diff_brits_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 13, 15, variables)
diff_brits_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 13, 16, variables)


diff_brits_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 14, 15, variables)
diff_brits_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 14, 16, variables)

diff_brits_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 15, 16, variables)

In [118]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_brits_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_brits_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_brits_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_brits_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_brits_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_brits_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.9340447366419231), ('Bilirubin', 1.0634205379618589), ('TroponinI', 0.8461866959272529), ('Cholesterol', 0.46427180720735495), ('Lactate', 0.432934002045685)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('SaO2', 0.3956365150352539), ('Cholesterol', 0.31334687951493556), ('TroponinI', 0.2811986010883921), ('Platelets', 0.21973579409347943), ('ALP', 0.14108821471054112)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.9122714170407268), ('SaO2', 0.2216147767636339), ('Bilirubin', 0.21564253602217226), ('TroponinI', 0.20102396419219326), ('ALT', 0.19595996986933695)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8860429007708258), ('TroponinI', 0.27796277400600267), ('SaO2', 0.2630706443883271), ('Albumin', 0.2194660852

In [119]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_brits_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_brits_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_brits_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_brits_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_brits_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 1.7929565219313819), ('Bilirubin', 0.9513305382204593), ('Cholesterol', 0.7776186867222905), ('SaO2', 0.6685476719781142), ('TroponinI', 0.5649880948388608)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 2.060865479901618), ('Cholesterol', 1.3765432242480817), ('Bilirubin', 1.2790630739840312), ('TroponinI', 1.0472106601194462), ('Glucose', 0.40732195398769067)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 2.0338878789564587), ('Bilirubin', 1.0689277903377052), ('Ph', 0.8810842114333002), ('TroponinI', 0.5682239219212503), ('Glucose', 0.5610521608500285)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 2.083344030352053), ('Bilirubin', 1.189908383818061), ('TroponinT', 0.8194450582455736), ('AST', 0.44272365749481374), ('HCO3', 0.434470181819755

In [120]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_brits_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_brits_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_brits_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_brits_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('SaO2', 0.6172512917988877), ('Cholesterol', 0.5989245375257912), ('TroponinI', 0.4822225652805854), ('Bilirubin', 0.32773253576357175), ('ALT', 0.2736328573496928)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8903667412858379), ('SaO2', 0.6587071594235809), ('Cholesterol', 0.36517235893560385), ('Albumin', 0.2900474046493634), ('ALP', 0.24093135702507662)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 1.220432293290763), ('Cholesterol', 0.688511448385237), ('TroponinI', 0.5649880948388608), ('SaO2', 0.4915174467035355), ('ALT', 0.3479177744530949)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('TroponinT', 0.9141402460637791), ('Cholesterol', 0.8876133876483889), ('ALT', 0.8550561867312895), ('SaO2', 0.5055719721885107), ('ALP', 0.3959647

In [121]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_brits_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_brits_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_brits_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Cholesterol', 0.9640968964613951), ('Ph', 0.8905324606918038), ('TroponinI', 0.4789867381981959), ('ALT', 0.28228479496676956), ('Bilirubin', 0.2101352836463259)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('Cholesterol', 1.2874359859110283), ('TroponinI', 1.0472106601194462), ('TroponinT', 1.0262220986441177), ('AST', 0.21817186815478096), ('BUN', 0.19186305260550485)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.4865379251741802), ('TroponinT', 0.7199300514171338), ('TroponinI', 0.6550984461594344), ('ALT', 0.5814233293815967), ('Glucose', 0.23574108257412307)]
----------------------------


In [122]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_brits_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_brits_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.193502313404874), ('Ph', 0.8867898203626943), ('TroponinI', 0.5682239219212503), ('ALT', 0.3565697120701717), ('Cholesterol', 0.32333908944963324)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Ph', 0.8897412963473199), ('TroponinT', 0.88721026617789), ('ALT', 0.8637081243483662), ('Cholesterol', 0.5224410287127851), ('Glucose', 0.38947128943646087)]
----------------------------


In [123]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_brits_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('ALT', 0.5071384122781946), ('TroponinI', 0.39211221396001184), ('TroponinT', 0.3062920472269839), ('HCO3', 0.23358518974369696), ('Glucose', 0.20095366197425335)]
----------------------------


In [124]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[0])
print(variables[6])
print(variables[32])
print(variables[31])
print(variables[5])

Top 5 variáveis BMI
-----------------------
ALP
Cholesterol
TroponinT
TroponinI
Bilirubin


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [125]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_brits_female = []

for i in indices:
       results_bootstrap_brits_female.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [126]:
means_bootstraps_brits_female,standards_deviations_brits_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_female)

print(means_bootstraps_brits_female)
print(standards_deviations_brits_female)


[np.float64(1.179863930834408), np.float64(0.5233882194493867), np.float64(0.5669394454773564), np.float64(0.4262927607316525), np.float64(0.2927745387996273), np.float64(0.30161012783732477), np.float64(0.2520432195506154), np.float64(0.6740430489599522), np.float64(0.29792444872181434), np.float64(0.2996217790430212), np.float64(0.060521396312743225), np.float64(0.5075399124539731)]
[np.float64(0.33967759869351355), np.float64(0.07230701862030067), np.float64(0.13940375834716123), np.float64(0.03847644149074913), np.float64(0.008670900439200447), np.float64(0.0848645872760513), np.float64(0.024759857939583314), np.float64(0.03441311480209853), np.float64(0.06605797747312059), np.float64(0.08435135157044552), np.float64(0.004754151191010087), np.float64(0.021740570722039883)]


#### Calculating lower bound and upper bound

In [127]:
lower_bounds_brits_female, upper_bounds_brits_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_female, means_bootstraps_brits_female,standards_deviations_brits_female)

print(lower_bounds_brits_female)
print(upper_bounds_brits_female)

[np.float64(0.5141080710400718), np.float64(0.38166906712413035), np.float64(0.29371309980739546), np.float64(0.35088032115652157), np.float64(0.2757798862252619), np.float64(0.1352785932134081), np.float64(0.20351478972670403), np.float64(0.6065945833519968), np.float64(0.1684531919829398), np.float64(0.13429616791767185), np.float64(0.05120343120130525), np.float64(0.464929176835429)]
[np.float64(1.8456197906287444), np.float64(0.6651073717746432), np.float64(0.8401657911473174), np.float64(0.5017052003067834), np.float64(0.30976919137399267), np.float64(0.46794166246124147), np.float64(0.30057164937452685), np.float64(0.7414915145679075), np.float64(0.4273957054606889), np.float64(0.4649473901683705), np.float64(0.0698393614241812), np.float64(0.5501506480725172)]


In [128]:
lower_bounds_brits_female, upper_bounds_brits_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_female)

print(lower_bounds_brits_female)
print(upper_bounds_brits_female)

[np.float64(0.4968807905077575), np.float64(0.38193937258413285), np.float64(0.32993095168375397), np.float64(0.3549349251700391), np.float64(0.27620089719597324), np.float64(0.16091099800584968), np.float64(0.20714247470515038), np.float64(0.6097020735297847), np.float64(0.18469541164541012), np.float64(0.15861808082118425), np.float64(0.05168324294505243), np.float64(0.4662975796151602)]
[np.float64(1.8393478124459337), np.float64(0.6682822710147335), np.float64(0.8757607526086687), np.float64(0.5049085262899291), np.float64(0.31004767626972984), np.float64(0.4877700875445269), np.float64(0.30484660206533915), np.float64(0.7450377725488991), np.float64(0.4418350473462839), np.float64(0.48734408360396114), np.float64(0.07025076961204074), np.float64(0.5509092356408299)]


#### Mean values of lower bound and upper bound


In [129]:
mean_values_ci_brits_female = toolkits.calc_mean_values_ci(lower_bounds_brits_female, upper_bounds_brits_female)

print(mean_values_ci_brits_female)

[np.float64(1.1681143014768456), np.float64(0.5251108217994331), np.float64(0.6028458521462113), np.float64(0.4299217257299841), np.float64(0.2931242867328515), np.float64(0.3243405427751883), np.float64(0.2559945383852448), np.float64(0.677369923039342), np.float64(0.313265229495847), np.float64(0.3229810822125727), np.float64(0.060967006278546586), np.float64(0.508603407627995)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [130]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_brits_male = []

for i in indices:
       results_bootstrap_brits_male.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [131]:
means_bootstraps_brits_male,standards_deviations_brits_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_male)

print(means_bootstraps_brits_male)
print(standards_deviations_brits_male)


[np.float64(0.854312785965969), np.float64(0.69793226655648), np.float64(0.5788874609838146), np.float64(0.5218477147546687), np.float64(0.3547467656480773), np.float64(0.12182354571033481), np.float64(0.4073172652594466), np.float64(0.6193417493863396), np.float64(0.28308368574229126), np.float64(0.12197737498234233), np.float64(0.05506604203706228), np.float64(0.4785682671425299)]
[np.float64(0.17112048417178624), np.float64(0.20950734713753777), np.float64(0.2222638014715227), np.float64(0.04261663101563413), np.float64(0.011539284042436456), np.float64(0.02574568952633491), np.float64(0.03829764363415067), np.float64(0.026269546181583104), np.float64(0.04821267153960221), np.float64(0.02561385052707165), np.float64(0.0033874977733877112), np.float64(0.018186917981419373)]


#### Calculating lower bound and upper bound

In [132]:
lower_bounds_brits_male, upper_bounds_brits_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_male, means_bootstraps_brits_male,standards_deviations_brits_male)

print(lower_bounds_brits_male)
print(upper_bounds_brits_male)

[np.float64(0.5189227999722117), np.float64(0.2873054116703752), np.float64(0.14325841503266945), np.float64(0.4383206528215932), np.float64(0.3321301845175241), np.float64(0.07136292148156831), np.float64(0.33225526304376163), np.float64(0.567854384980225), np.float64(0.18858858592621164), np.float64(0.07177515044388962), np.float64(0.04842666840351274), np.float64(0.44292256290916404)]
[np.float64(1.1897027719597264), np.float64(1.1085591214425847), np.float64(1.0145165069349598), np.float64(0.6053747766877442), np.float64(0.3773633467786305), np.float64(0.1722841699391013), np.float64(0.4823792674751316), np.float64(0.6708291137924541), np.float64(0.3775787855583709), np.float64(0.17217959952079503), np.float64(0.061705415670611816), np.float64(0.5142139713758958)]


In [133]:
lower_bounds_brits_male, upper_bounds_brits_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_male)

print(lower_bounds_brits_male)
print(upper_bounds_brits_male)

[np.float64(0.5301380329394242), np.float64(0.32927026897220596), np.float64(0.3079566108428539), np.float64(0.44188543304379085), np.float64(0.3333570075193866), np.float64(0.07656795703837219), np.float64(0.3343701939070786), np.float64(0.568717674572388), np.float64(0.1973842219941603), np.float64(0.07716981633329767), np.float64(0.04862952157381154), np.float64(0.44360949397630695)]
[np.float64(1.199117427080588), np.float64(1.155415750712769), np.float64(1.0880867890273216), np.float64(0.6065284197132054), np.float64(0.37796334312292557), np.float64(0.1770223180266911), np.float64(0.48572637424963916), np.float64(0.6717754724053021), np.float64(0.38535063725911683), np.float64(0.1773071477410518), np.float64(0.06191062058012285), np.float64(0.5153353845843236)]


#### Mean values of lower bound and upper bound


In [134]:
mean_values_ci_brits_male = toolkits.calc_mean_values_ci(lower_bounds_brits_male, upper_bounds_brits_male)

print(mean_values_ci_brits_male)

[np.float64(0.864627730010006), np.float64(0.7423430098424875), np.float64(0.6980216999350878), np.float64(0.5242069263784981), np.float64(0.35566017532115607), np.float64(0.12679513753253163), np.float64(0.4100482840783589), np.float64(0.620246573488845), np.float64(0.2913674296266386), np.float64(0.12723848203717475), np.float64(0.0552700710769672), np.float64(0.47947243928031524)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [135]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_brits_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_brits_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [136]:
means_bootstraps_brits_more_than_or_equal_to_65_subgroup,standards_deviations_brits_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_brits_more_than_or_equal_to_65_subgroup)
print(standards_deviations_brits_more_than_or_equal_to_65_subgroup)

[np.float64(0.662244906270694), np.float64(0.33551822688304433), np.float64(0.6362342133086535), np.float64(0.6383574162849505), np.float64(0.23878634109180352), np.float64(0.924878783743687), np.float64(0.5731951031171467)]
[np.float64(0.08841667646254166), np.float64(0.0631813692473505), np.float64(0.14098598742203416), np.float64(0.24085299923210227), np.float64(0.025378362701790873), np.float64(0.2879896525743772), np.float64(0.0619036891476668)]


#### Calculating lower bound and upper bound


In [137]:
lower_bounds_brits_more_than_or_equal_to_65_subgroup, upper_bounds_brits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_more_than_or_equal_to_65_subgroup, means_bootstraps_brits_more_than_or_equal_to_65_subgroup,standards_deviations_brits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_brits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_brits_more_than_or_equal_to_65_subgroup)


[np.float64(0.488951404771382), np.float64(0.2116850186643108), np.float64(0.3599067556366495), np.float64(0.16629421222157675), np.float64(0.1890456642096988), np.float64(0.3604294367777049), np.float64(0.4518661018775568)]
[np.float64(0.835538407770006), np.float64(0.4593514351017779), np.float64(0.9125616709806574), np.float64(1.1104206203483242), np.float64(0.28852701797390823), np.float64(1.4893281307096693), np.float64(0.6945241043567366)]


In [138]:
lower_bounds_brits_more_than_or_equal_to_65_subgroup, upper_bounds_brits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_brits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_brits_more_than_or_equal_to_65_subgroup)

[np.float64(0.49013614681391765), np.float64(0.22848200592268603), np.float64(0.39417516510179157), np.float64(0.3436040659088286), np.float64(0.1922309176367736), np.float64(0.461134543912614), np.float64(0.4554978714069311)]
[np.float64(0.8366997180001123), np.float64(0.47458613177319675), np.float64(0.9440111445954974), np.float64(1.1796209064031462), np.float64(0.29240414008122545), np.float64(1.5779181655450232), np.float64(0.6986735655061573)]


#### Mean values of lower bound and upper bound

In [139]:
mean_values_ci_brits_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_more_than_or_equal_to_65_subgroup , upper_bounds_brits_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_brits_more_than_or_equal_to_65_subgroup)

[np.float64(0.663417932407015), np.float64(0.35153406884794136), np.float64(0.6690931548486445), np.float64(0.7616124861559874), np.float64(0.24231752885899954), np.float64(1.0195263547288187), np.float64(0.5770857184565442)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [140]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_brits_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_brits_less_than_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [141]:
means_bootstraps_brits_less_than_65_subgroup,standards_deviations_brits_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_less_than_65_subgroup)

print(means_bootstraps_brits_less_than_65_subgroup)
print(standards_deviations_brits_less_than_65_subgroup)


[np.float64(0.7722524194006836), np.float64(0.36263183850017955), np.float64(1.084029737789501), np.float64(0.571527362778218), np.float64(0.4751239089993839), np.float64(0.5373403409647254), np.float64(0.7127001763464236)]
[np.float64(0.6086238944869353), np.float64(0.09109588956527907), np.float64(0.43935299070541206), np.float64(0.13437165851905025), np.float64(0.08710455881773665), np.float64(0.07056198341407932), np.float64(0.06180206326074808)]


#### Calculating lower bound and upper bound

In [142]:
lower_bounds_brits_less_than_65_subgroup, upper_bounds_brits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_less_than_65_subgroup, means_bootstraps_brits_less_than_65_subgroup, standards_deviations_brits_less_than_65_subgroup)

print(lower_bounds_brits_less_than_65_subgroup)
print(upper_bounds_brits_less_than_65_subgroup)

[np.float64(-0.42062849392421564), np.float64(0.18408717581259446), np.float64(0.22291369950693218), np.float64(0.3081637515379648), np.float64(0.30440211082736923), np.float64(0.3990413947954173), np.float64(0.5915703581850913)]
[np.float64(1.9651333327255829), np.float64(0.5411765011877646), np.float64(1.9451457760720698), np.float64(0.8348909740184712), np.float64(0.6458457071713986), np.float64(0.6756392871340335), np.float64(0.8338299945077559)]


In [143]:
lower_bounds_brits_less_than_65_subgroup, upper_bounds_brits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_less_than_65_subgroup)

print(lower_bounds_brits_less_than_65_subgroup)
print(upper_bounds_brits_less_than_65_subgroup)

[np.float64(0.0), np.float64(0.2074044801250295), np.float64(0.41237684781003814), np.float64(0.3415031837447295), np.float64(0.3212877496534465), np.float64(0.38080614880421926), np.float64(0.5948515926626379)]
[np.float64(1.6400321328986665), np.float64(0.5615428890114939), np.float64(2.08700193546815), np.float64(0.8688863074412774), np.float64(0.6621744792199287), np.float64(0.6432682344289811), np.float64(0.836412282791994)]


#### Mean values of lower bound and upper bound

In [144]:
mean_values_ci_brits_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_less_than_65_subgroup, upper_bounds_brits_less_than_65_subgroup)

print(mean_values_ci_brits_less_than_65_subgroup)

[np.float64(0.8200160664493332), np.float64(0.3844736845682617), np.float64(1.249689391639094), np.float64(0.6051947455930035), np.float64(0.4917311144366876), np.float64(0.5120371916166002), np.float64(0.7156319377273159)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [145]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_brits_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_1_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [146]:
means_bootstraps_brits_ICUType_1_subgroup, standards_deviations_brits_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_1_subgroup)

print(means_bootstraps_brits_ICUType_1_subgroup)
print(standards_deviations_brits_ICUType_1_subgroup)


[np.float64(0.5242391082650917), np.float64(1.3415657504066385), np.float64(0.9641261893849576), np.float64(0.22523224662477), np.float64(0.7214285617066842), np.float64(0.45725893867605666)]
[np.float64(0.179049986745374), np.float64(0.3732886535480633), np.float64(0.26648121535108293), np.float64(0.03475251277970779), np.float64(0.15381048461581723), np.float64(0.04360406748169409)]


#### Calculating lower bound and upper bound

In [147]:
lower_bounds_brits_ICUType_1_subgroup, upper_bounds_brits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_1_subgroup, means_bootstraps_brits_ICUType_1_subgroup, standards_deviations_brits_ICUType_1_subgroup)

print(lower_bounds_brits_ICUType_1_subgroup)
print(upper_bounds_brits_ICUType_1_subgroup)

[np.float64(0.1733075828117846), np.float64(0.6099334336149846), np.float64(0.4418326047403729), np.float64(0.15711857320427478), np.float64(0.4199655514150304), np.float64(0.3717965368324821)]
[np.float64(0.8751706337183988), np.float64(2.0731980671982924), np.float64(1.4864197740295424), np.float64(0.2933459200452653), np.float64(1.022891571998338), np.float64(0.5427213405196312)]


In [148]:
lower_bounds_brits_ICUType_1_subgroup, upper_bounds_brits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_1_subgroup)

print(lower_bounds_brits_ICUType_1_subgroup)
print(upper_bounds_brits_ICUType_1_subgroup)

[np.float64(0.2818701088590386), np.float64(0.6832919189918449), np.float64(0.5044988888435504), np.float64(0.15881062421212594), np.float64(0.4440415795895485), np.float64(0.375999018085907)]
[np.float64(0.9347071562797221), np.float64(2.1424587501889225), np.float64(1.545974285796128), np.float64(0.29692894998043906), np.float64(1.0435013017386077), np.float64(0.5459618084824918)]


#### Mean values of lower bound and upper bound

In [149]:
mean_values_ci_brits_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_1_subgroup, upper_bounds_brits_ICUType_1_subgroup)

print(mean_values_ci_brits_ICUType_1_subgroup)

[np.float64(0.6082886325693804), np.float64(1.4128753345903837), np.float64(1.0252365873198392), np.float64(0.2278697870962825), np.float64(0.7437714406640781), np.float64(0.46098041328419936)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [150]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_brits_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_2_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [151]:
means_bootstraps_brits_ICUType_2_subgroup, standards_deviations_brits_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_2_subgroup)

print(means_bootstraps_brits_ICUType_2_subgroup)
print(standards_deviations_brits_ICUType_2_subgroup)


[np.float64(0.5643987854289358), np.float64(0.3761977952224234), np.float64(0.0), np.float64(0.25400638345407145), np.float64(0.17284536767304798), np.float64(0.4397593172483772)]
[np.float64(0.2482394473633664), np.float64(0.12121432695131164), np.float64(0.0), np.float64(0.028671914446172952), np.float64(0.04785406762147616), np.float64(0.018046088644045204)]


#### Calculating lower bound and upper bound

In [152]:
lower_bounds_brits_ICUType_2_subgroup, upper_bounds_brits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_2_subgroup, means_bootstraps_brits_ICUType_2_subgroup, standards_deviations_brits_ICUType_2_subgroup)

print(lower_bounds_brits_ICUType_2_subgroup)
print(upper_bounds_brits_ICUType_2_subgroup)

[np.float64(0.07785840905461122), np.float64(0.1386220799875898), np.float64(0.0), np.float64(0.19781046377175876), np.float64(0.07905311862121037), np.float64(0.40438963344423134)]
[np.float64(1.0509391618032604), np.float64(0.613773510457257), np.float64(0.0), np.float64(0.31020230313638414), np.float64(0.2666376167248856), np.float64(0.47512900105252304)]


In [153]:
lower_bounds_brits_ICUType_2_subgroup, upper_bounds_brits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_2_subgroup)

#### Mean values of lower bound and upper bound

In [154]:
mean_values_ci_brits_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_2_subgroup, upper_bounds_brits_ICUType_2_subgroup)

print(mean_values_ci_brits_ICUType_2_subgroup)

[np.float64(0.7052698708754179), np.float64(0.3726295196823583), np.float64(0.0), np.float64(0.2556780334695357), np.float64(0.18326612954356436), np.float64(0.4404242902855008)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_brits_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_3_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_brits_ICUType_3_subgroup, standards_deviations_brits_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_3_subgroup)

print(means_bootstraps_brits_ICUType_3_subgroup)
print(standards_deviations_brits_ICUType_3_subgroup)


[np.float64(1.593236497237565), np.float64(1.1365310737170016), np.float64(0.6958192633459015), np.float64(0.5411579272281444), np.float64(0.23745879820743904)]
[np.float64(0.4559261164801034), np.float64(0.45097433675337983), np.float64(0.1878586172758027), np.float64(0.06133680568713781), np.float64(0.03548543506753726)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_brits_ICUType_3_subgroup, upper_bounds_brits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_3_subgroup, means_bootstraps_brits_ICUType_3_subgroup, standards_deviations_brits_ICUType_3_subgroup)

print(lower_bounds_brits_ICUType_3_subgroup)
print(upper_bounds_brits_ICUType_3_subgroup)

[np.float64(0.6996377293253487), np.float64(0.25263761572853904), np.float64(0.3276231392998342), np.float64(0.4209399971546227), np.float64(0.16790862349933133)]
[np.float64(2.4868352651497814), np.float64(2.020424531705464), np.float64(1.0640153873919687), np.float64(0.661375857301666), np.float64(0.30700897291554674)]


In [ ]:
lower_bounds_brits_ICUType_3_subgroup, upper_bounds_brits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_3_subgroup)

print(lower_bounds_brits_ICUType_3_subgroup)
print(upper_bounds_brits_ICUType_3_subgroup)

[np.float64(0.7811074732923107), np.float64(0.42602553654535447), np.float64(0.3370785960436855), np.float64(0.4225742806604089), np.float64(0.17750753035417519)]
[np.float64(2.5559583373020067), np.float64(2.1449893682804095), np.float64(1.1317699079657495), np.float64(0.664792763097355), np.float64(0.31561312193846874)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_brits_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_3_subgroup, upper_bounds_brits_ICUType_3_subgroup)

print(mean_values_ci_brits_ICUType_3_subgroup)

[np.float64(1.6685329052971587), np.float64(1.285507452412882), np.float64(0.7344242520047175), np.float64(0.543683521878882), np.float64(0.24656032614632195)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_brits_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_4_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_brits_ICUType_4_subgroup, standards_deviations_brits_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_4_subgroup)

print(means_bootstraps_brits_ICUType_4_subgroup)
print(standards_deviations_brits_ICUType_4_subgroup)


[np.float64(0.44306473111336137), np.float64(0.4912011878544669), np.float64(0.7287297527366805), np.float64(0.692426181621937), np.float64(0.18026286324155796)]
[np.float64(0.0935566892280583), np.float64(0.08074364451634707), np.float64(0.2681495472441715), np.float64(0.2724504216824265), np.float64(0.022189091660584512)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_brits_ICUType_4_subgroup, upper_bounds_brits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_4_subgroup, means_bootstraps_brits_ICUType_4_subgroup, standards_deviations_brits_ICUType_4_subgroup)

print(lower_bounds_brits_ICUType_4_subgroup)
print(upper_bounds_brits_ICUType_4_subgroup)

[np.float64(0.25969698971356064), np.float64(0.33294655262192163), np.float64(0.2031662976673827), np.float64(0.1584331675516304), np.float64(0.13677304273715427)]
[np.float64(0.626432472513162), np.float64(0.6494558230870122), np.float64(1.2542932078059783), np.float64(1.2264191956922437), np.float64(0.22375268374596166)]


In [ ]:
lower_bounds_brits_ICUType_4_subgroup, upper_bounds_brits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_4_subgroup)

print(lower_bounds_brits_ICUType_4_subgroup)
print(upper_bounds_brits_ICUType_4_subgroup)

[np.float64(0.2865496885836767), np.float64(0.34462669540294605), np.float64(0.22052486820293912), np.float64(0.031016071654553733), np.float64(0.13675082536379823)]
[np.float64(0.6531075788873016), np.float64(0.6642258024673423), np.float64(1.2816077240763408), np.float64(1.1697815665096722), np.float64(0.22392540460300034)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_brits_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_4_subgroup, upper_bounds_brits_ICUType_4_subgroup)

print(mean_values_ci_brits_ICUType_4_subgroup)

[np.float64(0.46982863373548917), np.float64(0.5044262489351442), np.float64(0.7510662961396399), np.float64(0.600398819082113), np.float64(0.1803381149833993)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_undefined_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_brits_undefined_classification_subgroup, standards_deviations_brits_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_undefined_classification_subgroup)

print(means_bootstraps_brits_undefined_classification_subgroup)
print(standards_deviations_brits_undefined_classification_subgroup)


[np.float64(0.4068612373822693), np.float64(0.5736205115093942), np.float64(0.275919003680376), np.float64(0.8430104136478819), np.float64(0.4131966854604587)]
[np.float64(0.05986934359644469), np.float64(0.17840974136243826), np.float64(0.044640773548291844), np.float64(0.2853216956527157), np.float64(0.12549413739177268)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_brits_undefined_classification_subgroup, upper_bounds_brits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_undefined_classification_subgroup, means_bootstraps_brits_undefined_classification_subgroup, standards_deviations_brits_undefined_classification_subgroup)

print(lower_bounds_brits_undefined_classification_subgroup)
print(upper_bounds_brits_undefined_classification_subgroup)

[np.float64(0.28951948015518403), np.float64(0.22394384394790923), np.float64(0.18842469528371564), np.float64(0.28379016616066055), np.float64(0.16723269590166293)]
[np.float64(0.5242029946093546), np.float64(0.9232971790708792), np.float64(0.3634133120770363), np.float64(1.4022306611351032), np.float64(0.6591606750192545)]


In [ ]:
lower_bounds_brits_undefined_classification_subgroup, upper_bounds_brits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_undefined_classification_subgroup)

print(lower_bounds_brits_undefined_classification_subgroup)
print(upper_bounds_brits_undefined_classification_subgroup)

[np.float64(0.2979609801580593), np.float64(0.24726862471409536), np.float64(0.2040999395523388), np.float64(0.40619839098934485), np.float64(0.2171298071300369)]
[np.float64(0.5319916259081999), np.float64(0.9465467461362519), np.float64(0.3763968339372512), np.float64(1.4766778181071094), np.float64(0.6956717509434502)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_brits_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_undefined_classification_subgroup, upper_bounds_brits_undefined_classification_subgroup)

print(mean_values_ci_brits_undefined_classification_subgroup)

[np.float64(0.4149763030331296), np.float64(0.5969076854251736), np.float64(0.29024838674479503), np.float64(0.9414381045482272), np.float64(0.4564007790367436)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_low_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_brits_low_weight_classification_subgroup, standards_deviations_brits_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_low_weight_classification_subgroup)

print(means_bootstraps_brits_low_weight_classification_subgroup)
print(standards_deviations_brits_low_weight_classification_subgroup)


[np.float64(2.0122806935650077), np.float64(0.06933332178741825), np.float64(0.4104061358032636), np.float64(0.0), np.float64(1.4606367473586466)]
[np.float64(1.666676645356208), np.float64(0.05309602591127397), np.float64(0.3139178166078663), np.float64(0.0), np.float64(1.2916342659860158)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_brits_low_weight_classification_subgroup, upper_bounds_brits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_low_weight_classification_subgroup, means_bootstraps_brits_low_weight_classification_subgroup, standards_deviations_brits_low_weight_classification_subgroup)

print(lower_bounds_brits_low_weight_classification_subgroup)
print(upper_bounds_brits_low_weight_classification_subgroup)

[np.float64(-1.2543455052071963), np.float64(-0.03473297672088424), np.float64(-0.2048614788536039), np.float64(0.0), np.float64(-1.070919895171773)]
[np.float64(5.278906892337211), np.float64(0.17339962029572073), np.float64(1.0256737504601312), np.float64(0.0), np.float64(3.9921933898890662)]


In [ ]:
lower_bounds_brits_low_weight_classification_subgroup, upper_bounds_brits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_low_weight_classification_subgroup)

print(lower_bounds_brits_low_weight_classification_subgroup)
print(upper_bounds_brits_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.020133205116918334)]
[np.float64(4.413896351691468), np.float64(0.10999469347559115), np.float64(0.6505204688677284), np.float64(0.0), np.float64(4.452386787366754)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_brits_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_low_weight_classification_subgroup, upper_bounds_brits_low_weight_classification_subgroup)

print(mean_values_ci_brits_low_weight_classification_subgroup)

[np.float64(2.206948175845734), np.float64(0.05499734673779558), np.float64(0.3252602344338642), np.float64(0.0), np.float64(2.236259996241836)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_normal_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_brits_normal_weight_classification_subgroup, standards_deviations_brits_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_normal_weight_classification_subgroup)

print(means_bootstraps_brits_normal_weight_classification_subgroup)
print(standards_deviations_brits_normal_weight_classification_subgroup)


[np.float64(0.548686639105505), np.float64(0.7625897872503594), np.float64(0.24931976137388573), np.float64(0.5648888072341054), np.float64(0.5232857652420211)]
[np.float64(0.12623025707018218), np.float64(0.3731308327837512), np.float64(0.07341434562099826), np.float64(0.089947135630567), np.float64(0.2013724566774524)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_brits_normal_weight_classification_subgroup, upper_bounds_brits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_normal_weight_classification_subgroup, means_bootstraps_brits_normal_weight_classification_subgroup, standards_deviations_brits_normal_weight_classification_subgroup)

print(lower_bounds_brits_normal_weight_classification_subgroup)
print(upper_bounds_brits_normal_weight_classification_subgroup)

[np.float64(0.3012798814887154), np.float64(0.03126679347276984), np.float64(0.10543028800815332), np.float64(0.38859566088565467), np.float64(0.12860300267586205)]
[np.float64(0.7960933967222945), np.float64(1.4939127810279489), np.float64(0.3932092347396181), np.float64(0.7411819535825561), np.float64(0.9179685278081802)]


In [ ]:
lower_bounds_brits_normal_weight_classification_subgroup, upper_bounds_brits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_normal_weight_classification_subgroup)

print(lower_bounds_brits_normal_weight_classification_subgroup)
print(upper_bounds_brits_normal_weight_classification_subgroup)

[np.float64(0.34098181461031984), np.float64(0.0), np.float64(0.13019003204955729), np.float64(0.3943044247772959), np.float64(0.20193025625589317)]
[np.float64(0.822822162147693), np.float64(1.1889972866535945), np.float64(0.41316823841970496), np.float64(0.7470447554607124), np.float64(0.9848085395794249)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_brits_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_normal_weight_classification_subgroup, upper_bounds_brits_normal_weight_classification_subgroup)

print(mean_values_ci_brits_normal_weight_classification_subgroup)

[np.float64(0.5819019883790064), np.float64(0.5944986433267972), np.float64(0.2716791352346311), np.float64(0.5706745901190041), np.float64(0.5933693979176591)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_overweight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_brits_overweight_classification_subgroup, standards_deviations_brits_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_overweight_classification_subgroup)

print(means_bootstraps_brits_overweight_classification_subgroup)
print(standards_deviations_brits_overweight_classification_subgroup)


[np.float64(0.2797503039647862), np.float64(1.4869184261643544), np.float64(0.44444907661681887), np.float64(1.0414359325082165), np.float64(0.1969866594273391)]
[np.float64(0.05690905629749502), np.float64(0.2748051570149171), np.float64(0.190452089816047), np.float64(0.5659200713279424), np.float64(0.06465085722820384)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_brits_overweight_classification_subgroup, upper_bounds_brits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_overweight_classification_subgroup, means_bootstraps_brits_overweight_classification_subgroup, standards_deviations_brits_overweight_classification_subgroup)

print(lower_bounds_brits_overweight_classification_subgroup)
print(upper_bounds_brits_overweight_classification_subgroup)

[np.float64(0.16821060322753356), np.float64(0.9483102156492423), np.float64(0.07116983979697916), np.float64(-0.06774702542288913), np.float64(0.07027330769041853)]
[np.float64(0.3912900047020388), np.float64(2.0255266366794666), np.float64(0.8177283134366586), np.float64(2.150618890439322), np.float64(0.3237000111642596)]


In [ ]:
lower_bounds_brits_overweight_classification_subgroup, upper_bounds_brits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_overweight_classification_subgroup)

print(lower_bounds_brits_overweight_classification_subgroup)
print(upper_bounds_brits_overweight_classification_subgroup)

[np.float64(0.18291051175428089), np.float64(0.9097202836560881), np.float64(0.14614286145696292), np.float64(0.3791864463768746), np.float64(0.10540658498615474)]
[np.float64(0.40407734173253634), np.float64(1.9873488893468048), np.float64(0.8647336127806425), np.float64(2.277614098374932), np.float64(0.3472386410647299)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_brits_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_overweight_classification_subgroup, upper_bounds_brits_overweight_classification_subgroup)

print(mean_values_ci_brits_overweight_classification_subgroup)

[np.float64(0.29349392674340863), np.float64(1.4485345865014465), np.float64(0.5054382371188028), np.float64(1.3284002723759034), np.float64(0.22632261302544232)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_obs1_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_brits_obs1_classification_subgroup, standards_deviations_brits_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_obs1_classification_subgroup)

print(means_bootstraps_brits_obs1_classification_subgroup)
print(standards_deviations_brits_obs1_classification_subgroup)


[np.float64(0.30729431457159123), np.float64(0.49879057873455573), np.float64(0.27486455646945995), np.float64(0.4929731039694339), np.float64(0.40518847013824805)]
[np.float64(0.05798396274724235), np.float64(0.25089896187847854), np.float64(0.08655489332963262), np.float64(0.19483046174517651), np.float64(0.12094499608671105)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_brits_obs1_classification_subgroup, upper_bounds_brits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_obs1_classification_subgroup, means_bootstraps_brits_obs1_classification_subgroup, standards_deviations_brits_obs1_classification_subgroup)

print(lower_bounds_brits_obs1_classification_subgroup)
print(upper_bounds_brits_obs1_classification_subgroup)

[np.float64(0.19364783590608406), np.float64(0.0070376496942498035), np.float64(0.10522008285767387), np.float64(0.11111241585757914), np.float64(0.1681406336979566)]
[np.float64(0.42094079323709843), np.float64(0.9905435077748617), np.float64(0.44450903008124604), np.float64(0.8748337920812886), np.float64(0.6422363065785395)]


In [ ]:
lower_bounds_brits_obs1_classification_subgroup, upper_bounds_brits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_obs1_classification_subgroup)

print(lower_bounds_brits_obs1_classification_subgroup)
print(upper_bounds_brits_obs1_classification_subgroup)

[np.float64(0.2017864229804127), np.float64(0.0), np.float64(0.12255724300388852), np.float64(0.0), np.float64(0.2152851960298434)]
[np.float64(0.4295377781988619), np.float64(1.0159483303577879), np.float64(0.4588269895582404), np.float64(0.621003118217197), np.float64(0.6770095865971394)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_brits_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_obs1_classification_subgroup, upper_bounds_brits_obs1_classification_subgroup)

print(mean_values_ci_brits_obs1_classification_subgroup)

[np.float64(0.3156621005896373), np.float64(0.5079741651788939), np.float64(0.29069211628106445), np.float64(0.3105015591085985), np.float64(0.4461473913134914)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_obs2_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_brits_obs2_classification_subgroup, standards_deviations_brits_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_obs2_classification_subgroup)

print(means_bootstraps_brits_obs2_classification_subgroup)
print(standards_deviations_brits_obs2_classification_subgroup)


[np.float64(0.2566297935741156), np.float64(0.12596516024051657), np.float64(1.3961240813101228), np.float64(0.0), np.float64(0.2858736875756382)]
[np.float64(0.049002577704985846), np.float64(0.09598273849535295), np.float64(1.0268585416236724), np.float64(0.0), np.float64(0.13790114187609237)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_brits_obs2_classification_subgroup, upper_bounds_brits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_obs2_classification_subgroup, means_bootstraps_brits_obs2_classification_subgroup, standards_deviations_brits_obs2_classification_subgroup)

print(lower_bounds_brits_obs2_classification_subgroup)
print(upper_bounds_brits_obs2_classification_subgroup)

[np.float64(0.1605865061227179), np.float64(-0.062157550347901436), np.float64(-0.6164816774895991), np.float64(0.0), np.float64(0.015592416071548876)]
[np.float64(0.35267308102551326), np.float64(0.3140878708289346), np.float64(3.408729840109845), np.float64(0.0), np.float64(0.5561549590797275)]


In [ ]:
lower_bounds_brits_obs2_classification_subgroup, upper_bounds_brits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_obs2_classification_subgroup)

print(lower_bounds_brits_obs2_classification_subgroup)
print(upper_bounds_brits_obs2_classification_subgroup)

[np.float64(0.15852749748120876), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.10914732115072498)]
[np.float64(0.35302742539780924), np.float64(0.1991019392632846), np.float64(3.5812912356313698), np.float64(0.0), np.float64(0.602445513613066)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_brits_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_obs2_classification_subgroup, upper_bounds_brits_obs2_classification_subgroup)

print(mean_values_ci_brits_obs2_classification_subgroup)

[np.float64(0.25577746143950897), np.float64(0.0995509696316423), np.float64(1.7906456178156849), np.float64(0.0), np.float64(0.3557964173818955)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_brits_obs3_classification_subgroup, standards_deviations_brits_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_obs3_classification_subgroup)

print(means_bootstraps_brits_obs3_classification_subgroup)
print(standards_deviations_brits_obs3_classification_subgroup)


[np.float64(0.15155417430747806), np.float64(0.0), np.float64(1.1593962687753612), np.float64(0.2442421185109458), np.float64(0.20886002502599546)]
[np.float64(0.050281311239894225), np.float64(0.0), np.float64(0.8237478596037178), np.float64(0.19004214803335054), np.float64(0.12978497928121874)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_obs3_classification_subgroup, means_bootstraps_brits_obs3_classification_subgroup, standards_deviations_brits_obs3_classification_subgroup)

print(lower_bounds_brits_obs3_classification_subgroup)
print(upper_bounds_brits_obs3_classification_subgroup)

[np.float64(0.05300461518183637), np.float64(0.0), np.float64(-0.4551198683898825), np.float64(-0.12823364717905072), np.float64(-0.045513860099470366)]
[np.float64(0.25010373343311976), np.float64(0.0), np.float64(2.773912405940605), np.float64(0.6167178842009423), np.float64(0.4632339101514613)]


In [ ]:
lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_obs3_classification_subgroup)

print(lower_bounds_brits_obs3_classification_subgroup)
print(upper_bounds_brits_obs3_classification_subgroup)

[np.float64(0.06726677849320917), np.float64(0.0), np.float64(0.15495381460601493), np.float64(0.0), np.float64(0.03269967207779254)]
[np.float64(0.2607674007481455), np.float64(0.0), np.float64(2.983221065811697), np.float64(0.3921118563324046), np.float64(0.487857401911803)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_brits_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup)

print(mean_values_ci_brits_obs3_classification_subgroup)

[np.float64(0.16401708962067732), np.float64(0.0), np.float64(1.569087440208856), np.float64(0.1960559281662023), np.float64(0.26027853699479775)]


## USGAN

### Calculate MAE and AE

In [ ]:
testing_mae_usgan_variables_standard, testing_ae_usgan_variables_standard = toolkits.calculate_mae(usgan_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for USGAN model

#### Difference between gender group

In [ ]:
diff_usgan_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 1, 2, variables)
diff_usgan_gender_subgroup

[('Cholesterol', 0.11396848832179907),
 ('K', 0.07962183727222638),
 ('Bilirubin', 0.07273530011716128),
 ('SaO2', 0.06724924437847979),
 ('AST', 0.06357150812579351)]

#### Difference between age group

In [ ]:
diff_usgan_age_subgroup = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 4, 5, variables)
diff_usgan_age_subgroup

[('Cholesterol', 1.4080332966852902),
 ('TroponinI', 1.2745449415570849),
 ('ALP', 0.46489726331108006),
 ('SaO2', 0.14584230629740058),
 ('Bilirubin', 0.14438651035533773)]

#### Difference between ICUType group

In [ ]:
diff_usgan_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 6, 7, variables)
diff_usgan_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 6, 8, variables)
diff_usgan_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 6, 9, variables)
diff_usgan_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 7, 8, variables)
diff_usgan_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 7, 9, variables)
diff_usgan_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 8, 9, variables)

In [ ]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_usgan_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_usgan_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_usgan_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_usgan_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_usgan_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_usgan_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 1.968532838973082), ('TroponinT', 0.3139744935635816), ('TroponinI', 0.2476191651050459), ('Glucose', 0.17361770275953042), ('Temp', 0.15812872602883854)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('TroponinI', 1.054054323556795), ('ALP', 0.9588980980944308), ('SaO2', 0.9557847823462609), ('Cholesterol', 0.720808558519628), ('TroponinT', 0.3983027553119586)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.8469546316747383), ('Cholesterol', 0.4515708800192406), ('TroponinT', 0.43928515289325093), ('ALP', 0.2595151866425361), ('Glucose', 0.1775868477420237)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('Cholesterol', 2.68934139749271), ('TroponinI', 1.3016734886618408), ('ALP', 0.9781535556167317), ('SaO2', 0.8722191782384636), ('Temp', 0.2501181579286637)]
----------------------------
Diferença(MAE

In [ ]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[6])
print(variables[31])
print(variables[0])
print(variables[28])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
Cholesterol
TroponinI
ALP
SaO2
TroponinT


#### Difference between BMI group

In [ ]:
diff_usgan_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 11, variables)
diff_usgan_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 12, variables)
diff_usgan_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 13, variables)
diff_usgan_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 14, variables)
diff_usgan_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 15, variables)
diff_usgan_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 16, variables)

diff_usgan_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 12, variables)
diff_usgan_classification_low_over = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 13, variables)
diff_usgan_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 14, variables)
diff_usgan_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 15, variables)
diff_usgan_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 16, variables)

diff_usgan_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 13, variables)
diff_usgan_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 14, variables)
diff_usgan_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 15, variables)
diff_usgan_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 16, variables)

diff_usgan_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 13, 14, variables)
diff_usgan_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 13, 15, variables)
diff_usgan_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 13, 16, variables)


diff_usgan_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 14, 15, variables)
diff_usgan_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 14, 16, variables)

diff_usgan_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 15, 16, variables)

In [ ]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_usgan_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_usgan_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_usgan_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_usgan_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_usgan_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_usgan_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.9859620009058019), ('TroponinI', 1.3663260793164698), ('Bilirubin', 1.0400217434134755), ('Cholesterol', 0.9605290481335007), ('SaO2', 0.3826989561208938)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('Cholesterol', 1.5579458557188144), ('TroponinI', 0.8172833755393478), ('SaO2', 0.5748494084312364), ('Platelets', 0.11877231573919644), ('ALP', 0.10391918734958416)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.7813610757706175), ('TroponinI', 0.528914503705096), ('SaO2', 0.26335364804066497), ('TroponinT', 0.15046354582798027), ('Bilirubin', 0.13730852400659188)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Cholesterol', 1.113359877891227), ('Ph', 0.8795575969438973), ('TroponinI', 0.5672432528016587), ('Albumin', 0.2786

In [ ]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_usgan_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_usgan_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_usgan_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_usgan_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_usgan_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('TroponinI', 2.1836094548558176), ('ALP', 1.8820428135562177), ('Bilirubin', 1.0046745883347163), ('SaO2', 0.9575483645521302), ('Cholesterol', 0.5974168075853137)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 2.057108090815585), ('TroponinI', 1.8952405830215657), ('Cholesterol', 1.7418901239041182), ('Bilirubin', 1.1773302674200674), ('Glucose', 0.41411061578867114)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.9622075950694193), ('TroponinI', 1.9335693321181284), ('Bilirubin', 1.2290397495761565), ('Ph', 0.8756933509066762), ('Glucose', 0.4445408934992038)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 2.2096916889234652), ('Bilirubin', 1.1530946428930196), ('TroponinT', 0.5311563166725247), ('HCO3', 0.38165713898274684), ('BUN', 0.3429542758853

In [ ]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_usgan_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_usgan_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_usgan_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_usgan_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('Cholesterol', 2.339306931489432), ('SaO2', 0.8382030564719014), ('TroponinI', 0.28836887183425186), ('Lactate', 0.17658435332815797), ('ALP', 0.17506527725936732)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8895493110798054), ('SaO2', 0.8483770038896792), ('Cholesterol', 0.44458597782758735), ('Albumin', 0.3456592406793608), ('TroponinI', 0.25004012273768916)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinI', 2.1836094548558176), ('Cholesterol', 0.9187488642412167), ('SaO2', 0.7611965781906), ('TroponinT', 0.6853385828101566), ('ALP', 0.32764887536724757)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('Cholesterol', 3.416495052966329), ('TroponinT', 0.7593013271876037), ('SaO2', 0.6919062266464278), ('ALT', 0.6246717920178768), ('Albumin', 0

In [ ]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_usgan_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_usgan_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_usgan_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Cholesterol', 1.8947209536618446), ('Ph', 0.8907999481313389), ('TroponinT', 0.3086857051131902), ('Albumin', 0.19249637129152009), ('ALP', 0.09490049574616588)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('TroponinI', 1.8952405830215657), ('Cholesterol', 1.4205580672482152), ('TroponinT', 0.5233142341164374), ('BUN', 0.22346096188397796), ('ALT', 0.16362555313936245)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.077188121476897), ('TroponinT', 0.5972769784938845), ('ALT', 0.5418725098570927), ('K', 0.15370072454147682), ('SaO2', 0.14629682982547365)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_usgan_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_usgan_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinI', 1.9335693321181284), ('Ph', 0.8942541473568749), ('TroponinT', 0.8319999392296276), ('Cholesterol', 0.47416288641362936), ('ALT', 0.25324161013363694)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Cholesterol', 2.9719090751387416), ('TroponinT', 0.9059626836070747), ('Ph', 0.8849431181997474), ('ALT', 0.6314885668513672), ('SaO2', 0.15647077724325148)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_usgan_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('Cholesterol', 2.4977461887251122), ('TroponinI', 2.02984317003595), ('ALT', 0.37824695671773023), ('ALP', 0.18814526095099215), ('BUN', 0.1542478468532586)]
----------------------------


In [ ]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[6])
print(variables[31])
print(variables[0])
print(variables[32])
print(variables[5])

Top 5 variáveis BMI
-----------------------
Cholesterol
TroponinI
ALP
TroponinT
Bilirubin


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_usgan_female = []

for i in indices:
       results_bootstrap_usgan_female.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_female,standards_deviations_usgan_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_female)

print(means_bootstraps_usgan_female)
print(standards_deviations_usgan_female)


[np.float64(1.9616424284201106), np.float64(0.6196154208125143), np.float64(0.519483116298636), np.float64(0.5851034594231386), np.float64(0.2973570783183189)]
[np.float64(0.965325843700134), np.float64(0.032467414392818306), np.float64(0.05648531272365469), np.float64(0.13398574136774344), np.float64(0.04907430379125418)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_female, upper_bounds_usgan_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_female, means_bootstraps_usgan_female,standards_deviations_usgan_female)

print(lower_bounds_usgan_female)
print(upper_bounds_usgan_female)

[np.float64(0.06963854142210657), np.float64(0.555980457931453), np.float64(0.4087739377047907), np.float64(0.322496231900463), np.float64(0.20117321032108332)]
[np.float64(3.8536463154181146), np.float64(0.6832503836935756), np.float64(0.6301922948924812), np.float64(0.8477106869458142), np.float64(0.39354094631555453)]


In [ ]:
lower_bounds_usgan_female, upper_bounds_usgan_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_female)

print(lower_bounds_usgan_female)
print(upper_bounds_usgan_female)

[np.float64(0.552297478025096), np.float64(0.5583214267508602), np.float64(0.4246632690220314), np.float64(0.35896940420117857), np.float64(0.2149279902218852)]
[np.float64(4.152961981190631), np.float64(0.6866208232404697), np.float64(0.6421835107886564), np.float64(0.8798822431398159), np.float64(0.40616254661494894)]


#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_usgan_female = toolkits.calc_mean_values_ci(lower_bounds_usgan_female, upper_bounds_usgan_female)

print(mean_values_ci_usgan_female)

[np.float64(2.3526297296078638), np.float64(0.6224711249956649), np.float64(0.5334233899053439), np.float64(0.6194258236704973), np.float64(0.31054526841841706)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_usgan_male = []

for i in indices:
       results_bootstrap_usgan_male.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_male,standards_deviations_usgan_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_male)

print(means_bootstraps_usgan_male)
print(standards_deviations_usgan_male)


[np.float64(2.0767878054267155), np.float64(0.5399034251040454), np.float64(0.44657963430801506), np.float64(0.6493438911986597), np.float64(0.23359850222558343)]
[np.float64(0.44038932298507266), np.float64(0.021861564734443558), np.float64(0.03845395415341268), np.float64(0.28618210212385997), np.float64(0.020229866887666283)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_male, upper_bounds_usgan_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_male, means_bootstraps_usgan_male,standards_deviations_usgan_male)

print(lower_bounds_usgan_male)
print(upper_bounds_usgan_male)

[np.float64(1.2136405931999956), np.float64(0.4970555455788451), np.float64(0.3712112691041718), np.float64(0.08843727801593038), np.float64(0.19394869171371812)]
[np.float64(2.9399350176534353), np.float64(0.5827513046292457), np.float64(0.5219479995118583), np.float64(1.210250504381389), np.float64(0.27324831273744876)]


In [ ]:
lower_bounds_usgan_male, upper_bounds_usgan_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_male)

print(lower_bounds_usgan_male)
print(upper_bounds_usgan_male)

[np.float64(1.238113116845262), np.float64(0.4985683179157677), np.float64(0.37446853780280714), np.float64(0.32087931106030454), np.float64(0.19532264990749149)]
[np.float64(2.9717877013948577), np.float64(0.5836320625746543), np.float64(0.5260074341746146), np.float64(1.2826079294368584), np.float64(0.27470822843533443)]


#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_usgan_male = toolkits.calc_mean_values_ci(lower_bounds_usgan_male, upper_bounds_usgan_male)

print(mean_values_ci_usgan_male)

[np.float64(2.1049504091200597), np.float64(0.541100190245211), np.float64(0.4502379859887109), np.float64(0.8017436202485815), np.float64(0.23501543917141296)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_usgan_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_usgan_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_more_than_or_equal_to_65_subgroup,standards_deviations_usgan_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_usgan_more_than_or_equal_to_65_subgroup)
print(standards_deviations_usgan_more_than_or_equal_to_65_subgroup)

[np.float64(2.3189420507457585), np.float64(2.4760165772884775), np.float64(0.6952501846735629), np.float64(0.712587400371639), np.float64(0.4595658035567099)]
[np.float64(0.34910444009022223), np.float64(0.42713917518827815), np.float64(0.14304532306086748), np.float64(0.31063401441857796), np.float64(0.030540235166214794)]


#### Calculating lower bound and upper bound


In [ ]:
lower_bounds_usgan_more_than_or_equal_to_65_subgroup, upper_bounds_usgan_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_more_than_or_equal_to_65_subgroup, means_bootstraps_usgan_more_than_or_equal_to_65_subgroup,standards_deviations_usgan_more_than_or_equal_to_65_subgroup)

print(lower_bounds_usgan_more_than_or_equal_to_65_subgroup)
print(upper_bounds_usgan_more_than_or_equal_to_65_subgroup)


[np.float64(1.634709921325902), np.float64(1.6388391775333075), np.float64(0.4148865033173658), np.float64(0.10375591973813036), np.float64(0.39970804255154524)]
[np.float64(3.0031741801656153), np.float64(3.3131939770436474), np.float64(0.97561386602976), np.float64(1.3214188810051477), np.float64(0.5194235645618746)]


In [ ]:
lower_bounds_usgan_more_than_or_equal_to_65_subgroup, upper_bounds_usgan_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_more_than_or_equal_to_65_subgroup)

print(lower_bounds_usgan_more_than_or_equal_to_65_subgroup)
print(upper_bounds_usgan_more_than_or_equal_to_65_subgroup)

[np.float64(1.6099234359465588), np.float64(1.6892337738417373), np.float64(0.4444906483603086), np.float64(0.341050868770908), np.float64(0.3994074614007054)]
[np.float64(2.994196971516218), np.float64(3.3762866598090917), np.float64(1.0030284452508729), np.float64(1.4208146456112776), np.float64(0.5204216027429522)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_more_than_or_equal_to_65_subgroup , upper_bounds_usgan_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_usgan_more_than_or_equal_to_65_subgroup)

[np.float64(2.3020602037313886), np.float64(2.5327602168254146), np.float64(0.7237595468055907), np.float64(0.8809327571910928), np.float64(0.4599145320718288)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_usgan_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_usgan_less_than_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_less_than_65_subgroup,standards_deviations_usgan_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_less_than_65_subgroup)

print(means_bootstraps_usgan_less_than_65_subgroup)
print(standards_deviations_usgan_less_than_65_subgroup)


[np.float64(3.225317594154863), np.float64(1.20019222042771), np.float64(1.1556052262786836), np.float64(0.569475045824097), np.float64(0.6041039979755715)]
[np.float64(1.8323129316205036), np.float64(0.1183567487219799), np.float64(0.44011750800878735), np.float64(0.13057105724065), np.float64(0.07977804165399471)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_less_than_65_subgroup, upper_bounds_usgan_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_less_than_65_subgroup, means_bootstraps_usgan_less_than_65_subgroup, standards_deviations_usgan_less_than_65_subgroup)

print(lower_bounds_usgan_less_than_65_subgroup)
print(upper_bounds_usgan_less_than_65_subgroup)

[np.float64(-0.3659497602283266), np.float64(0.9682172556053723), np.float64(0.2929907616159415), np.float64(0.3135604762091052), np.float64(0.4477419095766056)]
[np.float64(6.816584948538052), np.float64(1.4321671852500477), np.float64(2.0182196909414256), np.float64(0.8253896154390888), np.float64(0.7604660863745374)]


In [ ]:
lower_bounds_usgan_less_than_65_subgroup, upper_bounds_usgan_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_less_than_65_subgroup)

print(lower_bounds_usgan_less_than_65_subgroup)
print(upper_bounds_usgan_less_than_65_subgroup)

[np.float64(0.0), np.float64(0.9482049976927617), np.float64(0.4815596373488781), np.float64(0.34789332365656317), np.float64(0.46094317985899436)]
[np.float64(5.588981827530834), np.float64(1.409402096580033), np.float64(2.1470924734025645), np.float64(0.8643084231459723), np.float64(0.7719838817787542)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_less_than_65_subgroup, upper_bounds_usgan_less_than_65_subgroup)

print(mean_values_ci_usgan_less_than_65_subgroup)

[np.float64(2.794490913765417), np.float64(1.1788035471363973), np.float64(1.3143260553757212), np.float64(0.6061008734012677), np.float64(0.6164635308188743)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_usgan_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_1_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_ICUType_1_subgroup, standards_deviations_usgan_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_1_subgroup)

print(means_bootstraps_usgan_ICUType_1_subgroup)
print(standards_deviations_usgan_ICUType_1_subgroup)


[np.float64(1.969871597812144), np.float64(2.3676683028385983), np.float64(0.2523486811328111), np.float64(0.5596005919406728), np.float64(0.7458173996840364)]
[np.float64(0.3423238479681623), np.float64(0.5261676094824145), np.float64(0.04331120976644776), np.float64(0.17453562287196972), np.float64(0.11562132472621121)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_ICUType_1_subgroup, upper_bounds_usgan_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_1_subgroup, means_bootstraps_usgan_ICUType_1_subgroup, standards_deviations_usgan_ICUType_1_subgroup)

print(lower_bounds_usgan_ICUType_1_subgroup)
print(upper_bounds_usgan_ICUType_1_subgroup)

[np.float64(1.298929184745381), np.float64(1.3363987384215301), np.float64(0.16746026986371404), np.float64(0.2175170570923468), np.float64(0.5192037673758519)]
[np.float64(2.640814010878907), np.float64(3.3989378672556665), np.float64(0.3372370924019082), np.float64(0.9016841267889988), np.float64(0.9724310319922208)]


In [ ]:
lower_bounds_usgan_ICUType_1_subgroup, upper_bounds_usgan_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_1_subgroup)

print(lower_bounds_usgan_ICUType_1_subgroup)
print(upper_bounds_usgan_ICUType_1_subgroup)

[np.float64(1.3031627380681403), np.float64(1.465985393678674), np.float64(0.17499983514773637), np.float64(0.3213709315103343), np.float64(0.536055047520922)]
[np.float64(2.651869332243632), np.float64(3.4981248221816084), np.float64(0.3440748089407272), np.float64(0.9738938630764534), np.float64(0.9856625744890138)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_1_subgroup, upper_bounds_usgan_ICUType_1_subgroup)

print(mean_values_ci_usgan_ICUType_1_subgroup)

[np.float64(1.9775160351558863), np.float64(2.482055107930141), np.float64(0.25953732204423174), np.float64(0.6476323972933938), np.float64(0.7608588110049679)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_usgan_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_2_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_ICUType_2_subgroup, standards_deviations_usgan_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_2_subgroup)

print(means_bootstraps_usgan_ICUType_2_subgroup)
print(standards_deviations_usgan_ICUType_2_subgroup)


[np.float64(0.0), np.float64(2.590797588030485), np.float64(0.23274430781656597), np.float64(0.6418400993401857), np.float64(0.43159246622787867)]
[np.float64(0.0), np.float64(0.2738668122740247), np.float64(0.043792901603823924), np.float64(0.32173876707404403), np.float64(0.09300144922232004)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_ICUType_2_subgroup, upper_bounds_usgan_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_2_subgroup, means_bootstraps_usgan_ICUType_2_subgroup, standards_deviations_usgan_ICUType_2_subgroup)

print(lower_bounds_usgan_ICUType_2_subgroup)
print(upper_bounds_usgan_ICUType_2_subgroup)

[np.float64(0.0), np.float64(2.0540284994126043), np.float64(0.1469117978945647), np.float64(0.011243703444738018), np.float64(0.24931297524210078)]
[np.float64(0.0), np.float64(3.1275666766483656), np.float64(0.3185768177385672), np.float64(1.2724364952356333), np.float64(0.6138719572136566)]


In [ ]:
lower_bounds_usgan_ICUType_2_subgroup, upper_bounds_usgan_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_2_subgroup)

print(lower_bounds_usgan_ICUType_2_subgroup)
print(upper_bounds_usgan_ICUType_2_subgroup)

[np.float64(0.0), np.float64(2.206764978757124), np.float64(0.15108772752516886), np.float64(0.2932308055904985), np.float64(0.2526191046289952)]
[np.float64(0.0), np.float64(2.9590725029529943), np.float64(0.3244996368151167), np.float64(1.3661365312260427), np.float64(0.6190836468529262)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_2_subgroup, upper_bounds_usgan_ICUType_2_subgroup)

print(mean_values_ci_usgan_ICUType_2_subgroup)

[np.float64(0.0), np.float64(2.5829187408550593), np.float64(0.23779368217014277), np.float64(0.8296836684082707), np.float64(0.43585137574096067)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_usgan_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_3_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_ICUType_3_subgroup, standards_deviations_usgan_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_3_subgroup)

print(means_bootstraps_usgan_ICUType_3_subgroup)
print(standards_deviations_usgan_ICUType_3_subgroup)


[np.float64(2.638459020641269), np.float64(1.3077305771955074), np.float64(1.2067516011716777), np.float64(1.514706833854234), np.float64(0.3485654500010878)]
[np.float64(0.8708552536368372), np.float64(0.18664261261884646), np.float64(0.45216644752008617), np.float64(0.424133988411211), np.float64(0.035573694579512774)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_ICUType_3_subgroup, upper_bounds_usgan_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_3_subgroup, means_bootstraps_usgan_ICUType_3_subgroup, standards_deviations_usgan_ICUType_3_subgroup)

print(lower_bounds_usgan_ICUType_3_subgroup)
print(upper_bounds_usgan_ICUType_3_subgroup)

[np.float64(0.9316140877655739), np.float64(0.9419177784821073), np.float64(0.32052164901488833), np.float64(0.6834194919489317), np.float64(0.278842289828215)]
[np.float64(4.3453039535169635), np.float64(1.6735433759089076), np.float64(2.092981553328467), np.float64(2.3459941757595364), np.float64(0.4182886101739606)]


In [ ]:
lower_bounds_usgan_ICUType_3_subgroup, upper_bounds_usgan_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_3_subgroup)

print(lower_bounds_usgan_ICUType_3_subgroup)
print(upper_bounds_usgan_ICUType_3_subgroup)

[np.float64(0.34585239975461424), np.float64(0.9436379608808099), np.float64(0.5096642242756344), np.float64(0.7619652050134906), np.float64(0.2826252559261122)]
[np.float64(4.104556110989548), np.float64(1.6808424416961543), np.float64(2.2224704663916817), np.float64(2.4089454303397533), np.float64(0.42087910960409913)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_3_subgroup, upper_bounds_usgan_ICUType_3_subgroup)

print(mean_values_ci_usgan_ICUType_3_subgroup)

[np.float64(2.225204255372081), np.float64(1.312240201288482), np.float64(1.366067345333658), np.float64(1.585455317676622), np.float64(0.35175218276510567)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_usgan_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_4_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_ICUType_4_subgroup, standards_deviations_usgan_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_4_subgroup)

print(means_bootstraps_usgan_ICUType_4_subgroup)
print(standards_deviations_usgan_ICUType_4_subgroup)


[np.float64(2.410112355451811), np.float64(1.4854589866266972), np.float64(0.511271837947904), np.float64(0.4778724768350977), np.float64(0.30802591708887417)]
[np.float64(0.4328661857164766), np.float64(0.6234774392830332), np.float64(0.08542946698184836), np.float64(0.07328044561798011), np.float64(0.04044433612915916)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_ICUType_4_subgroup, upper_bounds_usgan_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_4_subgroup, means_bootstraps_usgan_ICUType_4_subgroup, standards_deviations_usgan_ICUType_4_subgroup)

print(lower_bounds_usgan_ICUType_4_subgroup)
print(upper_bounds_usgan_ICUType_4_subgroup)

[np.float64(1.5617102213222904), np.float64(0.2634656604586938), np.float64(0.34383315944502746), np.float64(0.33424544265281064), np.float64(0.2287564748970901)]
[np.float64(3.2585144895813314), np.float64(2.7074523127947003), np.float64(0.6787105164507805), np.float64(0.6214995110173848), np.float64(0.3872953592806582)]


In [ ]:
lower_bounds_usgan_ICUType_4_subgroup, upper_bounds_usgan_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_4_subgroup)

print(lower_bounds_usgan_ICUType_4_subgroup)
print(upper_bounds_usgan_ICUType_4_subgroup)

[np.float64(1.6095033807737984), np.float64(0.2032850750806556), np.float64(0.3570144472366515), np.float64(0.3507129401966817), np.float64(0.22846375904233956)]
[np.float64(3.2952410498193156), np.float64(3.0892252334427197), np.float64(0.6891759488794426), np.float64(0.6387310934261473), np.float64(0.38788499340192834)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_4_subgroup, upper_bounds_usgan_ICUType_4_subgroup)

print(mean_values_ci_usgan_ICUType_4_subgroup)

[np.float64(2.452372215296557), np.float64(1.6462551542616877), np.float64(0.523095198058047), np.float64(0.4947220168114145), np.float64(0.30817437622213395)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_undefined_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_undefined_classification_subgroup, standards_deviations_usgan_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_undefined_classification_subgroup)

print(means_bootstraps_usgan_undefined_classification_subgroup)
print(standards_deviations_usgan_undefined_classification_subgroup)


[np.float64(1.8633611646719113), np.float64(1.3648697644480443), np.float64(0.4309562068509655), np.float64(0.3795004656085497), np.float64(0.5879890600583709)]
[np.float64(0.2629785629509782), np.float64(0.1799536224861461), np.float64(0.05950892548445381), np.float64(0.0401617233008771), np.float64(0.1097447477015649)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_undefined_classification_subgroup, upper_bounds_usgan_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_undefined_classification_subgroup, means_bootstraps_usgan_undefined_classification_subgroup, standards_deviations_usgan_undefined_classification_subgroup)

print(lower_bounds_usgan_undefined_classification_subgroup)
print(upper_bounds_usgan_undefined_classification_subgroup)

[np.float64(1.3479326525818944), np.float64(1.0121671454876808), np.float64(0.31432085614275823), np.float64(0.3007849343817675), np.float64(0.3728933070708689)]
[np.float64(2.378789676761928), np.float64(1.7175723834084078), np.float64(0.5475915575591728), np.float64(0.45821599683533193), np.float64(0.803084813045873)]


In [ ]:
lower_bounds_usgan_undefined_classification_subgroup, upper_bounds_usgan_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_undefined_classification_subgroup)

print(lower_bounds_usgan_undefined_classification_subgroup)
print(upper_bounds_usgan_undefined_classification_subgroup)

[np.float64(1.350010606042941), np.float64(1.0209379525980362), np.float64(0.32230680246103255), np.float64(0.3107327182106672), np.float64(0.41380365681143066)]
[np.float64(2.373009597445674), np.float64(1.7117142060344472), np.float64(0.554778879554793), np.float64(0.46649491723393854), np.float64(0.8388898663345923)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_undefined_classification_subgroup, upper_bounds_usgan_undefined_classification_subgroup)

print(mean_values_ci_usgan_undefined_classification_subgroup)

[np.float64(1.8615101017443076), np.float64(1.3663260793162417), np.float64(0.43854284100791274), np.float64(0.38861381772230286), np.float64(0.6263467615730115)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_low_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_low_weight_classification_subgroup, standards_deviations_usgan_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_low_weight_classification_subgroup)

print(means_bootstraps_usgan_low_weight_classification_subgroup)
print(standards_deviations_usgan_low_weight_classification_subgroup)


[np.float64(1.7944999186435053), np.float64(0.0), np.float64(2.0705590744293545), np.float64(0.32734451188885255), np.float64(1.6036430213262178)]
[np.float64(1.3559519622669247), np.float64(0.0), np.float64(1.3236938821533002), np.float64(0.25211953106672513), np.float64(1.075376930577923)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_low_weight_classification_subgroup, upper_bounds_usgan_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_low_weight_classification_subgroup, means_bootstraps_usgan_low_weight_classification_subgroup, standards_deviations_usgan_low_weight_classification_subgroup)

print(lower_bounds_usgan_low_weight_classification_subgroup)
print(upper_bounds_usgan_low_weight_classification_subgroup)

[np.float64(-0.8631170921660813), np.float64(0.0), np.float64(-0.5238332611471206), np.float64(-0.16680068880105597), np.float64(-0.5040570324117415)]
[np.float64(4.4521169294530925), np.float64(0.0), np.float64(4.66495141000583), np.float64(0.821489712578761), np.float64(3.7113430750641774)]


In [ ]:
lower_bounds_usgan_low_weight_classification_subgroup, upper_bounds_usgan_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_low_weight_classification_subgroup)

print(lower_bounds_usgan_low_weight_classification_subgroup)
print(upper_bounds_usgan_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.27546796727308465)]
[np.float64(2.8190782453828946), np.float64(0.0), np.float64(3.8852331187690905), np.float64(0.5215260412464068), np.float64(4.0836010101688975)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_low_weight_classification_subgroup, upper_bounds_usgan_low_weight_classification_subgroup)

print(mean_values_ci_usgan_low_weight_classification_subgroup)

[np.float64(1.4095391226914473), np.float64(0.0), np.float64(1.9426165593845452), np.float64(0.2607630206232034), np.float64(2.179534488720991)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_normal_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_normal_weight_classification_subgroup, standards_deviations_usgan_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_normal_weight_classification_subgroup)

print(means_bootstraps_usgan_normal_weight_classification_subgroup)
print(standards_deviations_usgan_normal_weight_classification_subgroup)


[np.float64(2.9310843655917926), np.float64(2.181378781514663), np.float64(0.5338894428943264), np.float64(0.3672681202995015), np.float64(0.6224596631587035)]
[np.float64(1.20699229323851), np.float64(0.23906545341522484), np.float64(0.13730411739612933), np.float64(0.03730026655694922), np.float64(0.19259621824137446)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_normal_weight_classification_subgroup, upper_bounds_usgan_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_normal_weight_classification_subgroup, means_bootstraps_usgan_normal_weight_classification_subgroup, standards_deviations_usgan_normal_weight_classification_subgroup)

print(lower_bounds_usgan_normal_weight_classification_subgroup)
print(upper_bounds_usgan_normal_weight_classification_subgroup)

[np.float64(0.5654229412269052), np.float64(1.712819102873084), np.float64(0.2647783178688534), np.float64(0.29416094123413716), np.float64(0.24497801184699336)]
[np.float64(5.29674578995668), np.float64(2.6499384601562417), np.float64(0.8030005679197993), np.float64(0.44037529936486586), np.float64(0.9999413144704137)]


In [ ]:
lower_bounds_usgan_normal_weight_classification_subgroup, upper_bounds_usgan_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_normal_weight_classification_subgroup)

print(lower_bounds_usgan_normal_weight_classification_subgroup)
print(upper_bounds_usgan_normal_weight_classification_subgroup)

[np.float64(0.0), np.float64(1.7373360832788451), np.float64(0.2994476295652569), np.float64(0.29545333320431416), np.float64(0.32943927736916656)]
[np.float64(3.7219816477052636), np.float64(2.680109177430762), np.float64(0.8390208120967145), np.float64(0.4415446955246705), np.float64(1.054182119745126)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_normal_weight_classification_subgroup, upper_bounds_usgan_normal_weight_classification_subgroup)

print(mean_values_ci_usgan_normal_weight_classification_subgroup)

[np.float64(1.8609908238526318), np.float64(2.2087226303548038), np.float64(0.5692342208309857), np.float64(0.36849901436449234), np.float64(0.6918106985571463)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_overweight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_overweight_classification_subgroup, standards_deviations_usgan_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_overweight_classification_subgroup)

print(means_bootstraps_usgan_overweight_classification_subgroup)
print(standards_deviations_usgan_overweight_classification_subgroup)


[np.float64(1.075396532203622), np.float64(1.872042179464386), np.float64(0.35999031683120136), np.float64(0.5328450912704701), np.float64(0.4524863069473167)]
[np.float64(0.5104058795884194), np.float64(0.8247405396908457), np.float64(0.07920745904875176), np.float64(0.14756959271879605), np.float64(0.057714507927383904)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_overweight_classification_subgroup, upper_bounds_usgan_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_overweight_classification_subgroup, means_bootstraps_usgan_overweight_classification_subgroup, standards_deviations_usgan_overweight_classification_subgroup)

print(lower_bounds_usgan_overweight_classification_subgroup)
print(upper_bounds_usgan_overweight_classification_subgroup)

[np.float64(0.07501939071283248), np.float64(0.2555804250802014), np.float64(0.2047465497887167), np.float64(0.24361400432838565), np.float64(0.3393679500241928)]
[np.float64(2.0757736736944117), np.float64(3.4885039338485706), np.float64(0.515234083873686), np.float64(0.8220761782125545), np.float64(0.5656046638704406)]


In [ ]:
lower_bounds_usgan_overweight_classification_subgroup, upper_bounds_usgan_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_overweight_classification_subgroup)

print(lower_bounds_usgan_overweight_classification_subgroup)
print(upper_bounds_usgan_overweight_classification_subgroup)

[np.float64(0.06603806011135123), np.float64(0.4136570705153268), np.float64(0.2224299241349029), np.float64(0.29840420932497097), np.float64(0.34302140053056746)]
[np.float64(2.124373746769478), np.float64(3.66548434061988), np.float64(0.5314981302169055), np.float64(0.8606951032059154), np.float64(0.5716300933735432)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_overweight_classification_subgroup, upper_bounds_usgan_overweight_classification_subgroup)

print(mean_values_ci_usgan_overweight_classification_subgroup)

[np.float64(1.0952059034404147), np.float64(2.0395707055676033), np.float64(0.3769640271759042), np.float64(0.5795496562654432), np.float64(0.4573257469520553)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_obs1_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_obs1_classification_subgroup, standards_deviations_usgan_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_obs1_classification_subgroup)

print(means_bootstraps_usgan_obs1_classification_subgroup)
print(standards_deviations_usgan_obs1_classification_subgroup)


[np.float64(2.8218407362619407), np.float64(1.6731986326555415), np.float64(0.45443121552758176), np.float64(0.22074965653548329), np.float64(0.4002946698448305)]
[np.float64(0.7929785462493167), np.float64(0.8856841702093671), np.float64(0.07805545029024012), np.float64(0.06419008767981105), np.float64(0.08924269231922491)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_obs1_classification_subgroup, upper_bounds_usgan_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_obs1_classification_subgroup, means_bootstraps_usgan_obs1_classification_subgroup, standards_deviations_usgan_obs1_classification_subgroup)

print(lower_bounds_usgan_obs1_classification_subgroup)
print(upper_bounds_usgan_obs1_classification_subgroup)

[np.float64(1.2676313451003502), np.float64(-0.06271044263206105), np.float64(0.3014453441616546), np.float64(0.09493939651858538), np.float64(0.22538220701576034)]
[np.float64(4.376050127423531), np.float64(3.4091077079431438), np.float64(0.6074170868935089), np.float64(0.34655991655238116), np.float64(0.5752071326739006)]


In [ ]:
lower_bounds_usgan_obs1_classification_subgroup, upper_bounds_usgan_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_obs1_classification_subgroup)

print(lower_bounds_usgan_obs1_classification_subgroup)
print(upper_bounds_usgan_obs1_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.31453742203523133), np.float64(0.1025869189196057), np.float64(0.24021319647968892)]
[np.float64(3.550606754912776), np.float64(2.7761612905780666), np.float64(0.6210522778124141), np.float64(0.3528916967255257), np.float64(0.589960853315508)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_obs1_classification_subgroup, upper_bounds_usgan_obs1_classification_subgroup)

print(mean_values_ci_usgan_obs1_classification_subgroup)

[np.float64(1.775303377456388), np.float64(1.3880806452890333), np.float64(0.4677948499238227), np.float64(0.2277393078225657), np.float64(0.4150870248975984)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_obs2_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_obs2_classification_subgroup, standards_deviations_usgan_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_obs2_classification_subgroup)

print(means_bootstraps_usgan_obs2_classification_subgroup)
print(standards_deviations_usgan_obs2_classification_subgroup)


[np.float64(1.5927294863440795), np.float64(0.0), np.float64(0.20749292419050963), np.float64(1.0063747068837194), np.float64(0.47483049339833006)]
[np.float64(1.2006026767907316), np.float64(0.0), np.float64(0.05064881389349309), np.float64(0.8248253184585713), np.float64(0.07607719071200844)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_obs2_classification_subgroup, upper_bounds_usgan_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_obs2_classification_subgroup, means_bootstraps_usgan_obs2_classification_subgroup, standards_deviations_usgan_obs2_classification_subgroup)

print(lower_bounds_usgan_obs2_classification_subgroup)
print(upper_bounds_usgan_obs2_classification_subgroup)

[np.float64(-0.7604085199081374), np.float64(0.0), np.float64(0.10822307309959125), np.float64(-0.610253210831861), np.float64(0.32572193955780837)]
[np.float64(3.9458674925962964), np.float64(0.0), np.float64(0.30676277528142804), np.float64(2.6230026245993), np.float64(0.6239390472388517)]


In [ ]:
lower_bounds_usgan_obs2_classification_subgroup, upper_bounds_usgan_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_obs2_classification_subgroup)

print(lower_bounds_usgan_obs2_classification_subgroup)
print(upper_bounds_usgan_obs2_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.10697553345308664), np.float64(0.0), np.float64(0.32649892024799276)]
[np.float64(2.4977461887267776), np.float64(0.0), np.float64(0.3094408687724588), np.float64(2.7938387017906305), np.float64(0.625988375353698)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_obs2_classification_subgroup, upper_bounds_usgan_obs2_classification_subgroup)

print(mean_values_ci_usgan_obs2_classification_subgroup)

[np.float64(1.2488730943633888), np.float64(0.0), np.float64(0.20820820111277272), np.float64(1.3969193508953153), np.float64(0.4762436478008454)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_obs3_classification_subgroup, standards_deviations_usgan_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_obs3_classification_subgroup)

print(means_bootstraps_usgan_obs3_classification_subgroup)
print(standards_deviations_usgan_obs3_classification_subgroup)


[np.float64(0.0), np.float64(1.2828608834630135), np.float64(0.39544797667452436), np.float64(1.11637618750717), np.float64(0.5346626007661078)]
[np.float64(0.0), np.float64(0.9789148870482938), np.float64(0.11775231631875593), np.float64(0.7913284234733337), np.float64(0.15585034700017306)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_obs3_classification_subgroup, means_bootstraps_usgan_obs3_classification_subgroup, standards_deviations_usgan_obs3_classification_subgroup)

print(lower_bounds_usgan_obs3_classification_subgroup)
print(upper_bounds_usgan_obs3_classification_subgroup)

[np.float64(0.0), np.float64(-0.6357770390817372), np.float64(0.16465767759359465), np.float64(-0.4345990224434242), np.float64(0.22920153366769858)]
[np.float64(0.0), np.float64(3.2014988060077645), np.float64(0.6262382757554541), np.float64(2.6673513974577645), np.float64(0.8401236678645171)]


In [ ]:
lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_obs3_classification_subgroup)

print(lower_bounds_usgan_obs3_classification_subgroup)
print(upper_bounds_usgan_obs3_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.19854735090997908), np.float64(0.21440699541374367), np.float64(0.215067158386254)]
[np.float64(0.0), np.float64(2.029843170037303), np.float64(0.6557815274747014), np.float64(2.924908639490508), np.float64(0.8331282391505109)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup)

print(mean_values_ci_usgan_obs3_classification_subgroup)

[np.float64(0.0), np.float64(1.0149215850186515), np.float64(0.42716443919234026), np.float64(1.5696578174521258), np.float64(0.5240976987683825)]


## GPVAE

### Calculate MAE and AE

In [ ]:
testing_mae_gpvae_variables_standard, testing_ae_gpvae_variables_standard = toolkits.calculate_mae(gpvae_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for GPVAE model

#### Difference between gender group

In [ ]:
diff_gpvae_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 1, 2, variables)
diff_gpvae_gender_subgroup

[('Cholesterol', 0.7169863050614339),
 ('SaO2', 0.20890310598463563),
 ('AST', 0.20239902682582445),
 ('Creatinine', 0.14547286040991392),
 ('Lactate', 0.10004376501079093)]

#### Difference between age group

In [ ]:
diff_gpvae_age_subgroup = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 4, 5, variables)
diff_gpvae_age_subgroup

[('Cholesterol', 0.49957543635081736),
 ('ALP', 0.44452750749784475),
 ('TroponinI', 0.2618898374982579),
 ('SaO2', 0.21620679352892658),
 ('Albumin', 0.1910974661437378)]

#### Difference between ICUType group

In [ ]:
diff_gpvae_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 6, 7, variables)
diff_gpvae_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 6, 8, variables)
diff_gpvae_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 6, 9, variables)
diff_gpvae_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 7, 8, variables)
diff_gpvae_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 7, 9, variables)
diff_gpvae_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 8, 9, variables)

In [ ]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_gpvae_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_gpvae_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_gpvae_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_gpvae_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_gpvae_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_gpvae_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 0.9502364805777341), ('TroponinI', 0.8918236664146777), ('TroponinT', 0.3114883755971654), ('HCO3', 0.23628939822722694), ('SaO2', 0.23145972909185003)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('SaO2', 0.9937353873628099), ('ALP', 0.8302413758449083), ('TroponinI', 0.7412819088709723), ('Cholesterol', 0.406672113721821), ('TroponinT', 0.37481609205650046)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinT', 0.4347207972367254), ('TroponinI', 0.4317269923877344), ('Cholesterol', 0.31119956124464043), ('ALP', 0.24020355084081502), ('Bilirubin', 0.2227724629487818)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.22519511645466), ('ALP', 0.7677499234797236), ('Cholesterol', 0.5435643668559131), ('PaCO2', 0.40728844810871434), ('HCO3', 0.3961385611965824)]
----------------------------
Diferença(MAE

In [ ]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[31])
print(variables[0])
print(variables[6])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
SaO2
TroponinI
ALP
Cholesterol
TroponinT


#### Difference between BMI group

In [ ]:
diff_gpvae_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 11, variables)
diff_gpvae_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 12, variables)
diff_gpvae_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 13, variables)
diff_gpvae_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 14, variables)
diff_gpvae_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 15, variables)
diff_gpvae_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 16, variables)

diff_gpvae_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 12, variables)
diff_gpvae_classification_low_over = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 13, variables)
diff_gpvae_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 14, variables)
diff_gpvae_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 15, variables)
diff_gpvae_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 16, variables)

diff_gpvae_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 13, variables)
diff_gpvae_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 14, variables)
diff_gpvae_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 15, variables)
diff_gpvae_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 16, variables)

diff_gpvae_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 13, 14, variables)
diff_gpvae_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 13, 15, variables)
diff_gpvae_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 13, 16, variables)


diff_gpvae_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 14, 15, variables)
diff_gpvae_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 14, 16, variables)

diff_gpvae_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 15, 16, variables)

In [ ]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_gpvae_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_gpvae_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_gpvae_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_gpvae_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_gpvae_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_gpvae_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.4984287714039075), ('Bilirubin', 0.9811191215037678), ('TroponinI', 0.8126922038658223), ('Cholesterol', 0.5201117490423033), ('Lactate', 0.4554101566283363)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('PaCO2', 0.26413489264298506), ('SaO2', 0.2556269283251386), ('ALT', 0.20532099691202862), ('Platelets', 0.1705636160025804), ('HCO3', 0.1633457333551286)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.8097408650488799), ('TroponinI', 0.3305148845346736), ('TroponinT', 0.2897346238466169), ('PaCO2', 0.2579367576470314), ('SaO2', 0.24006883512626825)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8803621039317031), ('SaO2', 0.2591527877822734), ('PaCO2', 0.2568282809114367), ('Cholesterol', 0.23640482047021322), ('H

In [ ]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_gpvae_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_gpvae_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_gpvae_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_gpvae_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_gpvae_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 1.403021537470746), ('Bilirubin', 0.9461369295522424), ('TroponinI', 0.6835013443205654), ('Cholesterol', 0.5801292127536045), ('HCO3', 0.5142557195063615)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 1.5835896949503965), ('Cholesterol', 1.329852614091183), ('TroponinI', 1.143207088400496), ('Bilirubin', 1.128743649214257), ('HCO3', 0.5693350874264985)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.6272017145736122), ('Bilirubin', 1.0852515823152353), ('Ph', 0.8728858021745433), ('TroponinI', 0.5900021366441112), ('HCO3', 0.581347050390737)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 1.6404632350073978), ('Bilirubin', 1.0958944421751695), ('TroponinT', 0.7259970588377027), ('HCO3', 0.5949372708589092), ('ALT', 0.49226456329646173)]
-----

In [ ]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_gpvae_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_gpvae_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_gpvae_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_gpvae_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('Cholesterol', 0.7497234013375786), ('TroponinI', 0.4597057440799305), ('TroponinT', 0.35357540568797513), ('ALT', 0.32170038759579944), ('Bilirubin', 0.18260671966201464)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8880086026672669), ('Cholesterol', 0.29642228418151445), ('ALP', 0.22418017710286614), ('Mg', 0.17276537650943447), ('Creatinine', 0.16845587072682183)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 1.01714784202371), ('TroponinI', 0.6835013443205654), ('ALT', 0.539823955147861), ('Cholesterol', 0.33531144860598827), ('BUN', 0.2971806787390593)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('ALT', 1.435649509729886), ('TroponinT', 0.8074637349193878), ('Cholesterol', 0.6091621498209887), ('TroponinI', 0.41584146021681445), ('W

In [ ]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_gpvae_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_gpvae_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_gpvae_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Cholesterol', 1.0461456855190931), ('Ph', 0.8934708178133868), ('TroponinI', 0.5532049517563847), ('TroponinT', 0.3830546982731618), ('ALT', 0.2675466002043284)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('TroponinI', 1.143207088400496), ('Cholesterol', 1.085034849943567), ('TroponinT', 0.6635724363357348), ('BUN', 0.3170724188083759), ('AST', 0.21918031375498315)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.3588855511585674), ('ALT', 1.1139491221340865), ('TroponinI', 0.8755472042967449), ('TroponinT', 0.45388832923141276), ('Weight', 0.3685338792048445)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_gpvae_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_gpvae_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.0466271346088967), ('Ph', 0.8973569138255766), ('TroponinI', 0.5900021366441112), ('ALT', 0.48567016775638994), ('BUN', 0.25576191544464466)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('ALT', 1.381495722338415), ('Ph', 0.8941617584582486), ('TroponinT', 0.8369430275045746), ('Glucose', 0.3283979292549837), ('TroponinI', 0.3223422525403602)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_gpvae_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('ALT', 0.895825554582025), ('HCO3', 0.3183493427852112), ('Cholesterol', 0.27385070121500044), ('TroponinI', 0.26765988410375097), ('Weight', 0.26357190666548425)]
----------------------------


In [ ]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[1])
print(variables[31])
print(variables[6])
print(variables[0])
print(variables[32])

Top 5 variáveis BMI
-----------------------
ALT
TroponinI
Cholesterol
ALP
TroponinT


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_gpvae_female = []

for i in indices:
       results_bootstrap_gpvae_female.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_female,standards_deviations_gpvae_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_female)

print(means_bootstraps_gpvae_female)
print(standards_deviations_gpvae_female)


[np.float64(1.4848157361733463), np.float64(0.6668111886463424), np.float64(0.47737647996823174), np.float64(0.39908215120617485), np.float64(0.5039351671552207)]
[np.float64(0.27335769580200403), np.float64(0.1295367515183807), np.float64(0.09540930699622902), np.float64(0.026171397168805406), np.float64(0.03924940928581174)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_female, upper_bounds_gpvae_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_female, means_bootstraps_gpvae_female,standards_deviations_gpvae_female)

print(lower_bounds_gpvae_female)
print(upper_bounds_gpvae_female)

[np.float64(0.9490444975045624), np.float64(0.412923820996002), np.float64(0.29037767446569746), np.float64(0.3477871553302227), np.float64(0.42700773854055774)]
[np.float64(2.0205869748421303), np.float64(0.9206985562966827), np.float64(0.6643752854707661), np.float64(0.450377147082127), np.float64(0.5808625957698836)]


In [ ]:
lower_bounds_gpvae_female, upper_bounds_gpvae_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_female)

print(lower_bounds_gpvae_female)
print(upper_bounds_gpvae_female)

[np.float64(0.977136874860249), np.float64(0.4457911963940036), np.float64(0.31745099235018787), np.float64(0.35123165453553656), np.float64(0.4321853473015345)]
[np.float64(2.0025978006374734), np.float64(0.9502862174067457), np.float64(0.6911125547861318), np.float64(0.45343191690719126), np.float64(0.5861694706950245)]


#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_gpvae_female = toolkits.calc_mean_values_ci(lower_bounds_gpvae_female, upper_bounds_gpvae_female)

print(mean_values_ci_gpvae_female)

[np.float64(1.4898673377488612), np.float64(0.6980387069003746), np.float64(0.5042817735681598), np.float64(0.4023317857213639), np.float64(0.5091774089982795)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_gpvae_male = []

for i in indices:
       results_bootstrap_gpvae_male.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_male,standards_deviations_gpvae_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_male)

print(means_bootstraps_gpvae_male)
print(standards_deviations_gpvae_male)


[np.float64(0.7726863537225884), np.float64(0.4576597577157542), np.float64(0.27644328569292875), np.float64(0.5442021437135832), np.float64(0.6044791706835057)]
[np.float64(0.18372723205507108), np.float64(0.032052213837155986), np.float64(0.030446373086200986), np.float64(0.038998667368798756), np.float64(0.03858456091769444)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_male, upper_bounds_gpvae_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_male, means_bootstraps_gpvae_male,standards_deviations_gpvae_male)

print(lower_bounds_gpvae_male)
print(upper_bounds_gpvae_male)

[np.float64(0.41258759591541616), np.float64(0.39483857297015207), np.float64(0.2167694909841052), np.float64(0.46776616022568024), np.float64(0.5288548209255328)]
[np.float64(1.1327851115297607), np.float64(0.5204809424613562), np.float64(0.3361170804017523), np.float64(0.6206381272014863), np.float64(0.6801035204414785)]


In [ ]:
lower_bounds_gpvae_male, upper_bounds_gpvae_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_male)

print(lower_bounds_gpvae_male)
print(upper_bounds_gpvae_male)

[np.float64(0.4368280153967304), np.float64(0.3991497027910923), np.float64(0.21978021697753053), np.float64(0.4714374331908757), np.float64(0.5342284159084616)]
[np.float64(1.1471987936431198), np.float64(0.5248881927153958), np.float64(0.33803037133170055), np.float64(0.6234666347580539), np.float64(0.6837320236882118)]


#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_gpvae_male = toolkits.calc_mean_values_ci(lower_bounds_gpvae_male, upper_bounds_gpvae_male)

print(mean_values_ci_gpvae_male)

[np.float64(0.7920134045199251), np.float64(0.46201894775324404), np.float64(0.2789052941546155), np.float64(0.5474520339744648), np.float64(0.6089802197983367)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_more_than_or_equal_to_65_subgroup,standards_deviations_gpvae_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_gpvae_more_than_or_equal_to_65_subgroup)
print(standards_deviations_gpvae_more_than_or_equal_to_65_subgroup)

[np.float64(0.608887203121293), np.float64(0.700345984143619), np.float64(1.0339801574836969), np.float64(0.46714621747664187), np.float64(0.560984112179847)]
[np.float64(0.11083633995553831), np.float64(0.13617894863804594), np.float64(0.2883519622141744), np.float64(0.034427863228013514), np.float64(0.06276285553271992)]


#### Calculating lower bound and upper bound


In [ ]:
lower_bounds_gpvae_more_than_or_equal_to_65_subgroup, upper_bounds_gpvae_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup, means_bootstraps_gpvae_more_than_or_equal_to_65_subgroup,standards_deviations_gpvae_more_than_or_equal_to_65_subgroup)

print(lower_bounds_gpvae_more_than_or_equal_to_65_subgroup)
print(upper_bounds_gpvae_more_than_or_equal_to_65_subgroup)


[np.float64(0.39165196863020013), np.float64(0.43344014936051917), np.float64(0.4688206966724605), np.float64(0.3996688454850645), np.float64(0.4379711757688255)]
[np.float64(0.8261224376123858), np.float64(0.967251818926719), np.float64(1.5991396182949331), np.float64(0.5346235894682192), np.float64(0.6839970485908685)]


In [ ]:
lower_bounds_gpvae_more_than_or_equal_to_65_subgroup, upper_bounds_gpvae_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup)

print(lower_bounds_gpvae_more_than_or_equal_to_65_subgroup)
print(upper_bounds_gpvae_more_than_or_equal_to_65_subgroup)

[np.float64(0.39190948611658105), np.float64(0.46229934685756796), np.float64(0.5361267792431346), np.float64(0.40551301975577503), np.float64(0.4450653922645599)]
[np.float64(0.8314725501054785), np.float64(0.9942401223668195), np.float64(1.6623009712332595), np.float64(0.5384898473170325), np.float64(0.6904921282321769)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_more_than_or_equal_to_65_subgroup , upper_bounds_gpvae_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_gpvae_more_than_or_equal_to_65_subgroup)

[np.float64(0.6116910181110298), np.float64(0.7282697346121937), np.float64(1.099213875238197), np.float64(0.4720014335364038), np.float64(0.5677787602483684)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_gpvae_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_gpvae_less_than_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_less_than_65_subgroup,standards_deviations_gpvae_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_less_than_65_subgroup)

print(means_bootstraps_gpvae_less_than_65_subgroup)
print(standards_deviations_gpvae_less_than_65_subgroup)


[np.float64(0.9560096907679728), np.float64(1.140588189156281), np.float64(0.7693323029892964), np.float64(0.6835082176719636), np.float64(0.7534627873753272)]
[np.float64(0.7330739544337671), np.float64(0.4304713019273374), np.float64(0.08096032638510635), np.float64(0.12528515126794312), np.float64(0.06116884655781598)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_less_than_65_subgroup, upper_bounds_gpvae_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_less_than_65_subgroup, means_bootstraps_gpvae_less_than_65_subgroup, standards_deviations_gpvae_less_than_65_subgroup)

print(lower_bounds_gpvae_less_than_65_subgroup)
print(upper_bounds_gpvae_less_than_65_subgroup)

[np.float64(-0.48078885792656745), np.float64(0.2968799410006322), np.float64(0.6106529790978801), np.float64(0.4379538333891424), np.float64(0.6335740511461511)]
[np.float64(2.392808239462513), np.float64(1.9842964373119298), np.float64(0.9280116268807128), np.float64(0.9290626019547847), np.float64(0.8733515236045034)]


In [ ]:
lower_bounds_gpvae_less_than_65_subgroup, upper_bounds_gpvae_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_less_than_65_subgroup)

print(lower_bounds_gpvae_less_than_65_subgroup)
print(upper_bounds_gpvae_less_than_65_subgroup)

[np.float64(0.0), np.float64(0.486211900385218), np.float64(0.6161024734790559), np.float64(0.466207334666563), np.float64(0.6358936811064712)]
[np.float64(1.9961181933033252), np.float64(2.119871001798628), np.float64(0.9309326295462613), np.float64(0.9566584622339344), np.float64(0.8754958140715912)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_less_than_65_subgroup, upper_bounds_gpvae_less_than_65_subgroup)

print(mean_values_ci_gpvae_less_than_65_subgroup)

[np.float64(0.9980590966516626), np.float64(1.303041451091923), np.float64(0.7735175515126587), np.float64(0.7114328984502487), np.float64(0.7556947475890312)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_gpvae_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_1_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_ICUType_1_subgroup, standards_deviations_gpvae_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_1_subgroup)

print(means_bootstraps_gpvae_ICUType_1_subgroup)
print(standards_deviations_gpvae_ICUType_1_subgroup)


[np.float64(0.6483091463618806), np.float64(1.3005919320977053), np.float64(0.33621544291809946), np.float64(0.9516869230809619), np.float64(0.7011323074928193)]
[np.float64(0.1757378683731772), np.float64(0.394817511091146), np.float64(0.03659267024585622), np.float64(0.2564354771316096), np.float64(0.12503221922537475)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_ICUType_1_subgroup, upper_bounds_gpvae_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_1_subgroup, means_bootstraps_gpvae_ICUType_1_subgroup, standards_deviations_gpvae_ICUType_1_subgroup)

print(lower_bounds_gpvae_ICUType_1_subgroup)
print(upper_bounds_gpvae_ICUType_1_subgroup)

[np.float64(0.3038692536306127), np.float64(0.5267638298933158), np.float64(0.26449512713807083), np.float64(0.44908262354466255), np.float64(0.4560736609039683)]
[np.float64(0.9927490390931486), np.float64(2.074420034302095), np.float64(0.4079357586981281), np.float64(1.4542912226172613), np.float64(0.9461909540816703)]


In [ ]:
lower_bounds_gpvae_ICUType_1_subgroup, upper_bounds_gpvae_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_1_subgroup)

print(lower_bounds_gpvae_ICUType_1_subgroup)
print(upper_bounds_gpvae_ICUType_1_subgroup)

[np.float64(0.40125803176394215), np.float64(0.5936642239596918), np.float64(0.2683022144430938), np.float64(0.5033986852974973), np.float64(0.47548502830101885)]
[np.float64(1.063807121909927), np.float64(2.131667060036898), np.float64(0.41176634647184374), np.float64(1.4944066541525494), np.float64(0.9650383139521109)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_1_subgroup, upper_bounds_gpvae_ICUType_1_subgroup)

print(mean_values_ci_gpvae_ICUType_1_subgroup)

[np.float64(0.7325325768369346), np.float64(1.3626656419982948), np.float64(0.34003428045746875), np.float64(0.9989026697250234), np.float64(0.7202616711265649)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_gpvae_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_2_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_ICUType_2_subgroup, standards_deviations_gpvae_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_2_subgroup)

print(means_bootstraps_gpvae_ICUType_2_subgroup)
print(standards_deviations_gpvae_ICUType_2_subgroup) 


[np.float64(0.41865492248808783), np.float64(0.41329206020830106), np.float64(0.3986298803532976), np.float64(0.0), np.float64(0.3895726678951274)]
[np.float64(0.02584064352594675), np.float64(0.1273660522678165), np.float64(0.031072396700507557), np.float64(0.0), np.float64(0.04981840308787588)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_ICUType_2_subgroup, upper_bounds_gpvae_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_2_subgroup, means_bootstraps_gpvae_ICUType_2_subgroup, standards_deviations_gpvae_ICUType_2_subgroup)

print(lower_bounds_gpvae_ICUType_2_subgroup)
print(upper_bounds_gpvae_ICUType_2_subgroup)

[np.float64(0.3680081918398941), np.float64(0.16365918491033465), np.float64(0.3377291019069616), np.float64(0.0), np.float64(0.2919303920755917)]
[np.float64(0.4693016531362816), np.float64(0.6629249355062674), np.float64(0.4595306587996336), np.float64(0.0), np.float64(0.4872149437146631)]


In [ ]:
lower_bounds_gpvae_ICUType_2_subgroup, upper_bounds_gpvae_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_2_subgroup)

print(lower_bounds_gpvae_ICUType_2_subgroup)
print(upper_bounds_gpvae_ICUType_2_subgroup)

[np.float64(0.37003655737537194), np.float64(0.1282699726592907), np.float64(0.33761345993386227), np.float64(0.0), np.float64(0.2977723861432638)]
[np.float64(0.46990720163528205), np.float64(0.6636382998956522), np.float64(0.45932989831734555), np.float64(0.0), np.float64(0.493490121632228)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_2_subgroup, upper_bounds_gpvae_ICUType_2_subgroup)

print(mean_values_ci_gpvae_ICUType_2_subgroup)

[np.float64(0.419971879505327), np.float64(0.39595413627747145), np.float64(0.3984716791256039), np.float64(0.0), np.float64(0.3956312538877459)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_gpvae_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_3_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_ICUType_3_subgroup, standards_deviations_gpvae_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_3_subgroup)

print(means_bootstraps_gpvae_ICUType_3_subgroup)
print(standards_deviations_gpvae_ICUType_3_subgroup)


[np.float64(1.6419263666653052), np.float64(0.5686112856905978), np.float64(1.1680702008354567), np.float64(0.5330030471491659), np.float64(0.32651285135754826)]
[np.float64(0.4450416566208062), np.float64(0.08636182609599395), np.float64(0.44671984500461664), np.float64(0.09361158468271201), np.float64(0.03173084808226046)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_ICUType_3_subgroup, upper_bounds_gpvae_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_3_subgroup, means_bootstraps_gpvae_ICUType_3_subgroup, standards_deviations_gpvae_ICUType_3_subgroup)

print(lower_bounds_gpvae_ICUType_3_subgroup)
print(upper_bounds_gpvae_ICUType_3_subgroup)

[np.float64(0.7696607480684834), np.float64(0.3993452169033383), np.float64(0.2925153934470929), np.float64(0.349527712635329), np.float64(0.2643215319174059)]
[np.float64(2.5141919852621273), np.float64(0.7378773544778574), np.float64(2.0436250082238203), np.float64(0.7164783816630028), np.float64(0.3887041707976906)]


In [ ]:
lower_bounds_gpvae_ICUType_3_subgroup, upper_bounds_gpvae_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_3_subgroup)

print(lower_bounds_gpvae_ICUType_3_subgroup)
print(upper_bounds_gpvae_ICUType_3_subgroup)

[np.float64(0.8511417857281199), np.float64(0.3889523105084295), np.float64(0.4789041637559711), np.float64(0.4163198819867195), np.float64(0.27069763199889035)]
[np.float64(2.5823049879653097), np.float64(0.7286224728746608), np.float64(2.1860913430001165), np.float64(0.6696992193609029), np.float64(0.395096127094224)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_3_subgroup, upper_bounds_gpvae_ICUType_3_subgroup)

print(mean_values_ci_gpvae_ICUType_3_subgroup)

[np.float64(1.7167233868467149), np.float64(0.5587873916915451), np.float64(1.3324977533780438), np.float64(0.5430095506738112), np.float64(0.33289687954655717)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_gpvae_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_4_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_ICUType_4_subgroup, standards_deviations_gpvae_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_4_subgroup)

print(means_bootstraps_gpvae_ICUType_4_subgroup)
print(standards_deviations_gpvae_ICUType_4_subgroup)


[np.float64(0.5397782999782886), np.float64(0.859346936761451), np.float64(0.576710689772995), np.float64(0.6403309632458108), np.float64(0.2667977074457082)]
[np.float64(0.082608109086022), np.float64(0.208284459165866), np.float64(0.07224826311948214), np.float64(0.22693806359571855), np.float64(0.04697185841900373)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_ICUType_4_subgroup, upper_bounds_gpvae_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_4_subgroup, means_bootstraps_gpvae_ICUType_4_subgroup, standards_deviations_gpvae_ICUType_4_subgroup)

print(lower_bounds_gpvae_ICUType_4_subgroup)
print(upper_bounds_gpvae_ICUType_4_subgroup)

[np.float64(0.3778693813387295), np.float64(0.4511168982569501), np.float64(0.43510669611323655), np.float64(0.19554053187694215), np.float64(0.17473455665754634)]
[np.float64(0.7016872186178478), np.float64(1.267576975265952), np.float64(0.7183146834327534), np.float64(1.0851213946146796), np.float64(0.35886085823387004)]


In [ ]:
lower_bounds_gpvae_ICUType_4_subgroup, upper_bounds_gpvae_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_4_subgroup)

print(lower_bounds_gpvae_ICUType_4_subgroup)
print(upper_bounds_gpvae_ICUType_4_subgroup)

[np.float64(0.3962725121716412), np.float64(0.6102348466879253), np.float64(0.44557232590491413), np.float64(0.1897127655413451), np.float64(0.1860317343859407)]
[np.float64(0.7182152499516535), np.float64(1.3679689773409256), np.float64(0.7279097083542633), np.float64(1.106896006145555), np.float64(0.36649335311257814)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_4_subgroup, upper_bounds_gpvae_ICUType_4_subgroup)

print(mean_values_ci_gpvae_ICUType_4_subgroup)

[np.float64(0.5572438810616474), np.float64(0.9891019120144254), np.float64(0.5867410171295887), np.float64(0.6483043858434501), np.float64(0.2762625437492594)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_undefined_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_undefined_classification_subgroup, standards_deviations_gpvae_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_undefined_classification_subgroup)

print(means_bootstraps_gpvae_undefined_classification_subgroup)
print(standards_deviations_gpvae_undefined_classification_subgroup)


[np.float64(0.4380686480106528), np.float64(0.810384525274796), np.float64(0.5494012743288357), np.float64(0.48685569097517045), np.float64(0.3422795965485698)]
[np.float64(0.10669361031921458), np.float64(0.12159702527267358), np.float64(0.1923318409753984), np.float64(0.04982752076838099), np.float64(0.04088206729160392)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_undefined_classification_subgroup, upper_bounds_gpvae_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_undefined_classification_subgroup, means_bootstraps_gpvae_undefined_classification_subgroup, standards_deviations_gpvae_undefined_classification_subgroup)

print(lower_bounds_gpvae_undefined_classification_subgroup)
print(upper_bounds_gpvae_undefined_classification_subgroup)

[np.float64(0.22895301440444119), np.float64(0.572058735113149), np.float64(0.1724377929367698), np.float64(0.3891955448302221), np.float64(0.2621522170434832)]
[np.float64(0.6471842816168645), np.float64(1.048710315436443), np.float64(0.9263647557209016), np.float64(0.5845158371201188), np.float64(0.42240697605365646)]


In [ ]:
lower_bounds_gpvae_undefined_classification_subgroup, upper_bounds_gpvae_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_undefined_classification_subgroup)

print(lower_bounds_gpvae_undefined_classification_subgroup)
print(upper_bounds_gpvae_undefined_classification_subgroup)

[np.float64(0.27280385741275304), np.float64(0.6044411251679942), np.float64(0.17908935536195858), np.float64(0.39633931600605954), np.float64(0.2733956086551003)]
[np.float64(0.6833212230906859), np.float64(1.048822347072997), np.float64(0.9421401025105941), np.float64(0.5895113413591085), np.float64(0.43122712559066695)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_undefined_classification_subgroup, upper_bounds_gpvae_undefined_classification_subgroup)

print(mean_values_ci_gpvae_undefined_classification_subgroup)

[np.float64(0.47806254025171946), np.float64(0.8266317361204956), np.float64(0.5606147289362764), np.float64(0.49292532868258404), np.float64(0.35231136712288363)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_low_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_low_weight_classification_subgroup, standards_deviations_gpvae_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_low_weight_classification_subgroup)

print(means_bootstraps_gpvae_low_weight_classification_subgroup)
print(standards_deviations_gpvae_low_weight_classification_subgroup)


[np.float64(0.27962627886939556), np.float64(0.0), np.float64(0.018087519792984636), np.float64(1.7335622483529622), np.float64(0.35387523253294884)]
[np.float64(0.1314506383399375), np.float64(0.0), np.float64(0.014070374962782703), np.float64(1.338810760091147), np.float64(0.2759980988560088)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_low_weight_classification_subgroup, upper_bounds_gpvae_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_low_weight_classification_subgroup, means_bootstraps_gpvae_low_weight_classification_subgroup, standards_deviations_gpvae_low_weight_classification_subgroup)

print(lower_bounds_gpvae_low_weight_classification_subgroup)
print(upper_bounds_gpvae_low_weight_classification_subgroup)

[np.float64(0.021987761978318032), np.float64(0.0), np.float64(-0.009489908383043564), np.float64(-0.8904586235403804), np.float64(-0.18707110102635394)]
[np.float64(0.537264795760473), np.float64(0.0), np.float64(0.04566494796901284), np.float64(4.357583120246305), np.float64(0.8948215660922516)]


In [ ]:
lower_bounds_gpvae_low_weight_classification_subgroup, upper_bounds_gpvae_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_low_weight_classification_subgroup)

print(lower_bounds_gpvae_low_weight_classification_subgroup)
print(upper_bounds_gpvae_low_weight_classification_subgroup)

[np.float64(0.11261517001941171), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
[np.float64(0.5763994590970901), np.float64(0.0), np.float64(0.02903293706740358), np.float64(3.61181531810428), np.float64(0.5691345769831901)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_low_weight_classification_subgroup, upper_bounds_gpvae_low_weight_classification_subgroup)

print(mean_values_ci_gpvae_low_weight_classification_subgroup)

[np.float64(0.3445073145582509), np.float64(0.0), np.float64(0.01451646853370179), np.float64(1.80590765905214), np.float64(0.28456728849159507)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_normal_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_normal_weight_classification_subgroup, standards_deviations_gpvae_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_normal_weight_classification_subgroup)

print(means_bootstraps_gpvae_normal_weight_classification_subgroup)
print(standards_deviations_gpvae_normal_weight_classification_subgroup)


[np.float64(0.2346916968942821), np.float64(0.6829620586258408), np.float64(0.5302113222344054), np.float64(0.5808258829284493), np.float64(0.278969198148233)]
[np.float64(0.05009073639504404), np.float64(0.15515558413701028), np.float64(0.286628147720594), np.float64(0.10803654085291374), np.float64(0.057731073754505514)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_normal_weight_classification_subgroup, upper_bounds_gpvae_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_normal_weight_classification_subgroup, means_bootstraps_gpvae_normal_weight_classification_subgroup, standards_deviations_gpvae_normal_weight_classification_subgroup)

print(lower_bounds_gpvae_normal_weight_classification_subgroup)
print(upper_bounds_gpvae_normal_weight_classification_subgroup)

[np.float64(0.13651565760090606), np.float64(0.3788627017170265), np.float64(-0.03156952425338522), np.float64(0.36907815384244813), np.float64(0.16581837280057665)]
[np.float64(0.33286773618765814), np.float64(0.9870614155346551), np.float64(1.091992168722196), np.float64(0.7925736120144504), np.float64(0.39212002349588937)]


In [ ]:
lower_bounds_gpvae_normal_weight_classification_subgroup, upper_bounds_gpvae_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_normal_weight_classification_subgroup)

print(lower_bounds_gpvae_normal_weight_classification_subgroup)
print(upper_bounds_gpvae_normal_weight_classification_subgroup)

[np.float64(0.14588778009423162), np.float64(0.3973910819567376), np.float64(0.0), np.float64(0.4018972049104533), np.float64(0.18502148790676368)]
[np.float64(0.34023191188512586), np.float64(1.0041134592895729), np.float64(0.8911077112914882), np.float64(0.8207468070781362), np.float64(0.40722004351699315)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_normal_weight_classification_subgroup, upper_bounds_gpvae_normal_weight_classification_subgroup)

print(mean_values_ci_gpvae_normal_weight_classification_subgroup)

[np.float64(0.24305984598967872), np.float64(0.7007522706231553), np.float64(0.4455538556457441), np.float64(0.6113220059942948), np.float64(0.2961207657118784)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_overweight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_overweight_classification_subgroup, standards_deviations_gpvae_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_overweight_classification_subgroup)

print(means_bootstraps_gpvae_overweight_classification_subgroup)
print(standards_deviations_gpvae_overweight_classification_subgroup)


[np.float64(0.5564143670964794), np.float64(1.1301856012799774), np.float64(1.3561246737109942), np.float64(0.40160781552156305), np.float64(0.6333028409572671)]
[np.float64(0.15270642047354707), np.float64(0.5176214120430332), np.float64(0.30766416105174693), np.float64(0.054961310414240384), np.float64(0.14637975694197797)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_overweight_classification_subgroup, upper_bounds_gpvae_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_overweight_classification_subgroup, means_bootstraps_gpvae_overweight_classification_subgroup, standards_deviations_gpvae_overweight_classification_subgroup)

print(lower_bounds_gpvae_overweight_classification_subgroup)
print(upper_bounds_gpvae_overweight_classification_subgroup)

[np.float64(0.2571152827602972), np.float64(0.1156662760488647), np.float64(0.7531139987158394), np.float64(0.2938856265665257), np.float64(0.3464037892852633)]
[np.float64(0.8557134514326616), np.float64(2.14470492651109), np.float64(1.9591353487061491), np.float64(0.5093300044766004), np.float64(0.9202018926292709)]


In [ ]:
lower_bounds_gpvae_overweight_classification_subgroup, upper_bounds_gpvae_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_overweight_classification_subgroup)

print(lower_bounds_gpvae_overweight_classification_subgroup)
print(upper_bounds_gpvae_overweight_classification_subgroup)

[np.float64(0.2909818311881295), np.float64(0.5432087030758634), np.float64(0.7193394854565662), np.float64(0.30343781283710475), np.float64(0.41735766651888895)]
[np.float64(0.8911536264290908), np.float64(2.3252858079301166), np.float64(1.9182719093142704), np.float64(0.5184826723270521), np.float64(0.9705782544024717)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_overweight_classification_subgroup, upper_bounds_gpvae_overweight_classification_subgroup)

print(mean_values_ci_gpvae_overweight_classification_subgroup)

[np.float64(0.5910677288086101), np.float64(1.43424725550299), np.float64(1.3188056973854183), np.float64(0.4109602425820784), np.float64(0.6939679604606803)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_obs1_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_obs1_classification_subgroup, standards_deviations_gpvae_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_obs1_classification_subgroup)

print(means_bootstraps_gpvae_obs1_classification_subgroup)
print(standards_deviations_gpvae_obs1_classification_subgroup)


[np.float64(0.2892920377587374), np.float64(0.51097067128048), np.float64(0.2945826308705735), np.float64(0.35869806508856655), np.float64(0.2497083420368694)]
[np.float64(0.04477024141132264), np.float64(0.21764882143250286), np.float64(0.2087658921356739), np.float64(0.04158070331211801), np.float64(0.07261475573256627)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_obs1_classification_subgroup, upper_bounds_gpvae_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_obs1_classification_subgroup, means_bootstraps_gpvae_obs1_classification_subgroup, standards_deviations_gpvae_obs1_classification_subgroup)

print(lower_bounds_gpvae_obs1_classification_subgroup)
print(upper_bounds_gpvae_obs1_classification_subgroup)

[np.float64(0.20154397701338136), np.float64(0.08438681999518499), np.float64(-0.11459099891572105), np.float64(0.2772013841449699), np.float64(0.10738603605486607)]
[np.float64(0.3770400985040935), np.float64(0.9375545225657751), np.float64(0.7037562606568681), np.float64(0.4401947460321632), np.float64(0.3920306480188727)]


In [ ]:
lower_bounds_gpvae_obs1_classification_subgroup, upper_bounds_gpvae_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_obs1_classification_subgroup)

print(lower_bounds_gpvae_obs1_classification_subgroup)
print(upper_bounds_gpvae_obs1_classification_subgroup)

[np.float64(0.20594410232097654), np.float64(0.0), np.float64(0.0), np.float64(0.27792979388004074), np.float64(0.12724030520186394)]
[np.float64(0.38171536826509955), np.float64(0.7107226819397775), np.float64(0.7401267563256191), np.float64(0.4403301387718181), np.float64(0.4075854940220041)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_obs1_classification_subgroup, upper_bounds_gpvae_obs1_classification_subgroup)

print(mean_values_ci_gpvae_obs1_classification_subgroup)

[np.float64(0.293829735293038), np.float64(0.35536134096988875), np.float64(0.37006337816280954), np.float64(0.3591299663259294), np.float64(0.26741289961193404)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_obs2_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_obs2_classification_subgroup, standards_deviations_gpvae_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_obs2_classification_subgroup)

print(means_bootstraps_gpvae_obs2_classification_subgroup)
print(standards_deviations_gpvae_obs2_classification_subgroup)


[np.float64(0.7689178822445539), np.float64(0.0), np.float64(0.17283022032239628), np.float64(0.3444692144370151), np.float64(1.2349570331697446)]
[np.float64(0.3052430913477002), np.float64(0.0), np.float64(0.13213399248397165), np.float64(0.04300155060509514), np.float64(0.9093276033877988)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_obs2_classification_subgroup, upper_bounds_gpvae_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_obs2_classification_subgroup, means_bootstraps_gpvae_obs2_classification_subgroup, standards_deviations_gpvae_obs2_classification_subgroup)

print(lower_bounds_gpvae_obs2_classification_subgroup)
print(upper_bounds_gpvae_obs2_classification_subgroup)

[np.float64(0.1706524166733917), np.float64(0.0), np.float64(-0.08614764607967432), np.float64(0.2601877239716521), np.float64(-0.5472923196184636)]
[np.float64(1.367183347815716), np.float64(0.0), np.float64(0.4318080867244669), np.float64(0.42875070490237815), np.float64(3.0172063859579525)]


In [ ]:
lower_bounds_gpvae_obs2_classification_subgroup, upper_bounds_gpvae_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_obs2_classification_subgroup)

print(lower_bounds_gpvae_obs2_classification_subgroup)
print(upper_bounds_gpvae_obs2_classification_subgroup)

[np.float64(0.2612911869796691), np.float64(0.0), np.float64(0.0), np.float64(0.26645729138560365), np.float64(0.0)]
[np.float64(1.4180964126533249), np.float64(0.0), np.float64(0.273850701215183), np.float64(0.43659344450448784), np.float64(3.1824386190830314)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_obs2_classification_subgroup, upper_bounds_gpvae_obs2_classification_subgroup)

print(mean_values_ci_gpvae_obs2_classification_subgroup)

[np.float64(0.8396937998164969), np.float64(0.0), np.float64(0.1369253506075915), np.float64(0.35152536794504574), np.float64(1.5912193095415157)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]


results_bootstrap_gpvae_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_obs3_classification_subgroup, standards_deviations_gpvae_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_obs3_classification_subgroup)

print(means_bootstraps_gpvae_obs3_classification_subgroup)
print(standards_deviations_gpvae_obs3_classification_subgroup)


[np.float64(1.608856100366626), np.float64(0.16939896665059076), np.float64(0.0), np.float64(0.2552729861572449), np.float64(1.069378365532719)]
[np.float64(1.1100059636644055), np.float64(0.12901665736917378), np.float64(0.0), np.float64(0.04201587180787358), np.float64(0.7479996842553093)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_obs3_classification_subgroup, means_bootstraps_gpvae_obs3_classification_subgroup, standards_deviations_gpvae_obs3_classification_subgroup)

print(lower_bounds_gpvae_obs3_classification_subgroup)
print(upper_bounds_gpvae_obs3_classification_subgroup)

[np.float64(-0.5667156110402849), np.float64(-0.08346903519873403), np.float64(0.0), np.float64(0.1729233906347609), np.float64(-0.39667407605501936)]
[np.float64(3.784427811773537), np.float64(0.4222669684999155), np.float64(0.0), np.float64(0.3376225816797289), np.float64(2.5354308071204574)]


In [ ]:
lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_obs3_classification_subgroup)

print(lower_bounds_gpvae_obs3_classification_subgroup)
print(upper_bounds_gpvae_obs3_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.1746412923954669), np.float64(0.18069261179045693)]
[np.float64(3.9169417676929363), np.float64(0.26765988410392944), np.float64(0.0), np.float64(0.33800666615727104), np.float64(2.7764147172647666)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup)

print(mean_values_ci_gpvae_obs3_classification_subgroup)

[np.float64(1.9584708838464682), np.float64(0.13382994205196472), np.float64(0.0), np.float64(0.25632397927636896), np.float64(1.4785536645276118)]


## MRNN

### Calculate MAE and AE

In [ ]:
testing_mae_mrnn_variables_standard, testing_ae_mrnn_variables_standard = toolkits.calculate_mae(mrnn_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for MRNN model

#### Difference between gender group

In [ ]:
diff_mrnn_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 1, 2, variables)
diff_mrnn_gender_subgroup

[('Cholesterol', 0.46007480562614733),
 ('SaO2', 0.23089925994991106),
 ('Weight', 0.1781846451592244),
 ('AST', 0.1741083024220948),
 ('PaCO2', 0.12962865455656114)]

#### Difference between age group

In [ ]:
diff_mrnn_age_subgroup = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 4, 5, variables)
diff_mrnn_age_subgroup

[('Cholesterol', 0.5264706140850112),
 ('ALP', 0.42118039343106295),
 ('SaO2', 0.21218979884570444),
 ('TroponinI', 0.18821601789959796),
 ('Bilirubin', 0.1405204793192777)]

#### Difference between ICUType group

In [ ]:
diff_mrnn_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 6, 7, variables)
diff_mrnn_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 6, 8, variables)
diff_mrnn_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 6, 9, variables)
diff_mrnn_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 7, 8, variables)
diff_mrnn_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 7, 9, variables)
diff_mrnn_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 8, 9, variables)

In [ ]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_mrnn_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_mrnn_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_mrnn_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_mrnn_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_mrnn_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_mrnn_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 0.9010319943785403), ('TroponinI', 0.8885610723184879), ('TroponinT', 0.2790266043092085), ('AST', 0.2552394455397726), ('HR', 0.24475318001332502)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('SaO2', 1.126906434849816), ('ALP', 0.826833726615776), ('TroponinI', 0.6275078131032561), ('PaCO2', 0.3166885871390994), ('K', 0.24651464330000916)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.4752300632564468), ('TroponinT', 0.28805061664771353), ('Albumin', 0.2061163193636698), ('Glucose', 0.20571705409253582), ('Cholesterol', 0.20496776200009514)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.3273285721076542), ('Cholesterol', 0.7254168014070564), ('ALP', 0.6559594688208507), ('HCO3', 0.444614029946391), ('PaCO2', 0.4402514603625217)]
----------------------------
Diferença(MAE) ICUType 2 e 4


In [ ]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[6])
print(variables[31])
print(variables[0])
print(variables[24])

Top 5 variáveis ICUType
-----------------------
SaO2
Cholesterol
TroponinI
ALP
PaCO2


#### Difference between BMI group

In [ ]:
diff_mrnn_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 11, variables)
diff_mrnn_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 12, variables)
diff_mrnn_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 13, variables)
diff_mrnn_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 14, variables)
diff_mrnn_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 15, variables)
diff_mrnn_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 16, variables)

diff_mrnn_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 12, variables)
diff_mrnn_classification_low_over = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 13, variables)
diff_mrnn_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 14, variables)
diff_mrnn_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 15, variables)
diff_mrnn_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 16, variables)

diff_mrnn_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 13, variables)
diff_mrnn_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 14, variables)
diff_mrnn_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 15, variables)
diff_mrnn_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 16, variables)

diff_mrnn_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 13, 14, variables)
diff_mrnn_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 13, 15, variables)
diff_mrnn_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 13, 16, variables)


diff_mrnn_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 14, 15, variables)
diff_mrnn_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 14, 16, variables)

diff_mrnn_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 15, 16, variables)

In [ ]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_mrnn_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_mrnn_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_mrnn_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_mrnn_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_mrnn_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_mrnn_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.7276133990683362), ('Bilirubin', 0.9914312934423044), ('TroponinI', 0.7605306891067674), ('Weight', 0.5541701194492721), ('Cholesterol', 0.4686585973942653)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('Cholesterol', 0.3793471636845912), ('PaCO2', 0.2607229442068275), ('SaO2', 0.23204288622686337), ('TroponinI', 0.19152385318395748), ('HCO3', 0.18600797369321342)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.7311865583297629), ('TroponinI', 0.4223997471662162), ('Weight', 0.3506808760264265), ('PaCO2', 0.2270916455603793), ('SaO2', 0.2137441109303908)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8858111069773932), ('PaCO2', 0.2961064224948988), ('HCO3', 0.25893221785255016), ('SaO2', 0.2587867211362769), ('Albu

In [ ]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_mrnn_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_mrnn_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_mrnn_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_mrnn_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_mrnn_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 1.616655613152504), ('Bilirubin', 0.9207620318461242), ('Cholesterol', 0.8480057610788565), ('TroponinI', 0.5690068359228099), ('HCO3', 0.5367857762848458)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 1.8004731135079597), ('Cholesterol', 1.1998451557240282), ('TroponinI', 1.1829304362729836), ('Bilirubin', 1.1149758355325403), ('Weight', 0.9048509954756986)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.839342476713079), ('Bilirubin', 1.1198139228432797), ('Ph', 0.8807265511276338), ('Weight', 0.782381102471485), ('TroponinI', 0.6293802804910801)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 1.701601911486742), ('Bilirubin', 1.0552999585115248), ('TroponinT', 0.9759888843088509), ('HCO3', 0.6339041283774007), ('Weight', 0.5288315136741774)

In [ ]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_mrnn_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_mrnn_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_mrnn_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_mrnn_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('TroponinI', 0.6139236003501737), ('Weight', 0.39083485356388614), ('Cholesterol', 0.35183939464517167), ('ALT', 0.25331368516240216), ('Bilirubin', 0.19421380368641616)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8908243780585775), ('Cholesterol', 0.3310031978642243), ('Weight', 0.2683649605596725), ('ALP', 0.22268686356057493), ('Albumin', 0.1992342096852764)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 0.9827645580136316), ('Cholesterol', 0.8717828192132384), ('TroponinI', 0.5690068359228099), ('ALT', 0.2871264264440603), ('Creatinine', 0.2062260662166976)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('ALT', 1.0982371420715205), ('Cholesterol', 0.953380515668881), ('Weight', 0.86452363699577), ('TroponinT', 0.5796770889726964), ('HC

In [ ]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_mrnn_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_mrnn_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_mrnn_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Ph', 0.8888769174492269), ('Cholesterol', 0.6828425925093959), ('TroponinI', 0.5535501557819035), ('ALT', 0.2339750215206805), ('TroponinT', 0.23237983193846368)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('Cholesterol', 1.22362221385841), ('TroponinI', 1.1829304362729836), ('TroponinT', 0.8424714467402539), ('Weight', 0.3760194818015212), ('RespRate', 0.20419277047251283)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.3052199103140527), ('Weight', 1.2553584905596562), ('ALT', 0.8449234569091182), ('TroponinI', 0.8326977045645636), ('TroponinT', 0.4393839776993187)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_mrnn_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_mrnn_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.0748512786787177), ('Ph', 0.8644289951196773), ('TroponinI', 0.6293802804910801), ('Cholesterol', 0.5407796213490141), ('ALT', 0.2677877628023386)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Weight', 1.1328885975554424), ('ALT', 1.0788984784297986), ('Ph', 0.884502858503702), ('TroponinT', 0.6717638096377824), ('Cholesterol', 0.6223773178046568)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_mrnn_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('Weight', 0.8793390087581349), ('ALT', 0.8111107156274601), ('TroponinT', 0.4030874690409352), ('HCO3', 0.40178041918761953), ('TroponinI', 0.35023273170841995)]
----------------------------


In [ ]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[6])
print(variables[0])
print(variables[31])
print(variables[35])
print(variables[32])

Top 5 variáveis BMI
-----------------------
Cholesterol
ALP
TroponinI
Weight
TroponinT


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_mrnn_female = []

for i in indices:
       results_bootstrap_mrnn_female.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_female,standards_deviations_mrnn_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_female)

print(means_bootstraps_mrnn_female)
print(standards_deviations_mrnn_female)


[np.float64(1.2889617478015634), np.float64(0.6793488248636798), np.float64(0.8424471521729673), np.float64(0.5862177738523476), np.float64(0.8398809270249903)]
[np.float64(0.3357795625717358), np.float64(0.14584371229851048), np.float64(0.011319424019964401), np.float64(0.08549284516817783), np.float64(0.03231667834053996)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_female, upper_bounds_mrnn_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_female, means_bootstraps_mrnn_female,standards_deviations_mrnn_female)

print(lower_bounds_mrnn_female)
print(upper_bounds_mrnn_female)

[np.float64(0.6308458984163476), np.float64(0.3935004013869779), np.float64(0.8202614887680995), np.float64(0.41865487638685994), np.float64(0.7765414013775663)]
[np.float64(1.947077597186779), np.float64(0.9651972483403817), np.float64(0.8646328155778351), np.float64(0.7537806713178353), np.float64(0.9032204526724142)]


In [ ]:
lower_bounds_mrnn_female, upper_bounds_mrnn_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_female)

print(lower_bounds_mrnn_female)
print(upper_bounds_mrnn_female)

[np.float64(0.6126520319937722), np.float64(0.4293660610982223), np.float64(0.8206899465336797), np.float64(0.443153351338707), np.float64(0.7769730572247365)]
[np.float64(1.9795521249564374), np.float64(0.998280386835213), np.float64(0.8647490394958172), np.float64(0.7781546422723796), np.float64(0.9034747830686872)]


#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_mrnn_female = toolkits.calc_mean_values_ci(lower_bounds_mrnn_female, upper_bounds_mrnn_female)

print(mean_values_ci_mrnn_female)

[np.float64(1.2961020784751047), np.float64(0.7138232239667177), np.float64(0.8427194930147484), np.float64(0.6106539968055433), np.float64(0.8402239201467119)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_mrnn_male = []

for i in indices:
       results_bootstrap_mrnn_male.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_male,standards_deviations_mrnn_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_male)

print(means_bootstraps_mrnn_male)
print(standards_deviations_mrnn_male)


[np.float64(0.8445685112750926), np.float64(0.44841527819129745), np.float64(0.6642340103663307), np.float64(0.41083254941155223), np.float64(0.709842234225226)]
[np.float64(0.17983375950603753), np.float64(0.0358265633111031), np.float64(0.011191619427093449), np.float64(0.017704199334764567), np.float64(0.025604735411969396)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_male, upper_bounds_mrnn_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_male, means_bootstraps_mrnn_male,standards_deviations_mrnn_male)

print(lower_bounds_mrnn_male)
print(upper_bounds_mrnn_male)

[np.float64(0.49210081943882145), np.float64(0.3781965044116913), np.float64(0.6422988393605488), np.float64(0.3761329563402957), np.float64(0.6596578749840886)]
[np.float64(1.1970362031113637), np.float64(0.5186340519709036), np.float64(0.6861691813721126), np.float64(0.44553214248280876), np.float64(0.7600265934663634)]


In [ ]:
lower_bounds_mrnn_male, upper_bounds_mrnn_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_male)

print(lower_bounds_mrnn_male)
print(upper_bounds_mrnn_male)

[np.float64(0.49609853871696685), np.float64(0.38271541586935226), np.float64(0.6427265756526298), np.float64(0.37913152414672663), np.float64(0.6606005566331093)]
[np.float64(1.2074254101336381), np.float64(0.5212447768606311), np.float64(0.686410179614941), np.float64(0.4475509383118786), np.float64(0.7617919331238603)]


#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_mrnn_male = toolkits.calc_mean_values_ci(lower_bounds_mrnn_male, upper_bounds_mrnn_male)

print(mean_values_ci_mrnn_male)

[np.float64(0.8517619744253024), np.float64(0.4519800963649917), np.float64(0.6645683776337854), np.float64(0.4133412312293026), np.float64(0.7111962448784848)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_more_than_or_equal_to_65_subgroup,standards_deviations_mrnn_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_mrnn_more_than_or_equal_to_65_subgroup)
print(standards_deviations_mrnn_more_than_or_equal_to_65_subgroup)

[np.float64(0.6664471815194499), np.float64(0.7713441918549366), np.float64(0.4419490385477844), np.float64(0.9375133641190462), np.float64(0.48562470703319427)]
[np.float64(0.09940737024271447), np.float64(0.12945819564628502), np.float64(0.035835711363100396), np.float64(0.28200067108830146), np.float64(0.019127093478076996)]


#### Calculating lower bound and upper bound


In [ ]:
lower_bounds_mrnn_more_than_or_equal_to_65_subgroup, upper_bounds_mrnn_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup, means_bootstraps_mrnn_more_than_or_equal_to_65_subgroup,standards_deviations_mrnn_more_than_or_equal_to_65_subgroup)

print(lower_bounds_mrnn_more_than_or_equal_to_65_subgroup)
print(upper_bounds_mrnn_more_than_or_equal_to_65_subgroup)


[np.float64(0.4716123160458909), np.float64(0.517610790884678), np.float64(0.37171233491573485), np.float64(0.38480220516984964), np.float64(0.44813629268723243)]
[np.float64(0.8612820469930089), np.float64(1.0250775928251952), np.float64(0.5121857421798339), np.float64(1.4902245230682427), np.float64(0.5231131213791561)]


In [ ]:
lower_bounds_mrnn_more_than_or_equal_to_65_subgroup, upper_bounds_mrnn_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup)

print(lower_bounds_mrnn_more_than_or_equal_to_65_subgroup)
print(upper_bounds_mrnn_more_than_or_equal_to_65_subgroup)

[np.float64(0.4699119681436676), np.float64(0.5493511194743891), np.float64(0.3765201998918443), np.float64(0.4675343714728311), np.float64(0.4474030178628885)]
[np.float64(0.8617314375412235), np.float64(1.054573547121633), np.float64(0.5166212185748152), np.float64(1.5445279424896994), np.float64(0.522282392048733)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_more_than_or_equal_to_65_subgroup , upper_bounds_mrnn_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_mrnn_more_than_or_equal_to_65_subgroup)

[np.float64(0.6658217028424456), np.float64(0.801962333298011), np.float64(0.44657070923332975), np.float64(1.0060311569812652), np.float64(0.4848427049558107)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_mrnn_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_mrnn_less_than_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_less_than_65_subgroup,standards_deviations_mrnn_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_less_than_65_subgroup)

print(means_bootstraps_mrnn_less_than_65_subgroup)
print(standards_deviations_mrnn_less_than_65_subgroup)


[np.float64(1.029187718285846), np.float64(1.1923604185552845), np.float64(0.654087663038131), np.float64(0.7499906208627262), np.float64(0.6266684481775726)]
[np.float64(0.8328915889503894), np.float64(0.432296645057951), np.float64(0.13999873850370842), np.float64(0.0583718487785053), np.float64(0.0736833630421469)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_less_than_65_subgroup, upper_bounds_mrnn_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_less_than_65_subgroup, means_bootstraps_mrnn_less_than_65_subgroup, standards_deviations_mrnn_less_than_65_subgroup)

print(lower_bounds_mrnn_less_than_65_subgroup)
print(upper_bounds_mrnn_less_than_65_subgroup)

[np.float64(-0.6032497990832562), np.float64(0.3450745636042054), np.float64(0.3796951776898216), np.float64(0.6355838995458374), np.float64(0.482251710355175)]
[np.float64(2.661625235654948), np.float64(2.0396462735063636), np.float64(0.9284801483864404), np.float64(0.864397342179615), np.float64(0.7710851859999702)]


In [ ]:
lower_bounds_mrnn_less_than_65_subgroup, upper_bounds_mrnn_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_less_than_65_subgroup)

print(lower_bounds_mrnn_less_than_65_subgroup)
print(upper_bounds_mrnn_less_than_65_subgroup)

[np.float64(0.0), np.float64(0.5365028382858474), np.float64(0.41502977769515714), np.float64(0.6231741790597989), np.float64(0.4974875170875559)]
[np.float64(2.2237061316119653), np.float64(2.1451356410086992), np.float64(0.9571394705182901), np.float64(0.840094641091935), np.float64(0.7830983096643825)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_less_than_65_subgroup, upper_bounds_mrnn_less_than_65_subgroup)

print(mean_values_ci_mrnn_less_than_65_subgroup)

[np.float64(1.1118530658059826), np.float64(1.3408192396472733), np.float64(0.6860846241067236), np.float64(0.7316344100758669), np.float64(0.6402929133759692)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_mrnn_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_1_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_ICUType_1_subgroup, standards_deviations_mrnn_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_1_subgroup)

print(means_bootstraps_mrnn_ICUType_1_subgroup)
print(standards_deviations_mrnn_ICUType_1_subgroup)


[np.float64(0.5923263818868941), np.float64(0.8991498203303806), np.float64(1.2951781780888738), np.float64(0.38940273160830957), np.float64(0.7204738000835694)]
[np.float64(0.18517298169887855), np.float64(0.2406904990238948), np.float64(0.37899393156397077), np.float64(0.042177815695792834), np.float64(0.05230558266608225)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_ICUType_1_subgroup, upper_bounds_mrnn_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_1_subgroup, means_bootstraps_mrnn_ICUType_1_subgroup, standards_deviations_mrnn_ICUType_1_subgroup)

print(lower_bounds_mrnn_ICUType_1_subgroup)
print(upper_bounds_mrnn_ICUType_1_subgroup)

[np.float64(0.22939400684719763), np.float64(0.42740511082257376), np.float64(0.5523637218642531), np.float64(0.3067357318979874), np.float64(0.6179567418676657)]
[np.float64(0.9552587569265907), np.float64(1.3708945298381874), np.float64(2.0379926343134946), np.float64(0.4720697313186317), np.float64(0.8229908582994732)]


In [ ]:
lower_bounds_mrnn_ICUType_1_subgroup, upper_bounds_mrnn_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_1_subgroup)

print(lower_bounds_mrnn_ICUType_1_subgroup)
print(upper_bounds_mrnn_ICUType_1_subgroup)

[np.float64(0.33387672992511835), np.float64(0.4744406973511899), np.float64(0.6343524699589197), np.float64(0.3055874992951609), np.float64(0.61986295056262)]
[np.float64(1.0255384696106526), np.float64(1.4150713925130833), np.float64(2.1045419179083136), np.float64(0.4710435522679365), np.float64(0.8269107847530826)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_1_subgroup, upper_bounds_mrnn_ICUType_1_subgroup)

print(mean_values_ci_mrnn_ICUType_1_subgroup)

[np.float64(0.6797075997678855), np.float64(0.9447560449321366), np.float64(1.3694471939336166), np.float64(0.3883155257815487), np.float64(0.7233868676578512)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_mrnn_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_2_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_ICUType_2_subgroup, standards_deviations_mrnn_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_2_subgroup)

print(means_bootstraps_mrnn_ICUType_2_subgroup)
print(standards_deviations_mrnn_ICUType_2_subgroup) 


[np.float64(0.3911491147970715), np.float64(0.0), np.float64(0.40753655451843246), np.float64(0.5597474720403817), np.float64(0.5971345585631511)]
[np.float64(0.026120841089482264), np.float64(0.0), np.float64(0.13502306595614744), np.float64(0.021884859653926635), np.float64(0.022134890710794525)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_ICUType_2_subgroup, upper_bounds_mrnn_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_2_subgroup, means_bootstraps_mrnn_ICUType_2_subgroup, standards_deviations_mrnn_ICUType_2_subgroup)

print(lower_bounds_mrnn_ICUType_2_subgroup)
print(upper_bounds_mrnn_ICUType_2_subgroup)

[np.float64(0.33995320701579224), np.float64(0.0), np.float64(0.14289620816220722), np.float64(0.5168539353119718), np.float64(0.5537509699682637)]
[np.float64(0.44234502257835073), np.float64(0.0), np.float64(0.6721769008746576), np.float64(0.6026410087687916), np.float64(0.6405181471580386)]


In [ ]:
lower_bounds_mrnn_ICUType_2_subgroup, upper_bounds_mrnn_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_2_subgroup)

print(lower_bounds_mrnn_ICUType_2_subgroup)
print(upper_bounds_mrnn_ICUType_2_subgroup)

[np.float64(0.3431045463776856), np.float64(0.0), np.float64(0.09341967237713998), np.float64(0.515036131346664), np.float64(0.5546938441668393)]
[np.float64(0.4450448429779754), np.float64(0.0), np.float64(0.6781899601857575), np.float64(0.6012241978350398), np.float64(0.6413381337861924)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_2_subgroup, upper_bounds_mrnn_ICUType_2_subgroup)

print(mean_values_ci_mrnn_ICUType_2_subgroup)

[np.float64(0.3940746946778305), np.float64(0.0), np.float64(0.38580481628144875), np.float64(0.5581301645908519), np.float64(0.5980159889765159)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_mrnn_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_3_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_ICUType_3_subgroup, standards_deviations_mrnn_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_3_subgroup)

print(means_bootstraps_mrnn_ICUType_3_subgroup)
print(standards_deviations_mrnn_ICUType_3_subgroup)


[np.float64(1.719134934940246), np.float64(0.7148859869111546), np.float64(0.6705280051184919), np.float64(1.214910889467245), np.float64(1.0386339202179928)]
[np.float64(0.45599066158626456), np.float64(0.24937640245751141), np.float64(0.07494194033616502), np.float64(0.4409309876479161), np.float64(0.05313648672087894)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_ICUType_3_subgroup, upper_bounds_mrnn_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_3_subgroup, means_bootstraps_mrnn_ICUType_3_subgroup, standards_deviations_mrnn_ICUType_3_subgroup)

print(lower_bounds_mrnn_ICUType_3_subgroup)
print(upper_bounds_mrnn_ICUType_3_subgroup)

[np.float64(0.8254096609445756), np.float64(0.22611721950026636), np.float64(0.5236445011280588), np.float64(0.35070203400965394), np.float64(0.9344883199800793)]
[np.float64(2.6128602089359165), np.float64(1.2036547543220428), np.float64(0.8174115091089249), np.float64(2.079119744924836), np.float64(1.1427795204559064)]


In [ ]:
lower_bounds_mrnn_ICUType_3_subgroup, upper_bounds_mrnn_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_3_subgroup)

print(lower_bounds_mrnn_ICUType_3_subgroup)
print(upper_bounds_mrnn_ICUType_3_subgroup)

[np.float64(0.9073730753547186), np.float64(0.07943431525266052), np.float64(0.5018013901385182), np.float64(0.5220032075261235), np.float64(0.9362934830326102)]
[np.float64(2.68023466060627), np.float64(1.1458615561271466), np.float64(0.7919180977583171), np.float64(2.2313078354279243), np.float64(1.1467097315344643)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_3_subgroup, upper_bounds_mrnn_ICUType_3_subgroup)

print(mean_values_ci_mrnn_ICUType_3_subgroup)

[np.float64(1.7938038679804944), np.float64(0.6126479356899035), np.float64(0.6468597439484176), np.float64(1.376655521477024), np.float64(1.0415016072835372)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28, 31, 6, 0, 32, 24]


results_bootstrap_mrnn_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_4_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_ICUType_4_subgroup, standards_deviations_mrnn_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_4_subgroup)

print(means_bootstraps_mrnn_ICUType_4_subgroup)
print(standards_deviations_mrnn_ICUType_4_subgroup)


[np.float64(0.5143875969471539), np.float64(0.6932693072642694), np.float64(0.8103604095888144), np.float64(0.5926438887932957), np.float64(0.6708653384668115)]
[np.float64(0.1077200214987232), np.float64(0.2867206132388758), np.float64(0.1895512204845643), np.float64(0.06415554272726394), np.float64(0.02664062826735789)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_ICUType_4_subgroup, upper_bounds_mrnn_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_4_subgroup, means_bootstraps_mrnn_ICUType_4_subgroup, standards_deviations_mrnn_ICUType_4_subgroup)

print(lower_bounds_mrnn_ICUType_4_subgroup)
print(upper_bounds_mrnn_ICUType_4_subgroup)

[np.float64(0.30326023439577604), np.float64(0.13130723169083458), np.float64(0.43884684421345743), np.float64(0.46690133563923775), np.float64(0.6186506665372703)]
[np.float64(0.7255149594985317), np.float64(1.2552313828377042), np.float64(1.1818739749641713), np.float64(0.7183864419473536), np.float64(0.7230800103963527)]


In [ ]:
lower_bounds_mrnn_ICUType_4_subgroup, upper_bounds_mrnn_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_4_subgroup)

print(lower_bounds_mrnn_ICUType_4_subgroup)
print(upper_bounds_mrnn_ICUType_4_subgroup)

[np.float64(0.336414398103584), np.float64(0.23875262859924493), np.float64(0.5495089491374031), np.float64(0.474405403021104), np.float64(0.62041871440423)]
[np.float64(0.7511859659183101), np.float64(1.3382585193237966), np.float64(1.266807378181786), np.float64(0.7253036990143586), np.float64(0.7243081740543009)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_4_subgroup, upper_bounds_mrnn_ICUType_4_subgroup)

print(mean_values_ci_mrnn_ICUType_4_subgroup)

[np.float64(0.543800182010947), np.float64(0.7885055739615208), np.float64(0.9081581636595946), np.float64(0.5998545510177313), np.float64(0.6723634442292654)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_undefined_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_undefined_classification_subgroup, standards_deviations_mrnn_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_undefined_classification_subgroup)

print(means_bootstraps_mrnn_undefined_classification_subgroup)
print(standards_deviations_mrnn_undefined_classification_subgroup)


[np.float64(0.572329431448972), np.float64(0.5338853849284775), np.float64(0.7577229293241694), np.float64(0.7690223616774376), np.float64(0.48296232187961163)]
[np.float64(0.15863174449640172), np.float64(0.04803389244484797), np.float64(0.12507430978607165), np.float64(0.01346354958070844), np.float64(0.035705973066347)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_undefined_classification_subgroup, upper_bounds_mrnn_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_undefined_classification_subgroup, means_bootstraps_mrnn_undefined_classification_subgroup, standards_deviations_mrnn_undefined_classification_subgroup)

print(lower_bounds_mrnn_undefined_classification_subgroup)
print(upper_bounds_mrnn_undefined_classification_subgroup)

[np.float64(0.26141692543126466), np.float64(0.43974068569930486), np.float64(0.5125817867522633), np.float64(0.7426342893951797), np.float64(0.4129799006366143)]
[np.float64(0.8832419374666793), np.float64(0.6280300841576502), np.float64(1.0028640718960755), np.float64(0.7954104339596956), np.float64(0.552944743122609)]


In [ ]:
lower_bounds_mrnn_undefined_classification_subgroup, upper_bounds_mrnn_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_undefined_classification_subgroup)

print(lower_bounds_mrnn_undefined_classification_subgroup)
print(upper_bounds_mrnn_undefined_classification_subgroup)

[np.float64(0.28229290217266617), np.float64(0.44716250719586736), np.float64(0.5096508080497731), np.float64(0.7425805448717047), np.float64(0.42530560672235374)]
[np.float64(0.9009517424629762), np.float64(0.6355002898656733), np.float64(0.9861749079542917), np.float64(0.7961075136191128), np.float64(0.5636887749812975)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_undefined_classification_subgroup, upper_bounds_mrnn_undefined_classification_subgroup)

print(mean_values_ci_mrnn_undefined_classification_subgroup)

[np.float64(0.5916223223178212), np.float64(0.5413313985307704), np.float64(0.7479128580020324), np.float64(0.7693440292454088), np.float64(0.49449719085182564)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_low_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_low_weight_classification_subgroup, standards_deviations_mrnn_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_low_weight_classification_subgroup)

print(means_bootstraps_mrnn_low_weight_classification_subgroup)
print(standards_deviations_mrnn_low_weight_classification_subgroup)


[np.float64(0.06684271932828789), np.float64(1.9304242224835753), np.float64(0.0), np.float64(1.3239523724665192), np.float64(0.27513108924930096)]
[np.float64(0.05075023170537849), np.float64(1.4674221052022416), np.float64(0.0), np.float64(0.05118103352428714), np.float64(0.20904297686280882)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_low_weight_classification_subgroup, upper_bounds_mrnn_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_low_weight_classification_subgroup, means_bootstraps_mrnn_low_weight_classification_subgroup, standards_deviations_mrnn_low_weight_classification_subgroup)

print(lower_bounds_mrnn_low_weight_classification_subgroup)
print(upper_bounds_mrnn_low_weight_classification_subgroup)

[np.float64(-0.03262590702131672), np.float64(-0.9456702538307644), np.float64(0.0), np.float64(1.2236393900673792), np.float64(-0.13458561662284413)]
[np.float64(0.16631134567789252), np.float64(4.806518698797915), np.float64(0.0), np.float64(1.4242653548656592), np.float64(0.684847795121446)]


In [ ]:
lower_bounds_mrnn_low_weight_classification_subgroup, upper_bounds_mrnn_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_low_weight_classification_subgroup)

print(lower_bounds_mrnn_low_weight_classification_subgroup)
print(upper_bounds_mrnn_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(1.2212994888354451), np.float64(0.0)]
[np.float64(0.1053747545900948), np.float64(4.032034754392019), np.float64(0.0), np.float64(1.4225531788058816), np.float64(0.43396070859529773)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_low_weight_classification_subgroup, upper_bounds_mrnn_low_weight_classification_subgroup)

print(mean_values_ci_mrnn_low_weight_classification_subgroup)

[np.float64(0.0526873772950474), np.float64(2.0160173771960097), np.float64(0.0), np.float64(1.3219263338206635), np.float64(0.21698035429764886)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_normal_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_normal_weight_classification_subgroup, standards_deviations_mrnn_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_normal_weight_classification_subgroup)

print(means_bootstraps_mrnn_normal_weight_classification_subgroup)
print(standards_deviations_mrnn_normal_weight_classification_subgroup)


[np.float64(0.8260083154268454), np.float64(0.6437702877861539), np.float64(0.5700409295980765), np.float64(0.8089476168842691), np.float64(0.4269175173469857)]
[np.float64(0.3891522597320637), np.float64(0.09930904073266834), np.float64(0.11036754638049881), np.float64(0.013470317340671514), np.float64(0.04554262862294101)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_normal_weight_classification_subgroup, upper_bounds_mrnn_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_normal_weight_classification_subgroup, means_bootstraps_mrnn_normal_weight_classification_subgroup, standards_deviations_mrnn_normal_weight_classification_subgroup)

print(lower_bounds_mrnn_normal_weight_classification_subgroup)
print(upper_bounds_mrnn_normal_weight_classification_subgroup)

[np.float64(0.06328390184962385), np.float64(0.44912814461090267), np.float64(0.3537245136302448), np.float64(0.7825462800362275), np.float64(0.3376556054847383)]
[np.float64(1.588732729004067), np.float64(0.838412430961405), np.float64(0.7863573455659082), np.float64(0.8353489537323107), np.float64(0.5161794292092331)]


In [ ]:
lower_bounds_mrnn_normal_weight_classification_subgroup, upper_bounds_mrnn_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_normal_weight_classification_subgroup)

print(lower_bounds_mrnn_normal_weight_classification_subgroup)
print(upper_bounds_mrnn_normal_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.4837275708128196), np.float64(0.35059429410520515), np.float64(0.7824824951988859), np.float64(0.3402912382544941)]
[np.float64(1.2552017600298466), np.float64(0.8656249833560528), np.float64(0.7842600912740948), np.float64(0.8353857855788652), np.float64(0.520270504932545)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_normal_weight_classification_subgroup, upper_bounds_mrnn_normal_weight_classification_subgroup)

print(mean_values_ci_mrnn_normal_weight_classification_subgroup)

[np.float64(0.6276008800149233), np.float64(0.6746762770844361), np.float64(0.5674271926896499), np.float64(0.8089341403888756), np.float64(0.4302808715935196)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_overweight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_overweight_classification_subgroup, standards_deviations_mrnn_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_overweight_classification_subgroup)

print(means_bootstraps_mrnn_overweight_classification_subgroup)
print(standards_deviations_mrnn_overweight_classification_subgroup)


[np.float64(1.3044340682481634), np.float64(0.4608959351926642), np.float64(1.1729141749475005), np.float64(0.4185161733355873), np.float64(0.5654040243965782)]
[np.float64(0.2570976637522149), np.float64(0.04660768820178245), np.float64(0.44378480779330487), np.float64(0.009897569674754908), np.float64(0.14324296976206405)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_overweight_classification_subgroup, upper_bounds_mrnn_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_overweight_classification_subgroup, means_bootstraps_mrnn_overweight_classification_subgroup, standards_deviations_mrnn_overweight_classification_subgroup)

print(lower_bounds_mrnn_overweight_classification_subgroup)
print(upper_bounds_mrnn_overweight_classification_subgroup)

[np.float64(0.8005319067844333), np.float64(0.3695465449144982), np.float64(0.3031119347865927), np.float64(0.3991172932385919), np.float64(0.2846529626243727)]
[np.float64(1.8083362297118937), np.float64(0.5522453254708302), np.float64(2.0427164151084085), np.float64(0.43791505343258275), np.float64(0.8461550861687838)]


In [ ]:
lower_bounds_mrnn_overweight_classification_subgroup, upper_bounds_mrnn_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_overweight_classification_subgroup)

print(lower_bounds_mrnn_overweight_classification_subgroup)
print(upper_bounds_mrnn_overweight_classification_subgroup)

[np.float64(0.7647994579647753), np.float64(0.37720814141798864), np.float64(0.5996947070542901), np.float64(0.399127532664659), np.float64(0.3506072844284098)]
[np.float64(1.7839039553895528), np.float64(0.5606294766263048), np.float64(2.1804001127524586), np.float64(0.43806154177361717), np.float64(0.8982687379324473)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_overweight_classification_subgroup, upper_bounds_mrnn_overweight_classification_subgroup)

print(mean_values_ci_mrnn_overweight_classification_subgroup)

[np.float64(1.2743517066771641), np.float64(0.4689188090221467), np.float64(1.3900474099033744), np.float64(0.4185945372191381), np.float64(0.6244380111804286)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_obs1_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_obs1_classification_subgroup, standards_deviations_mrnn_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_obs1_classification_subgroup)

print(means_bootstraps_mrnn_obs1_classification_subgroup)
print(standards_deviations_mrnn_obs1_classification_subgroup)


[np.float64(0.5922894090697942), np.float64(0.42277013572667876), np.float64(0.5449478317670913), np.float64(0.54102718014115), np.float64(0.3353121840047957)]
[np.float64(0.30717694548751795), np.float64(0.0421134616419317), np.float64(0.2707550703154163), np.float64(0.018891426276303334), np.float64(0.0519305772003826)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_obs1_classification_subgroup, upper_bounds_mrnn_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_obs1_classification_subgroup, means_bootstraps_mrnn_obs1_classification_subgroup, standards_deviations_mrnn_obs1_classification_subgroup)

print(lower_bounds_mrnn_obs1_classification_subgroup)
print(upper_bounds_mrnn_obs1_classification_subgroup)

[np.float64(-0.00976634096676443), np.float64(0.3402292676441836), np.float64(0.014277645317265497), np.float64(0.5040006650230019), np.float64(0.23353012299566892)]
[np.float64(1.1943451591063527), np.float64(0.5053110038091739), np.float64(1.075618018216917), np.float64(0.5780536952592982), np.float64(0.43709424501392247)]


In [ ]:
lower_bounds_mrnn_obs1_classification_subgroup, upper_bounds_mrnn_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_obs1_classification_subgroup)

print(lower_bounds_mrnn_obs1_classification_subgroup)
print(upper_bounds_mrnn_obs1_classification_subgroup)

[np.float64(0.0), np.float64(0.3378037893951755), np.float64(0.0), np.float64(0.5045751283911896), np.float64(0.22956230525565455)]
[np.float64(1.218276945248262), np.float64(0.5043589421016695), np.float64(0.8630806178451833), np.float64(0.5783155333051042), np.float64(0.43639249642263145)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_obs1_classification_subgroup, upper_bounds_mrnn_obs1_classification_subgroup)

print(mean_values_ci_mrnn_obs1_classification_subgroup)

[np.float64(0.609138472624131), np.float64(0.42108136574842253), np.float64(0.43154030892259165), np.float64(0.5414453308481468), np.float64(0.332977400839143)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]
results_bootstrap_mrnn_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_obs2_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_obs2_classification_subgroup, standards_deviations_mrnn_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_obs2_classification_subgroup)

print(means_bootstraps_mrnn_obs2_classification_subgroup)
print(standards_deviations_mrnn_obs2_classification_subgroup) 


[np.float64(0.05092602899904994), np.float64(0.5603368612340359), np.float64(0.0), np.float64(0.7943294337661126), np.float64(1.3468289760835621)]
[np.float64(0.039521971437981245), np.float64(0.0518376155060004), np.float64(0.0), np.float64(0.042958475889205795), np.float64(0.9053853640841055)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_obs2_classification_subgroup, upper_bounds_mrnn_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_obs2_classification_subgroup, means_bootstraps_mrnn_obs2_classification_subgroup, standards_deviations_mrnn_obs2_classification_subgroup)

print(lower_bounds_mrnn_obs2_classification_subgroup)
print(upper_bounds_mrnn_obs2_classification_subgroup)

[np.float64(-0.026535611617413996), np.float64(0.4587370017978401), np.float64(0.0), np.float64(0.710132368192537), np.float64(-0.42769372965096886)]
[np.float64(0.12838766961551387), np.float64(0.6619367206702318), np.float64(0.0), np.float64(0.8785264993396883), np.float64(3.1213516818180933)]


In [ ]:
lower_bounds_mrnn_obs2_classification_subgroup, upper_bounds_mrnn_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_obs2_classification_subgroup)

print(lower_bounds_mrnn_obs2_classification_subgroup)
print(upper_bounds_mrnn_obs2_classification_subgroup)

[np.float64(0.0), np.float64(0.46379886105351265), np.float64(0.0), np.float64(0.7117601050373132), np.float64(0.0)]
[np.float64(0.08159769645569703), np.float64(0.6676735252252052), np.float64(0.0), np.float64(0.8800511608039773), np.float64(3.2940803837237835)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_obs2_classification_subgroup, upper_bounds_mrnn_obs2_classification_subgroup)

print(mean_values_ci_mrnn_obs2_classification_subgroup)

[np.float64(0.040798848227848517), np.float64(0.565736193139359), np.float64(0.0), np.float64(0.7959056329206453), np.float64(1.6470401918618918)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_obs3_classification_subgroup, standards_deviations_mrnn_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_obs3_classification_subgroup)

print(means_bootstraps_mrnn_obs3_classification_subgroup)
print(standards_deviations_mrnn_obs3_classification_subgroup)


[np.float64(0.0), np.float64(0.3479959658033868), np.float64(0.22290367635847794), np.float64(1.6741620128430856), np.float64(0.9844822311929602)]
[np.float64(0.0), np.float64(0.041627417743106174), np.float64(0.16846992178063286), np.float64(0.06480046781619947), np.float64(0.7275366689643368)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_obs3_classification_subgroup, means_bootstraps_mrnn_obs3_classification_subgroup, standards_deviations_mrnn_obs3_classification_subgroup)

print(lower_bounds_mrnn_obs3_classification_subgroup)
print(upper_bounds_mrnn_obs3_classification_subgroup)

[np.float64(0.0), np.float64(0.2664077262574951), np.float64(-0.10729130280984248), np.float64(1.5471554297419878), np.float64(-0.44146343740937954)]
[np.float64(0.0), np.float64(0.42958420534927855), np.float64(0.5530986555267984), np.float64(1.8011685959441834), np.float64(2.4104278997953)]


In [ ]:
lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_obs3_classification_subgroup)

print(lower_bounds_mrnn_obs3_classification_subgroup)
print(upper_bounds_mrnn_obs3_classification_subgroup)

[np.float64(0.0), np.float64(0.2709265002813849), np.float64(0.0), np.float64(1.5491380123564265), np.float64(0.11034801378358251)]
[np.float64(0.0), np.float64(0.434881553501857), np.float64(0.3502327317086535), np.float64(1.8043899881623382), np.float64(2.6597921463168905)]


#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup)

print(mean_values_ci_mrnn_obs3_classification_subgroup)

[np.float64(0.0), np.float64(0.3529040268916209), np.float64(0.17511636585432674), np.float64(1.6767640002593822), np.float64(1.3850700800502365)]


# Results mean absolute error

In [17]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

#### SAITS

<h4>Standard Scaler (C/Normalização)</h4>

In [13]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 0.35549184664706085
ALT : 0.34616397235246166
AST : 0.283203929613111
Albumin : 0.5865378684716107
BUN : 0.23549456829210566
Bilirubin : 0.29213390018033225
Cholesterol : 0.8619142944054452
Creatinine : 0.2272360108358054
DiasABP : 0.25696352027850733
FiO2 : 0.4843457493583031
GCS : 0.27782594851598286
Glucose : 0.5574566141755575
HCO3 : 0.42235495223366226
HCT : 0.5478476366595461
HR : 0.2786515469045242
K : 0.6173738181988982
Lactate : 0.3602081488093162
MAP : 0.276148040474521
MechVent : 0.016131509283529688
Mg : 0.5428868319523208
NIDiasABP : 0.23805346595387422
NIMAP : 0.17731126967659114
NISysABP : 0.3059993623684565
Na : 0.5094466185029012
PaCO2 : 0.5448751678244528
PaO2 : 0.5044175503727509
Platelets : 0.37910753219049526
RespRate : 0.4975549864022565
SaO2 : 0.395203639666887
SysABP : 0.2876315375696396
Temp : 0.3654739421516718
TroponinI : 0.5742820858879293
TroponinT : 0.562502744207675
Urine : 0.4555801478350003
WBC : 0.15

#### BRITS

<h4>Standard Scaler (C/Normalização)</h4>

In [47]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard, subgroups, variables)

BRITS - MAE
************
General
-------------
ALP : 0.4591248631105258
ALT : 0.2098298479270722
AST : 0.27798859929904113
Albumin : 0.6778925772529109
BUN : 0.39521960833661646
Bilirubin : 0.3833806392125974
Cholesterol : 0.9463970561439892
Creatinine : 0.3444718062995419
DiasABP : 0.20806233248448402
FiO2 : 0.34404522156046397
GCS : 0.21322463397612945
Glucose : 0.5644091100017328
HCO3 : 0.5047606820899914
HCT : 0.6338702223060172
HR : 0.2431173715923441
K : 0.6534995938478387
Lactate : 0.4167839664888235
MAP : 0.1564062923159312
MechVent : 0.007398844359593802
Mg : 0.5955810654854342
NIDiasABP : 0.15725503986364742
NIMAP : 0.09992748699932849
NISysABP : 0.2543230216923866
Na : 0.645442078485974
PaCO2 : 1.1895743136362507
PaO2 : 0.4776378612780617
Platelets : 0.6445916196510731
RespRate : 0.475295251812804
SaO2 : 0.348066606252948
SysABP : 0.25398352012962805
Temp : 0.240561647460938
TroponinI : 0.667214116433886
TroponinT : 0.29949287221387894
Urine : 0.3333883323992429
WBC : 0.1101